In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [ ]:
# %load NEW02OT-MakeConcordance-Exodus.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *


Exodus01 = [
    ['Exodus','Exo','01','01','1',"Now these \emph{are} the names of the children of Israel, which came into Egypt; every man and his household came with Jacob."],
    ['Exodus','Exo','01','02','2',"Reuben, Simeon, Levi, and Judah,"],
    ['Exodus','Exo','01','03','3',"Issachar, Zebulun, and Benjamin,"],
    ['Exodus','Exo','01','04','4',"Dan, and Naphtali, Gad, and Asher."],
    ['Exodus','Exo','01','05','5',"And all the souls that came out of the loins of Jacob were seventy souls: for Joseph was in Egypt \emph{already}."],
    ['Exodus','Exo','01','06','6',"And Joseph died, and all his brethren, and all that generation."],
    ['Exodus','Exo','01','07','7',"And the children of Israel were fruitful, and increased abundantly, and multiplied, and waxed exceeding mighty; and the land was filled with them."],
    ['Exodus','Exo','01','08','8',"Now there arose up a new king over Egypt, which knew not Joseph."],
    ['Exodus','Exo','01','09','9',"And he said unto his people, Behold, the people of the children of Israel \emph{are} more and mightier than we:"],
    ['Exodus','Exo','01','10','10',"Come on, let us deal wisely with them; lest they multiply, and it come to pass, that, when there falleth out any war, they join also unto our enemies, and fight against us, and \emph{so} get them up out of the land."],
    ['Exodus','Exo','01','11','11',"Therefore they did set over them taskmasters to afflict them with their burdens. And they built for Pharaoh treasure cities, Pithom and Raamses."],
    ['Exodus','Exo','01','12','12',"But the more they afflicted them, the more they multiplied and grew. And they were grieved because of the children of Israel."],
    ['Exodus','Exo','01','13','13',"And the Egyptians made the children of Israel to serve with rigour:"],
    ['Exodus','Exo','01','14','14',"And they made their lives bitter with hard bondage, in morter, and in brick, and in all manner of service in the field: all their service, wherein they made them serve, \emph{was} with rigour."],
    ['Exodus','Exo','01','15','15',"And the king of Egypt spake to the Hebrew midwives, of which the name of the one \emph{was} Shiphrah, and the name of the other Puah:"],
    ['Exodus','Exo','01','16','16',"And he said, When ye do the office of a midwife to the Hebrew women, and see \emph{them} upon the stools; if it \emph{be} a son, then ye shall kill him: but if it \emph{be} a daughter, then she shall live."],
    ['Exodus','Exo','01','17','17',"But the midwives feared God, and did not as the king of Egypt commanded them, but saved the men children alive."],
    ['Exodus','Exo','01','18','18',"And the king of Egypt called for the midwives, and said unto them, Why have ye done this thing, and have saved the men children alive?"],
    ['Exodus','Exo','01','19','19',"And the midwives said unto Pharaoh, Because the Hebrew women \emph{are} not as the Egyptian women; for they \emph{are} lively, and are delivered ere the midwives come in unto them."],
    ['Exodus','Exo','01','20','20',"Therefore God dealt well with the midwives: and the people multiplied, and waxed very mighty."],
    ['Exodus','Exo','01','21','21',"And it came to pass, because the midwives feared God, that he made them houses."],
    ['Exodus','Exo','01','22','22',"And Pharaoh charged all his people, saying, Every son that is born ye shall cast into the river, and every daughter ye shall save alive."]]  

Exodus02 = [
    ['Exodus','Exo','02','001','1',"And there went a man of the house of Levi, and took \emph{to} \emph{wife} a daughter of Levi."],
['Exodus','Exo','02','002','2',"And the woman conceived, and bare a son: and when she saw him that he \emph{was} \emph{a} goodly \emph{child}, she hid him three months."],
['Exodus','Exo','02','003','3',"And when she could not longer hide him, she took for him an ark of bulrushes, and daubed it with slime and with pitch, and put the child therein; and she laid \emph{it} in the flags by the river's brink."],
['Exodus','Exo','02','004','4',"And his sister stood afar off, to wit what would be done to him."],
['Exodus','Exo','02','005','5',"And the daughter of Pharaoh came down to wash \emph{herself} at the river; and her maidens walked along by the river's side; and when she saw the ark among the flags, she sent her maid to fetch it."],
['Exodus','Exo','02','006','6',"And when she had opened \emph{it}, she saw the child: and, behold, the babe wept. And she had compassion on him, and said, This \emph{is} \emph{one} of the Hebrews' children."],
['Exodus','Exo','02','007','7',"Then said his sister to Pharaoh's daughter, Shall I go and call to thee a nurse of the Hebrew women, that she may nurse the child for thee?"],
['Exodus','Exo','02','008','8',"And Pharaoh's daughter said to her, Go. And the maid went and called the child's mother."],
['Exodus','Exo','02','009','9',"And Pharaoh's daughter said unto her, Take this child away, and nurse it for me, and I will give \emph{thee} thy wages. And the woman took the child, and nursed it."],
['Exodus','Exo','02','010','10',"And the child grew, and she brought him unto Pharaoh's daughter, and he became her son. And she called his name Moses: and she said, Because I drew him out of the water."],
['Exodus','Exo','02','011','11',"And it came to pass in those days, when Moses was grown, that he went out unto his brethren, and looked on their burdens: and he spied an Egyptian smiting an Hebrew, one of his brethren."],
['Exodus','Exo','02','012','12',"And he looked this way and that way, and when he saw that \emph{there} \emph{was} no man, he slew the Egyptian, and hid him in the sand."],
['Exodus','Exo','02','013','13',"And when he went out the second day, behold, two men of the Hebrews strove together: and he said to him that did the wrong, Wherefore smitest thou thy fellow?"],
['Exodus','Exo','02','014','14',"And he said, Who made thee a prince and a judge over us? intendest thou to kill me, as thou killedst the Egyptian? And Moses feared, and said, Surely this thing is known."],
['Exodus','Exo','02','015','15',"Now when Pharaoh heard this thing, he sought to slay Moses. But Moses fled from the face of Pharaoh, and dwelt in the land of Midian: and he sat down by a well."],
['Exodus','Exo','02','016','16',"Now the priest of Midian had seven daughters: and they came and drew \emph{water}, and filled the troughs to water their father's flock."],
['Exodus','Exo','02','017','17',"And the shepherds came and drove them away: but Moses stood up and helped them, and watered their flock."],
['Exodus','Exo','02','018','18',"And when they came to Reuel their father, he said, How \emph{is} \emph{it} \emph{that} ye are come so soon to day?"],
['Exodus','Exo','02','019','19',"And they said, An Egyptian delivered us out of the hand of the shepherds, and also drew \emph{water} enough for us, and watered the flock."],
['Exodus','Exo','02','020','20',"And he said unto his daughters, And where \emph{is} he? why \emph{is} it \emph{that} ye have left the man? call him, that he may eat bread."],
['Exodus','Exo','02','021','21',"And Moses was content to dwell with the man: and he gave Moses Zipporah his daughter."],
['Exodus','Exo','02','022','22',"And she bare \emph{him} a son, and he called his name Gershom: for he said, I have been a stranger in a strange land."],
['Exodus','Exo','02','023','23',"And it came to pass in process of time, that the king of Egypt died: and the children of Israel sighed by reason of the bondage, and they cried, and their cry came up unto God by reason of the bondage."],
['Exodus','Exo','02','024','24',"And God heard their groaning, and God remembered his covenant with Abraham, with Isaac, and with Jacob."],
['Exodus','Exo','02','025','25',"And God looked upon the children of Israel, and God had respect unto \emph{them}."]
]

Exodus03 = [
    ['Exodus','Exo','03','001','1',"Now Moses kept the flock of Jethro his father in law, the priest of Midian: and he led the flock to the backside of the desert, and came to the mountain of God, \emph{even} to Horeb."],
['Exodus','Exo','03','002','2',"And the angel of the LORD appeared unto him in a flame of fire out of the midst of a bush: and he looked, and, behold, the bush burned with fire, and the bush \emph{was} not consumed."],
['Exodus','Exo','03','003','3',"And Moses said, I will now turn aside, and see this great sight, why the bush is not burnt."],
['Exodus','Exo','03','004','4',"And when the LORD saw that he turned aside to see, God called unto him out of the midst of the bush, and said, Moses, Moses. And he said, Here \emph{am} I."],
['Exodus','Exo','03','005','5',"And he said, Draw not nigh hither: put off thy shoes from off thy feet, for the place whereon thou standest \emph{is} holy ground."],
['Exodus','Exo','03','006','6',"Moreover he said, I \emph{am} the God of thy father, the God of Abraham, the God of Isaac, and the God of Jacob. And Moses hid his face; for he was afraid to look upon God."],
['Exodus','Exo','03','007','7',"And the LORD said, I have surely seen the affliction of my people which \emph{are} in Egypt, and have heard their cry by reason of their taskmasters; for I know their sorrows;"],
['Exodus','Exo','03','008','8',"And I am come down to deliver them out of the hand of the Egyptians, and to bring them up out of that land unto a good land and a large, unto a land flowing with milk and honey; unto the place of the Canaanites, and the Hittites, and the Amorites, and the Perizzites, and the Hivites, and the Jebusites."],
['Exodus','Exo','03','009','9',"Now therefore, behold, the cry of the children of Israel is come unto me: and I have also seen the oppression wherewith the Egyptians oppress them."],
['Exodus','Exo','03','010','10',"Come now therefore, and I will send thee unto Pharaoh, that thou mayest bring forth my people the children of Israel out of Egypt."],
['Exodus','Exo','03','011','11',"And Moses said unto God, Who \emph{am} I, that I should go unto Pharaoh, and that I should bring forth the children of Israel out of Egypt?"],
['Exodus','Exo','03','012','12',"And he said, Certainly I will be with thee; and this \emph{shall} \emph{be} a token unto thee, that I have sent thee: When thou hast brought forth the people out of Egypt, ye shall serve God upon this mountain."],
['Exodus','Exo','03','013','13',"And Moses said unto God, Behold, \emph{when} I come unto the children of Israel, and shall say unto them, The God of your fathers hath sent me unto you; and they shall say to me, What \emph{is} his name? what shall I say unto them?"],
['Exodus','Exo','03','014','14',"And God said unto Moses, I AM THAT I AM: and he said, Thus shalt thou say unto the children of Israel, I AM hath sent me unto you."],
['Exodus','Exo','03','015','15',"And God said moreover unto Moses, Thus shalt thou say unto the children of Israel, The LORD God of your fathers, the God of Abraham, the God of Isaac, and the God of Jacob, hath sent me unto you: this \emph{is} my name for ever, and this \emph{is} my memorial unto all generations."],
['Exodus','Exo','03','016','16',"Go, and gather the elders of Israel together, and say unto them, The LORD God of your fathers, the God of Abraham, of Isaac, and of Jacob, appeared unto me, saying, I have surely visited you, and \emph{seen} that which is done to you in Egypt:"],
['Exodus','Exo','03','017','17',"And I have said, I will bring you up out of the affliction of Egypt unto the land of the Canaanites, and the Hittites, and the Amorites, and the Perizzites, and the Hivites, and the Jebusites, unto a land flowing with milk and honey."],
['Exodus','Exo','03','018','18',"And they shall hearken to thy voice: and thou shalt come, thou and the elders of Israel, unto the king of Egypt, and ye shall say unto him, The LORD God of the Hebrews hath met with us: and now let us go, we beseech thee, three days' journey into the wilderness, that we may sacrifice to the LORD our God."],
['Exodus','Exo','03','019','19',"And I am sure that the king of Egypt will not let you go, no, not by a mighty hand."],
['Exodus','Exo','03','020','20',"And I will stretch out my hand, and smite Egypt with all my wonders which I will do in the midst thereof: and after that he will let you go."],
['Exodus','Exo','03','021','21',"And I will give this people favour in the sight of the Egyptians: and it shall come to pass, that, when ye go, ye shall not go empty:"],
['Exodus','Exo','03','022','22',"But every woman shall borrow of her neighbour, and of her that sojourneth in her house, jewels of silver, and jewels of gold, and raiment: and ye shall put \emph{them} upon your sons, and upon your daughters; and ye shall spoil the Egyptians."]
]

Exodus04 = [
    ['Exodus','Exo','04','001','1',"And Moses answered and said, But, behold, they will not believe me, nor hearken unto my voice: for they will say, The LORD hath not appeared unto thee."],
['Exodus','Exo','04','002','2',"And the LORD said unto him, What \emph{is} that in thine hand? And he said, A rod."],
['Exodus','Exo','04','003','3',"And he said, Cast it on the ground. And he cast it on the ground, and it became a serpent; and Moses fled from before it."],
['Exodus','Exo','04','004','4',"And the LORD said unto Moses, Put forth thine hand, and take it by the tail. And he put forth his hand, and caught it, and it became a rod in his hand:"],
['Exodus','Exo','04','005','5',"That they may believe that the LORD God of their fathers, the God of Abraham, the God of Isaac, and the God of Jacob, hath appeared unto thee."],
['Exodus','Exo','04','006','6',"And the LORD said furthermore unto him, Put now thine hand into thy bosom. And he put his hand into his bosom: and when he took it out, behold, his hand \emph{was} leprous as snow."],
['Exodus','Exo','04','007','7',"And he said, Put thine hand into thy bosom again. And he put his hand into his bosom again; and plucked it out of his bosom, and, behold, it was turned again as his \emph{other} flesh."],
['Exodus','Exo','04','008','8',"And it shall come to pass, if they will not believe thee, neither hearken to the voice of the first sign, that they will believe the voice of the latter sign."],
['Exodus','Exo','04','009','9',"And it shall come to pass, if they will not believe also these two signs, neither hearken unto thy voice, that thou shalt take of the water of the river, and pour \emph{it} upon the dry \emph{land}: and the water which thou takest out of the river shall become blood upon the dry \emph{land}."],
['Exodus','Exo','04','010','10',"And Moses said unto the LORD, O my Lord, I \emph{am} not eloquent, neither heretofore, nor since thou hast spoken unto thy servant: but I \emph{am} slow of speech, and of a slow tongue."],
['Exodus','Exo','04','011','11',"And the LORD said unto him, Who hath made man's mouth? or who maketh the dumb, or deaf, or the seeing, or the blind? have not I the LORD?"],
['Exodus','Exo','04','012','12',"Now therefore go, and I will be with thy mouth, and teach thee what thou shalt say."],
['Exodus','Exo','04','013','13',"And he said, O my Lord, send, I pray thee, by the hand \emph{of} \emph{him} \emph{whom} thou wilt send."],
['Exodus','Exo','04','014','14',"And the anger of the LORD was kindled against Moses, and he said, \emph{Is} not Aaron the Levite thy brother? I know that he can speak well. And also, behold, he cometh forth to meet thee: and when he seeth thee, he will be glad in his heart."],
['Exodus','Exo','04','015','15',"And thou shalt speak unto him, and put words in his mouth: and I will be with thy mouth, and with his mouth, and will teach you what ye shall do."],
['Exodus','Exo','04','016','16',"And he shall be thy spokesman unto the people: and he shall be, \emph{even} he shall be to thee instead of a mouth, and thou shalt be to him instead of God."],
['Exodus','Exo','04','017','17',"And thou shalt take this rod in thine hand, wherewith thou shalt do signs."],
['Exodus','Exo','04','018','18',"And Moses went and returned to Jethro his father in law, and said unto him, Let me go, I pray thee, and return unto my brethren which \emph{are} in Egypt, and see whether they be yet alive. And Jethro said to Moses, Go in peace."],
['Exodus','Exo','04','019','19',"And the LORD said unto Moses in Midian, Go, return into Egypt: for all the men are dead which sought thy life."],
['Exodus','Exo','04','020','20',"And Moses took his wife and his sons, and set them upon an ass, and he returned to the land of Egypt: and Moses took the rod of God in his hand."],
['Exodus','Exo','04','021','21',"And the LORD said unto Moses, When thou goest to return into Egypt, see that thou do all those wonders before Pharaoh, which I have put in thine hand: but I will harden his heart, that he shall not let the people go."],
['Exodus','Exo','04','022','22',"And thou shalt say unto Pharaoh, Thus saith the LORD, Israel \emph{is} my son, \emph{even} my firstborn:"],
['Exodus','Exo','04','023','23',"And I say unto thee, Let my son go, that he may serve me: and if thou refuse to let him go, behold, I will slay thy son, \emph{even} thy firstborn."],
['Exodus','Exo','04','024','24',"And it came to pass by the way in the inn, that the LORD met him, and sought to kill him."],
['Exodus','Exo','04','025','25',"Then Zipporah took a sharp stone, and cut off the foreskin of her son, and cast \emph{it} at his feet, and said, Surely a bloody husband \emph{art} thou to me."],
['Exodus','Exo','04','026','26',"So he let him go: then she said, A bloody husband \emph{thou} \emph{art}, because of the circumcision."],
['Exodus','Exo','04','027','27',"And the LORD said to Aaron, Go into the wilderness to meet Moses. And he went, and met him in the mount of God, and kissed him."],
['Exodus','Exo','04','028','28',"And Moses told Aaron all the words of the LORD who had sent him, and all the signs which he had commanded him."],
['Exodus','Exo','04','029','29',"And Moses and Aaron went and gathered together all the elders of the children of Israel:"],
['Exodus','Exo','04','030','30',"And Aaron spake all the words which the LORD had spoken unto Moses, and did the signs in the sight of the people."],
['Exodus','Exo','04','031','31',"And the people believed: and when they heard that the LORD had visited the children of Israel, and that he had looked upon their affliction, then they bowed their heads and worshipped."]
    ]
    
Exodus05 = [
    ['Exodus','Exo','05','001','1',"And afterward Moses and Aaron went in, and told Pharaoh, Thus saith the LORD God of Israel, Let my people go, that they may hold a feast unto me in the wilderness."],
['Exodus','Exo','05','002','2',"And Pharaoh said, Who \emph{is} the LORD, that I should obey his voice to let Israel go? I know not the LORD, neither will I let Israel go."],
['Exodus','Exo','05','003','3',"And they said, The God of the Hebrews hath met with us: let us go, we pray thee, three days' journey into the desert, and sacrifice unto the LORD our God; lest he fall upon us with pestilence, or with the sword."],
['Exodus','Exo','05','004','4',"And the king of Egypt said unto them, Wherefore do ye, Moses and Aaron, let the people from their works? get you unto your burdens."],
['Exodus','Exo','05','005','5',"And Pharaoh said, Behold, the people of the land now \emph{are} many, and ye make them rest from their burdens."],
['Exodus','Exo','05','006','6',"And Pharaoh commanded the same day the taskmasters of the people, and their officers, saying,"],
['Exodus','Exo','05','007','7',"Ye shall no more give the people straw to make brick, as heretofore: let them go and gather straw for themselves."],
['Exodus','Exo','05','008','8',"And the tale of the bricks, which they did make heretofore, ye shall lay upon them; ye shall not diminish \emph{ought} thereof: for they \emph{be} idle; therefore they cry, saying, Let us go \emph{and} sacrifice to our God."],
['Exodus','Exo','05','009','9',"Let there more work be laid upon the men, that they may labour therein; and let them not regard vain words."],
['Exodus','Exo','05','010','10',"And the taskmasters of the people went out, and their officers, and they spake to the people, saying, Thus saith Pharaoh, I will not give you straw."],
['Exodus','Exo','05','011','11',"Go ye, get you straw where ye can find it: yet not ought of your work shall be diminished."],
['Exodus','Exo','05','012','12',"So the people were scattered abroad throughout all the land of Egypt to gather stubble instead of straw."],
['Exodus','Exo','05','013','13',"And the taskmasters hasted \emph{them}, saying, Fulfil your works, \emph{your} daily tasks, as when there was straw."],
['Exodus','Exo','05','014','14',"And the officers of the children of Israel, which Pharaoh's taskmasters had set over them, were beaten, \emph{and} demanded, Wherefore have ye not fulfilled your task in making brick both yesterday and to day, as heretofore?"],
['Exodus','Exo','05','015','15',"Then the officers of the children of Israel came and cried unto Pharaoh, saying, Wherefore dealest thou thus with thy servants?"],
['Exodus','Exo','05','016','16',"There is no straw given unto thy servants, and they say to us, Make brick: and, behold, thy servants \emph{are} beaten; but the fault \emph{is} in thine own people."],
['Exodus','Exo','05','017','17',"But he said, Ye \emph{are} idle, \emph{ye} \emph{are} idle: therefore ye say, Let us go \emph{and} do sacrifice to the LORD."],
['Exodus','Exo','05','018','18',"Go therefore now, \emph{and} work; for there shall no straw be given you, yet shall ye deliver the tale of bricks."],
['Exodus','Exo','05','019','19',"And the officers of the children of Israel did see \emph{that} they \emph{were} in evil \emph{case}, after it was said, Ye shall not minish \emph{ought} from your bricks of your daily task."],
['Exodus','Exo','05','020','20',"And they met Moses and Aaron, who stood in the way, as they came forth from Pharaoh:"],
['Exodus','Exo','05','021','21',"And they said unto them, The LORD look upon you, and judge; because ye have made our savour to be abhorred in the eyes of Pharaoh, and in the eyes of his servants, to put a sword in their hand to slay us."],
['Exodus','Exo','05','022','22',"And Moses returned unto the LORD, and said, Lord, wherefore hast thou \emph{so} evil entreated this people? why \emph{is} it \emph{that} thou hast sent me?"],
['Exodus','Exo','05','023','23',"For since I came to Pharaoh to speak in thy name, he hath done evil to this people; neither hast thou delivered thy people at all."]
]
    
Exodus06 = [
    ['Exodus','Exo','06','001','1',"Then the LORD said unto Moses, Now shalt thou see what I will do to Pharaoh: for with a strong hand shall he let them go, and with a strong hand shall he drive them out of his land."],
['Exodus','Exo','06','002','2',"And God spake unto Moses, and said unto him, I \emph{am} the LORD:"],
['Exodus','Exo','06','003','3',"And I appeared unto Abraham, unto Isaac, and unto Jacob, by \emph{the} \emph{name} \emph{of} God Almighty, but by my name JEHOVAH was I not known to them."],
['Exodus','Exo','06','004','4',"And I have also established my covenant with them, to give them the land of Canaan, the land of their pilgrimage, wherein they were strangers."],
['Exodus','Exo','06','005','5',"And I have also heard the groaning of the children of Israel, whom the Egyptians keep in bondage; and I have remembered my covenant."],
['Exodus','Exo','06','006','6',"Wherefore say unto the children of Israel, I \emph{am} the LORD, and I will bring you out from under the burdens of the Egyptians, and I will rid you out of their bondage, and I will redeem you with a stretched out arm, and with great judgments:"],
['Exodus','Exo','06','007','7',"And I will take you to me for a people, and I will be to you a God: and ye shall know that I \emph{am} the LORD your God, which bringeth you out from under the burdens of the Egyptians."],
['Exodus','Exo','06','008','8',"And I will bring you in unto the land, concerning the which I did swear to give it to Abraham, to Isaac, and to Jacob; and I will give it you for an heritage: I \emph{am} the LORD."],
['Exodus','Exo','06','009','9',"And Moses spake so unto the children of Israel: but they hearkened not unto Moses for anguish of spirit, and for cruel bondage."],
['Exodus','Exo','06','010','10',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','06','011','11',"Go in, speak unto Pharaoh king of Egypt, that he let the children of Israel go out of his land."],
['Exodus','Exo','06','012','12',"And Moses spake before the LORD, saying, Behold, the children of Israel have not hearkened unto me; how then shall Pharaoh hear me, who \emph{am} of uncircumcised lips?"],
['Exodus','Exo','06','013','13',"And the LORD spake unto Moses and unto Aaron, and gave them a charge unto the children of Israel, and unto Pharaoh king of Egypt, to bring the children of Israel out of the land of Egypt."],
['Exodus','Exo','06','014','14',"These \emph{be} the heads of their fathers' houses: The sons of Reuben the firstborn of Israel; Hanoch, and Pallu, Hezron, and Carmi: these \emph{be} the families of Reuben."],
['Exodus','Exo','06','015','15',"And the sons of Simeon; Jemuel, and Jamin, and Ohad, and Jachin, and Zohar, and Shaul the son of a Canaanitish woman: these \emph{are} the families of Simeon."],
['Exodus','Exo','06','016','16',"And these \emph{are} the names of the sons of Levi according to their generations; Gershon, and Kohath, and Merari: and the years of the life of Levi \emph{were} an hundred thirty and seven years."],
['Exodus','Exo','06','017','17',"The sons of Gershon; Libni, and Shimi, according to their families."],
['Exodus','Exo','06','018','18',"And the sons of Kohath; Amram, and Izhar, and Hebron, and Uzziel: and the years of the life of Kohath \emph{were} an hundred thirty and three years."],
['Exodus','Exo','06','019','19',"And the sons of Merari; Mahali and Mushi: these \emph{are} the families of Levi according to their generations."],
['Exodus','Exo','06','020','20',"And Amram took him Jochebed his father's sister to wife; and she bare him Aaron and Moses: and the years of the life of Amram \emph{were} an hundred and thirty and seven years."],
['Exodus','Exo','06','021','21',"And the sons of Izhar; Korah, and Nepheg, and Zichri."],
['Exodus','Exo','06','022','22',"And the sons of Uzziel; Mishael, and Elzaphan, and Zithri."],
['Exodus','Exo','06','023','23',"And Aaron took him Elisheba, daughter of Amminadab, sister of Naashon, to wife; and she bare him Nadab, and Abihu, Eleazar, and Ithamar."],
['Exodus','Exo','06','024','24',"And the sons of Korah; Assir, and Elkanah, and Abiasaph: these \emph{are} the families of the Korhites."],
['Exodus','Exo','06','025','25',"And Eleazar Aaron's son took him \emph{one} of the daughters of Putiel to wife; and she bare him Phinehas: these \emph{are} the heads of the fathers of the Levites according to their families."],
['Exodus','Exo','06','026','26',"These \emph{are} that Aaron and Moses, to whom the LORD said, Bring out the children of Israel from the land of Egypt according to their armies."],
['Exodus','Exo','06','027','27',"These \emph{are} they which spake to Pharaoh king of Egypt, to bring out the children of Israel from Egypt: these \emph{are} that Moses and Aaron."],
['Exodus','Exo','06','028','28',"And it came to pass on the day \emph{when} the LORD spake unto Moses in the land of Egypt,"],
['Exodus','Exo','06','029','29',"That the LORD spake unto Moses, saying, I \emph{am} the LORD: speak thou unto Pharaoh king of Egypt all that I say unto thee."],
['Exodus','Exo','06','030','30',"And Moses said before the LORD, Behold, I \emph{am} of uncircumcised lips, and how shall Pharaoh hearken unto me?"]
]    
    
Exodus07 = [
    ['Exodus','Exo','07','001','1',"And the LORD said unto Moses, See, I have made thee a god to Pharaoh: and Aaron thy brother shall be thy prophet."],
['Exodus','Exo','07','002','2',"Thou shalt speak all that I command thee: and Aaron thy brother shall speak unto Pharaoh, that he send the children of Israel out of his land."],
['Exodus','Exo','07','003','3',"And I will harden Pharaoh's heart, and multiply my signs and my wonders in the land of Egypt."],
['Exodus','Exo','07','004','4',"But Pharaoh shall not hearken unto you, that I may lay my hand upon Egypt, and bring forth mine armies, \emph{and} my people the children of Israel, out of the land of Egypt by great judgments."],
['Exodus','Exo','07','005','5',"And the Egyptians shall know that I \emph{am} the LORD, when I stretch forth mine hand upon Egypt, and bring out the children of Israel from among them."],
['Exodus','Exo','07','006','6',"And Moses and Aaron did as the LORD commanded them, so did they."],
['Exodus','Exo','07','007','7',"And Moses \emph{was} fourscore years old, and Aaron fourscore and three years old, when they spake unto Pharaoh."],
['Exodus','Exo','07','008','8',"And the LORD spake unto Moses and unto Aaron, saying,"],
['Exodus','Exo','07','009','9',"When Pharaoh shall speak unto you, saying, Shew a miracle for you: then thou shalt say unto Aaron, Take thy rod, and cast \emph{it} before Pharaoh, \emph{and} it shall become a serpent."],
['Exodus','Exo','07','010','10',"And Moses and Aaron went in unto Pharaoh, and they did so as the LORD had commanded: and Aaron cast down his rod before Pharaoh, and before his servants, and it became a serpent."],
['Exodus','Exo','07','011','11',"Then Pharaoh also called the wise men and the sorcerers: now the magicians of Egypt, they also did in like manner with their enchantments."],
['Exodus','Exo','07','012','12',"For they cast down every man his rod, and they became serpents: but Aaron's rod swallowed up their rods."],
['Exodus','Exo','07','013','13',"And he hardened Pharaoh's heart, that he hearkened not unto them; as the LORD had said."],
['Exodus','Exo','07','014','14',"And the LORD said unto Moses, Pharaoh's heart \emph{is} hardened, he refuseth to let the people go."],
['Exodus','Exo','07','015','15',"Get thee unto Pharaoh in the morning; lo, he goeth out unto the water; and thou shalt stand by the river's brink against he come; and the rod which was turned to a serpent shalt thou take in thine hand."],
['Exodus','Exo','07','016','16',"And thou shalt say unto him, The LORD God of the Hebrews hath sent me unto thee, saying, Let my people go, that they may serve me in the wilderness: and, behold, hitherto thou wouldest not hear."],
['Exodus','Exo','07','017','17',"Thus saith the LORD, In this thou shalt know that I \emph{am} the LORD: behold, I will smite with the rod that \emph{is} in mine hand upon the waters which \emph{are} in the river, and they shall be turned to blood."],
['Exodus','Exo','07','018','18',"And the fish that \emph{is} in the river shall die, and the river shall stink; and the Egyptians shall lothe to drink of the water of the river."],
['Exodus','Exo','07','019','19',"And the LORD spake unto Moses, Say unto Aaron, Take thy rod, and stretch out thine hand upon the waters of Egypt, upon their streams, upon their rivers, and upon their ponds, and upon all their pools of water, that they may become blood; and \emph{that} there may be blood throughout all the land of Egypt, both in \emph{vessels} \emph{of} wood, and in \emph{vessels} \emph{of} stone."],
['Exodus','Exo','07','020','20',"And Moses and Aaron did so, as the LORD commanded; and he lifted up the rod, and smote the waters that \emph{were} in the river, in the sight of Pharaoh, and in the sight of his servants; and all the waters that \emph{were} in the river were turned to blood."],
['Exodus','Exo','07','021','21',"And the fish that \emph{was} in the river died; and the river stank, and the Egyptians could not drink of the water of the river; and there was blood throughout all the land of Egypt."],
['Exodus','Exo','07','022','22',"And the magicians of Egypt did so with their enchantments: and Pharaoh's heart was hardened, neither did he hearken unto them; as the LORD had said."],
['Exodus','Exo','07','023','23',"And Pharaoh turned and went into his house, neither did he set his heart to this also."],
['Exodus','Exo','07','024','24',"And all the Egyptians digged round about the river for water to drink; for they could not drink of the water of the river."],
['Exodus','Exo','07','025','25',"And seven days were fulfilled, after that the LORD had smitten the river."]
]
     
Exodus08 = [
    ['Exodus','Exo','08','001','1',"And the LORD spake unto Moses, Go unto Pharaoh, and say unto him, Thus saith the LORD, Let my people go, that they may serve me."],
    ['Exodus','Exo','08','002','2',"And if thou refuse to let \emph{them} go, behold, I will smite all thy borders with frogs:"],
    ['Exodus','Exo','08','003','3',"And the river shall bring forth frogs abundantly, which shall go up and come into thine house, and into thy bedchamber, and upon thy bed, and into the house of thy servants, and upon thy people, and into thine ovens, and into thy kneadingtroughs:"],
    ['Exodus','Exo','08','004','4',"And the frogs shall come up both on thee, and upon thy people, and upon all thy servants."],
    ['Exodus','Exo','08','005','5',"And the LORD spake unto Moses, Say unto Aaron, Stretch forth thine hand with thy rod over the streams, over the rivers, and over the ponds, and cause frogs to come up upon the land of Egypt."],
    ['Exodus','Exo','08','006','6',"And Aaron stretched out his hand over the waters of Egypt; and the frogs came up, and covered the land of Egypt."],
['Exodus','Exo','08','007','7',"And the magicians did so with their enchantments, and brought up frogs upon the land of Egypt."],
['Exodus','Exo','08','008','8',"Then Pharaoh called for Moses and Aaron, and said, Intreat the LORD, that he may take away the frogs from me, and from my people; and I will let the people go, that they may do sacrifice unto the LORD."],
['Exodus','Exo','08','009','9',"And Moses said unto Pharaoh, Glory over me: when shall I intreat for thee, and for thy servants, and for thy people, to destroy the frogs from thee and thy houses, \emph{that} they may remain in the river only?"],
['Exodus','Exo','08','010','10',"And he said, To morrow. And he said, \emph{Be} \emph{it} according to thy word: that thou mayest know that \emph{there} \emph{is} none like unto the LORD our God."],
['Exodus','Exo','08','011','11',"And the frogs shall depart from thee, and from thy houses, and from thy servants, and from thy people; they shall remain in the river only."],
['Exodus','Exo','08','012','12',"And Moses and Aaron went out from Pharaoh: and Moses cried unto the LORD because of the frogs which he had brought against Pharaoh."],
['Exodus','Exo','08','013','13',"And the LORD did according to the word of Moses; and the frogs died out of the houses, out of the villages, and out of the fields."],
['Exodus','Exo','08','014','14',"And they gathered them together upon heaps: and the land stank."],
['Exodus','Exo','08','015','15',"But when Pharaoh saw that there was respite, he hardened his heart, and hearkened not unto them; as the LORD had said."],
['Exodus','Exo','08','016','16',"And the LORD said unto Moses, Say unto Aaron, Stretch out thy rod, and smite the dust of the land, that it may become lice throughout all the land of Egypt."],
['Exodus','Exo','08','017','17',"And they did so; for Aaron stretched out his hand with his rod, and smote the dust of the earth, and it became lice in man, and in beast; all the dust of the land became lice throughout all the land of Egypt."],
['Exodus','Exo','08','018','18',"And the magicians did so with their enchantments to bring forth lice, but they could not: so there were lice upon man, and upon beast."],
['Exodus','Exo','08','019','19',"Then the magicians said unto Pharaoh, This \emph{is} the finger of God: and Pharaoh's heart was hardened, and he hearkened not unto them; as the LORD had said."],
['Exodus','Exo','08','020','20',"And the LORD said unto Moses, Rise up early in the morning, and stand before Pharaoh; lo, he cometh forth to the water; and say unto him, Thus saith the LORD, Let my people go, that they may serve me."],
['Exodus','Exo','08','021','21',"Else, if thou wilt not let my people go, behold, I will send swarms \emph{of} \emph{flies} upon thee, and upon thy servants, and upon thy people, and into thy houses: and the houses of the Egyptians shall be full of swarms \emph{of} \emph{flies}, and also the ground whereon they \emph{are}."],
['Exodus','Exo','08','022','22',"And I will sever in that day the land of Goshen, in which my people dwell, that no swarms \emph{of} \emph{flies} shall be there; to the end thou mayest know that I \emph{am} the LORD in the midst of the earth."],
['Exodus','Exo','08','023','23',"And I will put a division between my people and thy people: to morrow shall this sign be."],
    ['Exodus','Exo','08','024','24',"And the LORD did so; and there came a grievous swarm \emph{of} \emph{flies} into the house of Pharaoh, and \emph{into} his servants' houses, and into all the land of Egypt: the land was corrupted by reason of the swarm \emph{of} \emph{flies}."],
    ['Exodus','Exo','08','025','25',"And Pharaoh called for Moses and for Aaron, and said, Go ye, sacrifice to your God in the land."],
    ['Exodus','Exo','08','026','26',"And Moses said, It is not meet so to do; for we shall sacrifice the abomination of the Egyptians to the LORD our God: lo, shall we sacrifice the abomination of the Egyptians before their eyes, and will they not stone us?"],
    ['Exodus','Exo','08','027','27',"We will go three days' journey into the wilderness, and sacrifice to the LORD our God, as he shall command us."],
    ['Exodus','Exo','08','028','28',"And Pharaoh said, I will let you go, that ye may sacrifice to the LORD your God in the wilderness; only ye shall not go very far away: intreat for me."],
    ['Exodus','Exo','08','029','29',"And Moses said, Behold, I go out from thee, and I will intreat the LORD that the swarms \emph{of} \emph{flies} may depart from Pharaoh, from his servants, and from his people, to morrow: but let not Pharaoh deal deceitfully any more in not letting the people go to sacrifice to the LORD."],
    ['Exodus','Exo','08','030','30',"And Moses went out from Pharaoh, and intreated the LORD."],
    ['Exodus','Exo','08','031','31',"And the LORD did according to the word of Moses; and he removed the swarms \emph{of} \emph{flies} from Pharaoh, from his servants, and from his people; there remained not one."],
    ['Exodus','Exo','08','032','32',"And Pharaoh hardened his heart at this time also, neither would he let the people go."]
]

Exodus09 = [
    ['Exodus','Exo','09','001','1',"Then the LORD said unto Moses, Go in unto Pharaoh, and tell him, Thus saith the LORD God of the Hebrews, Let my people go, that they may serve me."],
    ['Exodus','Exo','09','002','2',"For if thou refuse to let \emph{them} go, and wilt hold them still,"],
    ['Exodus','Exo','09','003','3',"Behold, the hand of the LORD is upon thy cattle which \emph{is} in the field, upon the horses, upon the asses, upon the camels, upon the oxen, and upon the sheep: \emph{there} \emph{shall} \emph{be} a very grievous murrain."],
    ['Exodus','Exo','09','004','4',"And the LORD shall sever between the cattle of Israel and the cattle of Egypt: and there shall nothing die of all \emph{that} \emph{is} the children's of Israel."],
    ['Exodus','Exo','09','005','5',"And the LORD appointed a set time, saying, To morrow the LORD shall do this thing in the land."],
    ['Exodus','Exo','09','006','6',"And the LORD did that thing on the morrow, and all the cattle of Egypt died: but of the cattle of the children of Israel died not one."],
    ['Exodus','Exo','09','007','7',"And Pharaoh sent, and, behold, there was not one of the cattle of the Israelites dead. And the heart of Pharaoh was hardened, and he did not let the people go."],
    ['Exodus','Exo','09','008','8',"And the LORD said unto Moses and unto Aaron, Take to you handfuls of ashes of the furnace, and let Moses sprinkle it toward the heaven in the sight of Pharaoh."],
    ['Exodus','Exo','09','009','9',"And it shall become small dust in all the land of Egypt, and shall be a boil breaking forth \emph{with} blains upon man, and upon beast, throughout all the land of Egypt."],
    ['Exodus','Exo','09','010','10',"And they took ashes of the furnace, and stood before Pharaoh; and Moses sprinkled it up toward heaven; and it became a boil breaking forth \emph{with} blains upon man, and upon beast."],
    ['Exodus','Exo','09','011','11',"And the magicians could not stand before Moses because of the boils; for the boil was upon the magicians, and upon all the Egyptians."],
    ['Exodus','Exo','09','012','12',"And the LORD hardened the heart of Pharaoh, and he hearkened not unto them; as the LORD had spoken unto Moses."],
    ['Exodus','Exo','09','013','13',"And the LORD said unto Moses, Rise up early in the morning, and stand before Pharaoh, and say unto him, Thus saith the LORD God of the Hebrews, Let my people go, that they may serve me."],
    ['Exodus','Exo','09','014','14',"For I will at this time send all my plagues upon thine heart, and upon thy servants, and upon thy people; that thou mayest know that \emph{there} \emph{is} none like me in all the earth."],
    ['Exodus','Exo','09','015','15',"For now I will stretch out my hand, that I may smite thee and thy people with pestilence; and thou shalt be cut off from the earth."],
    ['Exodus','Exo','09','016','16',"And in very deed for this \emph{cause} have I raised thee up, for to shew \emph{in} thee my power; and that my name may be declared throughout all the earth."],
    ['Exodus','Exo','09','017','17',"As yet exaltest thou thyself against my people, that thou wilt not let them go?"],
    ['Exodus','Exo','09','018','18',"Behold, to morrow about this time I will cause it to rain a very grievous hail, such as hath not been in Egypt since the foundation thereof even until now."],
    ['Exodus','Exo','09','019','19',"Send therefore now, \emph{and} gather thy cattle, and all that thou hast in the field; \emph{for} \emph{upon} every man and beast which shall be found in the field, and shall not be brought home, the hail shall come down upon them, and they shall die."],
    ['Exodus','Exo','09','020','20',"He that feared the word of the LORD among the servants of Pharaoh made his servants and his cattle flee into the houses:"],
    ['Exodus','Exo','09','021','21',"And he that regarded not the word of the LORD left his servants and his cattle in the field."],
    ['Exodus','Exo','09','022','22',"And the LORD said unto Moses, Stretch forth thine hand toward heaven, that there may be hail in all the land of Egypt, upon man, and upon beast, and upon every herb of the field, throughout the land of Egypt."],
    ['Exodus','Exo','09','023','23',"And Moses stretched forth his rod toward heaven: and the LORD sent thunder and hail, and the fire ran along upon the ground; and the LORD rained hail upon the land of Egypt."],
    ['Exodus','Exo','09','024','24',"So there was hail, and fire mingled with the hail, very grievous, such as there was none like it in all the land of Egypt since it became a nation."],
    ['Exodus','Exo','09','025','25',"And the hail smote throughout all the land of Egypt all that \emph{was} in the field, both man and beast; and the hail smote every herb of the field, and brake every tree of the field."],
    ['Exodus','Exo','09','026','26',"Only in the land of Goshen, where the children of Israel \emph{were}, was there no hail."],
    ['Exodus','Exo','09','027','27',"And Pharaoh sent, and called for Moses and Aaron, and said unto them, I have sinned this time: the LORD \emph{is} righteous, and I and my people \emph{are} wicked."],
    ['Exodus','Exo','09','028','28',"Intreat the LORD (for \emph{it} \emph{is} enough) that there be no \emph{more} mighty thunderings and hail; and I will let you go, and ye shall stay no longer."],
    ['Exodus','Exo','09','029','29',"And Moses said unto him, As soon as I am gone out of the city, I will spread abroad my hands unto the LORD; \emph{and} the thunder shall cease, neither shall there be any more hail; that thou mayest know how that the earth \emph{is} the LORD'S."],
    ['Exodus','Exo','09','030','30',"But as for thee and thy servants, I know that ye will not yet fear the LORD God."],
    ['Exodus','Exo','09','031','31',"And the flax and the barley was smitten: for the barley \emph{was} in the ear, and the flax \emph{was} bolled."],
    ['Exodus','Exo','09','032','32',"But the wheat and the rie were not smitten: for they \emph{were} not grown up."],
    ['Exodus','Exo','09','033','33',"And Moses went out of the city from Pharaoh, and spread abroad his hands unto the LORD: and the thunders and hail ceased, and the rain was not poured upon the earth."],
    ['Exodus','Exo','09','034','34',"And when Pharaoh saw that the rain and the hail and the thunders were ceased, he sinned yet more, and hardened his heart, he and his servants."],
    ['Exodus','Exo','09','035','35',"And the heart of Pharaoh was hardened, neither would he let the children of Israel go; as the LORD had spoken by Moses."]
]
 
Exodus10 = [
    ['Exodus','Exo','10','001','1',"And the LORD said unto Moses, Go in unto Pharaoh: for I have hardened his heart, and the heart of his servants, that I might shew these my signs before him:"],
    ['Exodus','Exo','10','002','2',"And that thou mayest tell in the ears of thy son, and of thy son's son, what things I have wrought in Egypt, and my signs which I have done among them; that ye may know how that I \emph{am} the LORD."],
    ['Exodus','Exo','10','003','3',"And Moses and Aaron came in unto Pharaoh, and said unto him, Thus saith the LORD God of the Hebrews, How long wilt thou refuse to humble thyself before me? let my people go, that they may serve me."],
    ['Exodus','Exo','10','004','4',"Else, if thou refuse to let my people go, behold, to morrow will I bring the locusts into thy coast:"],
['Exodus','Exo','10','005','5',"And they shall cover the face of the earth, that one cannot be able to see the earth: and they shall eat the residue of that which is escaped, which remaineth unto you from the hail, and shall eat every tree which groweth for you out of the field:"],
['Exodus','Exo','10','006','6',"And they shall fill thy houses, and the houses of all thy servants, and the houses of all the Egyptians; which neither thy fathers, nor thy fathers' fathers have seen, since the day that they were upon the earth unto this day. And he turned himself, and went out from Pharaoh."],
['Exodus','Exo','10','007','7',"And Pharaoh's servants said unto him, How long shall this man be a snare unto us? let the men go, that they may serve the LORD their God: knowest thou not yet that Egypt is destroyed?"],
['Exodus','Exo','10','008','8',"And Moses and Aaron were brought again unto Pharaoh: and he said unto them, Go, serve the LORD your God: \emph{but} who \emph{are} they that shall go?"],
['Exodus','Exo','10','009','9',"And Moses said, We will go with our young and with our old, with our sons and with our daughters, with our flocks and with our herds will we go; for we \emph{must} \emph{hold} a feast unto the LORD."],
['Exodus','Exo','10','010','10',"And he said unto them, Let the LORD be so with you, as I will let you go, and your little ones: look \emph{to} \emph{it}; for evil \emph{is} before you."],
['Exodus','Exo','10','011','11',"Not so: go now ye \emph{that} \emph{are} men, and serve the LORD; for that ye did desire. And they were driven out from Pharaoh's presence."],
['Exodus','Exo','10','012','12',"And the LORD said unto Moses, Stretch out thine hand over the land of Egypt for the locusts, that they may come up upon the land of Egypt, and eat every herb of the land, \emph{even} all that the hail hath left."],
['Exodus','Exo','10','013','13',"And Moses stretched forth his rod over the land of Egypt, and the LORD brought an east wind upon the land all that day, and all \emph{that} night; \emph{and} when it was morning, the east wind brought the locusts."],
['Exodus','Exo','10','014','14',"And the locusts went up over all the land of Egypt, and rested in all the coasts of Egypt: very grievous \emph{were} \emph{they}; before them there were no such locusts as they, neither after them shall be such."],
['Exodus','Exo','10','015','15',"For they covered the face of the whole earth, so that the land was darkened; and they did eat every herb of the land, and all the fruit of the trees which the hail had left: and there remained not any green thing in the trees, or in the herbs of the field, through all the land of Egypt."],
['Exodus','Exo','10','016','16',"Then Pharaoh called for Moses and Aaron in haste; and he said, I have sinned against the LORD your God, and against you."],
['Exodus','Exo','10','017','17',"Now therefore forgive, I pray thee, my sin only this once, and intreat the LORD your God, that he may take away from me this death only."],
['Exodus','Exo','10','018','18',"And he went out from Pharaoh, and intreated the LORD."],
['Exodus','Exo','10','019','19',"And the LORD turned a mighty strong west wind, which took away the locusts, and cast them into the Red sea; there remained not one locust in all the coasts of Egypt."],
['Exodus','Exo','10','020','20',"But the LORD hardened Pharaoh's heart, so that he would not let the children of Israel go."],
['Exodus','Exo','10','021','21',"And the LORD said unto Moses, Stretch out thine hand toward heaven, that there may be darkness over the land of Egypt, even darkness \emph{which} may be felt."],
['Exodus','Exo','10','022','22',"And Moses stretched forth his hand toward heaven; and there was a thick darkness in all the land of Egypt three days:"],
['Exodus','Exo','10','023','23',"They saw not one another, neither rose any from his place for three days: but all the children of Israel had light in their dwellings."],
['Exodus','Exo','10','024','24',"And Pharaoh called unto Moses, and said, Go ye, serve the LORD; only let your flocks and your herds be stayed: let your little ones also go with you."],
['Exodus','Exo','10','025','25',"And Moses said, Thou must give us also sacrifices and burnt offerings, that we may sacrifice unto the LORD our God."],
    ['Exodus','Exo','10','026','26',"Our cattle also shall go with us; there shall not an hoof be left behind; for thereof must we take to serve the LORD our God; and we know not with what we must serve the LORD, until we come thither."],
    ['Exodus','Exo','10','027','27',"But the LORD hardened Pharaoh's heart, and he would not let them go."],
    ['Exodus','Exo','10','028','28',"And Pharaoh said unto him, Get thee from me, take heed to thyself, see my face no more; for in \emph{that} day thou seest my face thou shalt die."],
    ['Exodus','Exo','10','029','29',"And Moses said, Thou hast spoken well, I will see thy face again no more."]
]    
    
Exodus11 = [
    ['Exodus','Exo','11','001','1',"And the LORD said unto Moses, Yet will I bring one plague \emph{more} upon Pharaoh, and upon Egypt; afterwards he will let you go hence: when he shall let \emph{you} go, he shall surely thrust you out hence altogether."],
['Exodus','Exo','11','002','2',"Speak now in the ears of the people, and let every man borrow of his neighbour, and every woman of her neighbour, jewels of silver, and jewels of gold."],
['Exodus','Exo','11','003','3',"And the LORD gave the people favour in the sight of the Egyptians. Moreover the man Moses \emph{was} very great in the land of Egypt, in the sight of Pharaoh's servants, and in the sight of the people."],
['Exodus','Exo','11','004','4',"And Moses said, Thus saith the LORD, About midnight will I go out into the midst of Egypt:"],
['Exodus','Exo','11','005','5',"And all the firstborn in the land of Egypt shall die, from the firstborn of Pharaoh that sitteth upon his throne, even unto the firstborn of the maidservant that \emph{is} behind the mill; and all the firstborn of beasts."],
['Exodus','Exo','11','006','6',"And there shall be a great cry throughout all the land of Egypt, such as there was none like it, nor shall be like it any more."],
['Exodus','Exo','11','007','7',"But against any of the children of Israel shall not a dog move his tongue, against man or beast: that ye may know how that the LORD doth put a difference between the Egyptians and Israel."],
['Exodus','Exo','11','008','8',"And all these thy servants shall come down unto me, and bow down themselves unto me, saying, Get thee out, and all the people that follow thee: and after that I will go out. And he went out from Pharaoh in a great anger."],
['Exodus','Exo','11','009','9',"And the LORD said unto Moses, Pharaoh shall not hearken unto you; that my wonders may be multiplied in the land of Egypt."],
['Exodus','Exo','11','010','10',"And Moses and Aaron did all these wonders before Pharaoh: and the LORD hardened Pharaoh's heart, so that he would not let the children of Israel go out of his land."]
]

Exodus12 = [
    ['Exodus','Exo','12','001','1',"And the LORD spake unto Moses and Aaron in the land of Egypt, saying,"],
    ['Exodus','Exo','12','002','2',"This month \emph{shall} \emph{be} unto you the beginning of months: it \emph{shall} \emph{be} the first month of the year to you."],
    ['Exodus','Exo','12','003','3',"Speak ye unto all the congregation of Israel, saying, In the tenth \emph{day} of this month they shall take to them every man a lamb, according to the house of \emph{their} fathers, a lamb for an house:"],
    ['Exodus','Exo','12','004','4',"And if the household be too little for the lamb, let him and his neighbour next unto his house take \emph{it} according to the number of the souls; every man according to his eating shall make your count for the lamb."],
['Exodus','Exo','12','005','5',"Your lamb shall be without blemish, a male of the first year: ye shall take \emph{it} out from the sheep, or from the goats:"],
['Exodus','Exo','12','006','6',"And ye shall keep it up until the fourteenth day of the same month: and the whole assembly of the congregation of Israel shall kill it in the evening."],
['Exodus','Exo','12','007','7',"And they shall take of the blood, and strike \emph{it} on the two side posts and on the upper door post of the houses, wherein they shall eat it."],
['Exodus','Exo','12','008','8',"And they shall eat the flesh in that night, roast with fire, and unleavened bread; \emph{and} with bitter \emph{herbs} they shall eat it."],
['Exodus','Exo','12','009','9',"Eat not of it raw, nor sodden at all with water, but roast \emph{with} fire; his head with his legs, and with the purtenance thereof."],
['Exodus','Exo','12','010','10',"And ye shall let nothing of it remain until the morning; and that which remaineth of it until the morning ye shall burn with fire."],
['Exodus','Exo','12','011','11',"And thus shall ye eat it; \emph{with} your loins girded, your shoes on your feet, and your staff in your hand; and ye shall eat it in haste: it \emph{is} the LORD'S passover."],
['Exodus','Exo','12','012','12',"For I will pass through the land of Egypt this night, and will smite all the firstborn in the land of Egypt, both man and beast; and against all the gods of Egypt I will execute judgment: I \emph{am} the LORD."],
['Exodus','Exo','12','013','13',"And the blood shall be to you for a token upon the houses where ye \emph{are}: and when I see the blood, I will pass over you, and the plague shall not be upon you to destroy \emph{you}, when I smite the land of Egypt."],
['Exodus','Exo','12','014','14',"And this day shall be unto you for a memorial; and ye shall keep it a feast to the LORD throughout your generations; ye shall keep it a feast by an ordinance for ever."],
['Exodus','Exo','12','015','15',"Seven days shall ye eat unleavened bread; even the first day ye shall put away leaven out of your houses: for whosoever eateth leavened bread from the first day until the seventh day, that soul shall be cut off from Israel."],
['Exodus','Exo','12','016','16',"And in the first day \emph{there} \emph{shall} \emph{be} an holy convocation, and in the seventh day there shall be an holy convocation to you; no manner of work shall be done in them, save \emph{that} which every man must eat, that only may be done of you."],
['Exodus','Exo','12','017','17',"And ye shall observe \emph{the} \emph{feast} \emph{of} unleavened bread; for in this selfsame day have I brought your armies out of the land of Egypt: therefore shall ye observe this day in your generations by an ordinance for ever."],
['Exodus','Exo','12','018','18',"In the first \emph{month}, on the fourteenth day of the month at even, ye shall eat unleavened bread, until the one and twentieth day of the month at even."],
['Exodus','Exo','12','019','19',"Seven days shall there be no leaven found in your houses: for whosoever eateth that which is leavened, even that soul shall be cut off from the congregation of Israel, whether he be a stranger, or born in the land."],
['Exodus','Exo','12','020','20',"Ye shall eat nothing leavened; in all your habitations shall ye eat unleavened bread."],
['Exodus','Exo','12','021','21',"Then Moses called for all the elders of Israel, and said unto them, Draw out and take you a lamb according to your families, and kill the passover."],
['Exodus','Exo','12','022','22',"And ye shall take a bunch of hyssop, and dip \emph{it} in the blood that \emph{is} in the bason, and strike the lintel and the two side posts with the blood that \emph{is} in the bason; and none of you shall go out at the door of his house until the morning."],
['Exodus','Exo','12','023','23',"For the LORD will pass through to smite the Egyptians; and when he seeth the blood upon the lintel, and on the two side posts, the LORD will pass over the door, and will not suffer the destroyer to come in unto your houses to smite \emph{you}."],
['Exodus','Exo','12','024','24',"And ye shall observe this thing for an ordinance to thee and to thy sons for ever."],
['Exodus','Exo','12','025','25',"And it shall come to pass, when ye be come to the land which the LORD will give you, according as he hath promised, that ye shall keep this service."],
['Exodus','Exo','12','026','26',"And it shall come to pass, when your children shall say unto you, What mean ye by this service?"],
['Exodus','Exo','12','027','27',"That ye shall say, It \emph{is} the sacrifice of the LORD'S passover, who passed over the houses of the children of Israel in Egypt, when he smote the Egyptians, and delivered our houses. And the people bowed the head and worshipped."],
['Exodus','Exo','12','028','28',"And the children of Israel went away, and did as the LORD had commanded Moses and Aaron, so did they."],
['Exodus','Exo','12','029','29',"And it came to pass, that at midnight the LORD smote all the firstborn in the land of Egypt, from the firstborn of Pharaoh that sat on his throne unto the firstborn of the captive that \emph{was} in the dungeon; and all the firstborn of cattle."],
['Exodus','Exo','12','030','30',"And Pharaoh rose up in the night, he, and all his servants, and all the Egyptians; and there was a great cry in Egypt; for \emph{there} \emph{was} not a house where \emph{there} \emph{was} not one dead."],
['Exodus','Exo','12','031','31',"And he called for Moses and Aaron by night, and said, Rise up, \emph{and} get you forth from among my people, both ye and the children of Israel; and go, serve the LORD, as ye have said."],
['Exodus','Exo','12','032','32',"Also take your flocks and your herds, as ye have said, and be gone; and bless me also."],
['Exodus','Exo','12','033','33',"And the Egyptians were urgent upon the people, that they might send them out of the land in haste; for they said, We \emph{be} all dead \emph{men}."],
['Exodus','Exo','12','034','34',"And the people took their dough before it was leavened, their kneadingtroughs being bound up in their clothes upon their shoulders."],
['Exodus','Exo','12','035','35',"And the children of Israel did according to the word of Moses; and they borrowed of the Egyptians jewels of silver, and jewels of gold, and raiment:"],
['Exodus','Exo','12','036','36',"And the LORD gave the people favour in the sight of the Egyptians, so that they lent unto them \emph{such} \emph{things} \emph{as} \emph{they} \emph{required}. And they spoiled the Egyptians."],
['Exodus','Exo','12','037','37',"And the children of Israel journeyed from Rameses to Succoth, about six hundred thousand on foot \emph{that} \emph{were} men, beside children."],
['Exodus','Exo','12','038','38',"And a mixed multitude went up also with them; and flocks, and herds, \emph{even} very much cattle."],
['Exodus','Exo','12','039','39',"And they baked unleavened cakes of the dough which they brought forth out of Egypt, for it was not leavened; because they were thrust out of Egypt, and could not tarry, neither had they prepared for themselves any victual."],
['Exodus','Exo','12','040','40',"Now the sojourning of the children of Israel, who dwelt in Egypt, \emph{was} four hundred and thirty years."],
['Exodus','Exo','12','041','41',"And it came to pass at the end of the four hundred and thirty years, even the selfsame day it came to pass, that all the hosts of the LORD went out from the land of Egypt."],
    ['Exodus','Exo','12','042','42',"It \emph{is} a night to be much observed unto the LORD for bringing them out from the land of Egypt: this \emph{is} that night of the LORD to be observed of all the children of Israel in their generations."],
    ['Exodus','Exo','12','043','43',"And the LORD said unto Moses and Aaron, This \emph{is} the ordinance of the passover: There shall no stranger eat thereof:"],
    ['Exodus','Exo','12','044','44',"But every man's servant that is bought for money, when thou hast circumcised him, then shall he eat thereof."],
    ['Exodus','Exo','12','045','45',"A foreigner and an hired servant shall not eat thereof."],
    ['Exodus','Exo','12','046','46',"In one house shall it be eaten; thou shalt not carry forth ought of the flesh abroad out of the house; neither shall ye break a bone thereof."],
    ['Exodus','Exo','12','047','47',"All the congregation of Israel shall keep it."],
    ['Exodus','Exo','12','048','48',"And when a stranger shall sojourn with thee, and will keep the passover to the LORD, let all his males be circumcised, and then let him come near and keep it; and he shall be as one that is born in the land: for no uncircumcised person shall eat thereof."],
    ['Exodus','Exo','12','049','49',"One law shall be to him that is homeborn, and unto the stranger that sojourneth among you."],
    ['Exodus','Exo','12','050','50',"Thus did all the children of Israel; as the LORD commanded Moses and Aaron, so did they."],
    ['Exodus','Exo','12','051','51',"And it came to pass the selfsame day, \emph{that} the LORD did bring the children of Israel out of the land of Egypt by their armies."]
]

Exodus13 = [
    ['Exodus','Exo','13','001','1',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','13','002','2',"Sanctify unto me all the firstborn, whatsoever openeth the womb among the children of Israel, \emph{both} of man and of beast: it \emph{is} mine."],
['Exodus','Exo','13','003','3',"And Moses said unto the people, Remember this day, in which ye came out from Egypt, out of the house of bondage; for by strength of hand the LORD brought you out from this \emph{place}: there shall no leavened bread be eaten."],
['Exodus','Exo','13','004','4',"This day came ye out in the month Abib."],
['Exodus','Exo','13','005','5',"And it shall be when the LORD shall bring thee into the land of the Canaanites, and the Hittites, and the Amorites, and the Hivites, and the Jebusites, which he sware unto thy fathers to give thee, a land flowing with milk and honey, that thou shalt keep this service in this month."],
['Exodus','Exo','13','006','6',"Seven days thou shalt eat unleavened bread, and in the seventh day \emph{shall} \emph{be} a feast to the LORD."],
['Exodus','Exo','13','007','7',"Unleavened bread shall be eaten seven days; and there shall no leavened bread be seen with thee, neither shall there be leaven seen with thee in all thy quarters."],
['Exodus','Exo','13','008','8',"And thou shalt shew thy son in that day, saying, \emph{This} \emph{is} \emph{done} because of that \emph{which} the LORD did unto me when I came forth out of Egypt."],
['Exodus','Exo','13','009','9',"And it shall be for a sign unto thee upon thine hand, and for a memorial between thine eyes, that the LORD'S law may be in thy mouth: for with a strong hand hath the LORD brought thee out of Egypt."],
['Exodus','Exo','13','010','10',"Thou shalt therefore keep this ordinance in his season from year to year."],
['Exodus','Exo','13','011','11',"And it shall be when the LORD shall bring thee into the land of the Canaanites, as he sware unto thee and to thy fathers, and shall give it thee,"],
['Exodus','Exo','13','012','12',"That thou shalt set apart unto the LORD all that openeth the matrix, and every firstling that cometh of a beast which thou hast; the males \emph{shall} \emph{be} the LORD'S."],
['Exodus','Exo','13','013','13',"And every firstling of an ass thou shalt redeem with a lamb; and if thou wilt not redeem it, then thou shalt break his neck: and all the firstborn of man among thy children shalt thou redeem."],
['Exodus','Exo','13','014','14',"And it shall be when thy son asketh thee in time to come, saying, What \emph{is} this? that thou shalt say unto him, By strength of hand the LORD brought us out from Egypt, from the house of bondage:"],
['Exodus','Exo','13','015','15',"And it came to pass, when Pharaoh would hardly let us go, that the LORD slew all the firstborn in the land of Egypt, both the firstborn of man, and the firstborn of beast: therefore I sacrifice to the LORD all that openeth the matrix, being males; but all the firstborn of my children I redeem."],
['Exodus','Exo','13','016','16',"And it shall be for a token upon thine hand, and for frontlets between thine eyes: for by strength of hand the LORD brought us forth out of Egypt."],
['Exodus','Exo','13','017','17',"And it came to pass, when Pharaoh had let the people go, that God led them not \emph{through} the way of the land of the Philistines, although that \emph{was} near; for God said, Lest peradventure the people repent when they see war, and they return to Egypt:"],
['Exodus','Exo','13','018','18',"But God led the people about, \emph{through} the way of the wilderness of the Red sea: and the children of Israel went up harnessed out of the land of Egypt."],
['Exodus','Exo','13','019','19',"And Moses took the bones of Joseph with him: for he had straitly sworn the children of Israel, saying, God will surely visit you; and ye shall carry up my bones away hence with you."],
['Exodus','Exo','13','020','20',"And they took their journey from Succoth, and encamped in Etham, in the edge of the wilderness."],
['Exodus','Exo','13','021','21',"And the LORD went before them by day in a pillar of a cloud, to lead them the way; and by night in a pillar of fire, to give them light; to go by day and night:"],
['Exodus','Exo','13','022','22',"He took not away the pillar of the cloud by day, nor the pillar of fire by night, \emph{from} before the people."]
] 
   
Exodus14 = [
    ['Exodus','Exo','14','001','1',"And the LORD spake unto Moses, saying,"],
    ['Exodus','Exo','14','002','2',"Speak unto the children of Israel, that they turn and encamp before Pi-hahiroth, between Migdol and the sea, over against Baal-zephon: before it shall ye encamp by the sea."],
    ['Exodus','Exo','14','003','3',"For Pharaoh will say of the children of Israel, They \emph{are} entangled in the land, the wilderness hath shut them in."],
    ['Exodus','Exo','14','004','4',"And I will harden Pharaoh's heart, that he shall follow after them; and I will be honoured upon Pharaoh, and upon all his host; that the Egyptians may know that I \emph{am} the LORD. And they did so."],
    ['Exodus','Exo','14','005','5',"And it was told the king of Egypt that the people fled: and the heart of Pharaoh and of his servants was turned against the people, and they said, Why have we done this, that we have let Israel go from serving us?"],
    ['Exodus','Exo','14','006','6',"And he made ready his chariot, and took his people with him:"],
    ['Exodus','Exo','14','007','7',"And he took six hundred chosen chariots, and all the chariots of Egypt, and captains over every one of them."],
    ['Exodus','Exo','14','008','8',"And the LORD hardened the heart of Pharaoh king of Egypt, and he pursued after the children of Israel: and the children of Israel went out with an high hand."],
    ['Exodus','Exo','14','009','9',"But the Egyptians pursued after them, all the horses \emph{and} chariots of Pharaoh, and his horsemen, and his army, and overtook them encamping by the sea, beside Pi-hahiroth, before Baal-zephon."],
['Exodus','Exo','14','010','10',"And when Pharaoh drew nigh, the children of Israel lifted up their eyes, and, behold, the Egyptians marched after them; and they were sore afraid: and the children of Israel cried out unto the LORD."],
['Exodus','Exo','14','011','11',"And they said unto Moses, Because \emph{there} \emph{were} no graves in Egypt, hast thou taken us away to die in the wilderness? wherefore hast thou dealt thus with us, to carry us forth out of Egypt?"],
['Exodus','Exo','14','012','12',"\emph{Is} not this the word that we did tell thee in Egypt, saying, Let us alone, that we may serve the Egyptians? For \emph{it} \emph{had} \emph{been} better for us to serve the Egyptians, than that we should die in the wilderness."],
['Exodus','Exo','14','013','13',"And Moses said unto the people, Fear ye not, stand still, and see the salvation of the LORD, which he will shew to you to day: for the Egyptians whom ye have seen to day, ye shall see them again no more for ever."],
['Exodus','Exo','14','014','14',"The LORD shall fight for you, and ye shall hold your peace."],
['Exodus','Exo','14','015','15',"And the LORD said unto Moses, Wherefore criest thou unto me? speak unto the children of Israel, that they go forward:"],
['Exodus','Exo','14','016','16',"But lift thou up thy rod, and stretch out thine hand over the sea, and divide it: and the children of Israel shall go on dry \emph{ground} through the midst of the sea."],
['Exodus','Exo','14','017','17',"And I, behold, I will harden the hearts of the Egyptians, and they shall follow them: and I will get me honour upon Pharaoh, and upon all his host, upon his chariots, and upon his horsemen."],
['Exodus','Exo','14','018','18',"And the Egyptians shall know that I \emph{am} the LORD, when I have gotten me honour upon Pharaoh, upon his chariots, and upon his horsemen."],
['Exodus','Exo','14','019','19',"And the angel of God, which went before the camp of Israel, removed and went behind them; and the pillar of the cloud went from before their face, and stood behind them:"],
['Exodus','Exo','14','020','20',"And it came between the camp of the Egyptians and the camp of Israel; and it was a cloud and darkness \emph{to} \emph{them}, but it gave light by night \emph{to} \emph{these}: so that the one came not near the other all the night."],
['Exodus','Exo','14','021','21',"And Moses stretched out his hand over the sea; and the LORD caused the sea to go \emph{back} by a strong east wind all that night, and made the sea dry \emph{land}, and the waters were divided."],
['Exodus','Exo','14','022','22',"And the children of Israel went into the midst of the sea upon the dry \emph{ground}: and the waters \emph{were} a wall unto them on their right hand, and on their left."],
['Exodus','Exo','14','023','23',"And the Egyptians pursued, and went in after them to the midst of the sea, \emph{even} all Pharaoh's horses, his chariots, and his horsemen."],
['Exodus','Exo','14','024','24',"And it came to pass, that in the morning watch the LORD looked unto the host of the Egyptians through the pillar of fire and of the cloud, and troubled the host of the Egyptians,"],
['Exodus','Exo','14','025','25',"And took off their chariot wheels, that they drave them heavily: so that the Egyptians said, Let us flee from the face of Israel; for the LORD fighteth for them against the Egyptians."],
['Exodus','Exo','14','026','26',"And the LORD said unto Moses, Stretch out thine hand over the sea, that the waters may come again upon the Egyptians, upon their chariots, and upon their horsemen."],
['Exodus','Exo','14','027','27',"And Moses stretched forth his hand over the sea, and the sea returned to his strength when the morning appeared; and the Egyptians fled against it; and the LORD overthrew the Egyptians in the midst of the sea."],
    ['Exodus','Exo','14','028','28',"And the waters returned, and covered the chariots, and the horsemen, \emph{and} all the host of Pharaoh that came into the sea after them; there remained not so much as one of them."],
    ['Exodus','Exo','14','029','29',"But the children of Israel walked upon dry \emph{land} in the midst of the sea; and the waters \emph{were} a wall unto them on their right hand, and on their left."],
    ['Exodus','Exo','14','030','30',"Thus the LORD saved Israel that day out of the hand of the Egyptians; and Israel saw the Egyptians dead upon the sea shore."],
    ['Exodus','Exo','14','031','31',"And Israel saw that great work which the LORD did upon the Egyptians: and the people feared the LORD, and believed the LORD, and his servant Moses."]
]   

Exodus15 = [
    ['Exodus','Exo','15','001','1',"Then sang Moses and the children of Israel this song unto the LORD, and spake, saying, I will sing unto the LORD, for he hath triumphed gloriously: the horse and his rider hath he thrown into the sea."],
    ['Exodus','Exo','15','002','2',"The LORD \emph{is} my strength and song, and he is become my salvation: he \emph{is} my God, and I will prepare him an habitation; my father's God, and I will exalt him."],
    ['Exodus','Exo','15','003','3',"The LORD \emph{is} a man of war: the LORD \emph{is} his name."],
    ['Exodus','Exo','15','004','4',"Pharaoh's chariots and his host hath he cast into the sea: his chosen captains also are drowned in the Red sea."],
    ['Exodus','Exo','15','005','5',"The depths have covered them: they sank into the bottom as a stone."],
['Exodus','Exo','15','006','6',"Thy right hand, O LORD, is become glorious in power: thy right hand, O LORD, hath dashed in pieces the enemy."],
['Exodus','Exo','15','007','7',"And in the greatness of thine excellency thou hast overthrown them that rose up against thee: thou sentest forth thy wrath, \emph{which} consumed them as stubble."],
['Exodus','Exo','15','008','8',"And with the blast of thy nostrils the waters were gathered together, the floods stood upright as an heap, \emph{and} the depths were congealed in the heart of the sea."],
['Exodus','Exo','15','009','9',"The enemy said, I will pursue, I will overtake, I will divide the spoil; my lust shall be satisfied upon them; I will draw my sword, my hand shall destroy them."],
['Exodus','Exo','15','010','10',"Thou didst blow with thy wind, the sea covered them: they sank as lead in the mighty waters."],
['Exodus','Exo','15','011','11',"Who \emph{is} like unto thee, O LORD, among the gods? who \emph{is} like thee, glorious in holiness, fearful \emph{in} praises, doing wonders?"],
['Exodus','Exo','15','012','12',"Thou stretchedst out thy right hand, the earth swallowed them."],
['Exodus','Exo','15','013','13',"Thou in thy mercy hast led forth the people \emph{which} thou hast redeemed: thou hast guided \emph{them} in thy strength unto thy holy habitation."],
['Exodus','Exo','15','014','14',"The people shall hear, \emph{and} be afraid: sorrow shall take hold on the inhabitants of Palestina."],
['Exodus','Exo','15','015','15',"Then the dukes of Edom shall be amazed; the mighty men of Moab, trembling shall take hold upon them; all the inhabitants of Canaan shall melt away."],
['Exodus','Exo','15','016','16',"Fear and dread shall fall upon them; by the greatness of thine arm they shall be \emph{as} still as a stone; till thy people pass over, O LORD, till the people pass over, \emph{which} thou hast purchased."],
['Exodus','Exo','15','017','17',"Thou shalt bring them in, and plant them in the mountain of thine inheritance, \emph{in} the place, O LORD, \emph{which} thou hast made for thee to dwell in, \emph{in} the Sanctuary, O Lord, \emph{which} thy hands have established."],
['Exodus','Exo','15','018','18',"The LORD shall reign for ever and ever."],
['Exodus','Exo','15','019','19',"For the horse of Pharaoh went in with his chariots and with his horsemen into the sea, and the LORD brought again the waters of the sea upon them; but the children of Israel went on dry \emph{land} in the midst of the sea."],
['Exodus','Exo','15','020','20',"And Miriam the prophetess, the sister of Aaron, took a timbrel in her hand; and all the women went out after her with timbrels and with dances."],
    ['Exodus','Exo','15','021','21',"And Miriam answered them, Sing ye to the LORD, for he hath triumphed gloriously; the horse and his rider hath he thrown into the sea."],
    ['Exodus','Exo','15','022','22',"So Moses brought Israel from the Red sea, and they went out into the wilderness of Shur; and they went three days in the wilderness, and found no water."],
    ['Exodus','Exo','15','023','23',"And when they came to Marah, they could not drink of the waters of Marah, for they \emph{were} bitter: therefore the name of it was called Marah."],
    ['Exodus','Exo','15','024','24',"And the people murmured against Moses, saying, What shall we drink?"],
    ['Exodus','Exo','15','025','25',"And he cried unto the LORD; and the LORD shewed him a tree, \emph{which} when he had cast into the waters, the waters were made sweet: there he made for them a statute and an ordinance, and there he proved them,"],
    ['Exodus','Exo','15','026','26',"And said, If thou wilt diligently hearken to the voice of the LORD thy God, and wilt do that which is right in his sight, and wilt give ear to his commandments, and keep all his statutes, I will put none of these diseases upon thee, which I have brought upon the Egyptians: for I \emph{am} the LORD that healeth thee."],
    ['Exodus','Exo','15','027','27',"And they came to Elim, where \emph{were} twelve wells of water, and threescore and ten palm trees: and they encamped there by the waters."]
]    

Exodus16 = [
    ['Exodus','Exo','16','001','1',"And they took their journey from Elim, and all the congregation of the children of Israel came unto the wilderness of Sin, which \emph{is} between Elim and Sinai, on the fifteenth day of the second month after their departing out of the land of Egypt."],
    ['Exodus','Exo','16','002','2',"And the whole congregation of the children of Israel murmured against Moses and Aaron in the wilderness:"],
    ['Exodus','Exo','16','003','3',"And the children of Israel said unto them, Would to God we had died by the hand of the LORD in the land of Egypt, when we sat by the flesh pots, \emph{and} when we did eat bread to the full; for ye have brought us forth into this wilderness, to kill this whole assembly with hunger."],
    ['Exodus','Exo','16','004','4',"Then said the LORD unto Moses, Behold, I will rain bread from heaven for you; and the people shall go out and gather a certain rate every day, that I may prove them, whether they will walk in my law, or no."],
    ['Exodus','Exo','16','005','5',"And it shall come to pass, that on the sixth day they shall prepare \emph{that} which they bring in; and it shall be twice as much as they gather daily."],
    ['Exodus','Exo','16','006','6',"And Moses and Aaron said unto all the children of Israel, At even, then ye shall know that the LORD hath brought you out from the land of Egypt:"],
    ['Exodus','Exo','16','007','7',"And in the morning, then ye shall see the glory of the LORD; for that he heareth your murmurings against the LORD: and what \emph{are} we, that ye murmur against us?"],
    ['Exodus','Exo','16','008','8',"And Moses said, \emph{This} \emph{shall} \emph{be}, when the LORD shall give you in the evening flesh to eat, and in the morning bread to the full; for that the LORD heareth your murmurings which ye murmur against him: and what \emph{are} we? your murmurings \emph{are} not against us, but against the LORD."],
    ['Exodus','Exo','16','009','9',"And Moses spake unto Aaron, Say unto all the congregation of the children of Israel, Come near before the LORD: for he hath heard your murmurings."],
    ['Exodus','Exo','16','010','10',"And it came to pass, as Aaron spake unto the whole congregation of the children of Israel, that they looked toward the wilderness, and, behold, the glory of the LORD appeared in the cloud."],
    ['Exodus','Exo','16','011','11',"And the LORD spake unto Moses, saying,"],
    ['Exodus','Exo','16','012','12',"I have heard the murmurings of the children of Israel: speak unto them, saying, At even ye shall eat flesh, and in the morning ye shall be filled with bread; and ye shall know that I \emph{am} the LORD your God."],
    ['Exodus','Exo','16','013','13',"And it came to pass, that at even the quails came up, and covered the camp: and in the morning the dew lay round about the host."],
    ['Exodus','Exo','16','014','14',"And when the dew that lay was gone up, behold, upon the face of the wilderness \emph{there} \emph{lay} a small round thing, \emph{as} small as the hoar frost on the ground."],
    ['Exodus','Exo','16','015','15',"And when the children of Israel saw \emph{it}, they said one to another, It \emph{is} manna: for they wist not what it \emph{was}. And Moses said unto them, This \emph{is} the bread which the LORD hath given you to eat."],
    ['Exodus','Exo','16','016','16',"This \emph{is} the thing which the LORD hath commanded, Gather of it every man according to his eating, an omer for every man, \emph{according} \emph{to} the number of your persons; take ye every man for \emph{them} which \emph{are} in his tents."],
    ['Exodus','Exo','16','017','17',"And the children of Israel did so, and gathered, some more, some less."],
    ['Exodus','Exo','16','018','18',"And when they did mete \emph{it} with an omer, he that gathered much had nothing over, and he that gathered little had no lack; they gathered every man according to his eating."],
    ['Exodus','Exo','16','019','19',"And Moses said, Let no man leave of it till the morning."],
    ['Exodus','Exo','16','020','20',"Notwithstanding they hearkened not unto Moses; but some of them left of it until the morning, and it bred worms, and stank: and Moses was wroth with them."],
    ['Exodus','Exo','16','021','21',"And they gathered it every morning, every man according to his eating: and when the sun waxed hot, it melted."],
    ['Exodus','Exo','16','022','22',"And it came to pass, \emph{that} on the sixth day they gathered twice as much bread, two omers for one \emph{man}: and all the rulers of the congregation came and told Moses."],
    ['Exodus','Exo','16','023','23',"And he said unto them, This \emph{is} \emph{that} which the LORD hath said, To morrow \emph{is} the rest of the holy sabbath unto the LORD: bake \emph{that} which ye will bake \emph{to} \emph{day}, and seethe that ye will seethe; and that which remaineth over lay up for you to be kept until the morning."],
    ['Exodus','Exo','16','024','24',"And they laid it up till the morning, as Moses bade: and it did not stink, neither was there any worm therein."],
    ['Exodus','Exo','16','025','25',"And Moses said, Eat that to day; for to day \emph{is} a sabbath unto the LORD: to day ye shall not find it in the field."],
    ['Exodus','Exo','16','026','26',"Six days ye shall gather it; but on the seventh day, \emph{which} \emph{is} the sabbath, in it there shall be none."],
    ['Exodus','Exo','16','027','27',"And it came to pass, \emph{that} there went out \emph{some} of the people on the seventh day for to gather, and they found none."],
    ['Exodus','Exo','16','028','28',"And the LORD said unto Moses, How long refuse ye to keep my commandments and my laws?"],
    ['Exodus','Exo','16','029','29',"See, for that the LORD hath given you the sabbath, therefore he giveth you on the sixth day the bread of two days; abide ye every man in his place, let no man go out of his place on the seventh day."],
    ['Exodus','Exo','16','030','30',"So the people rested on the seventh day."],
    ['Exodus','Exo','16','031','31',"And the house of Israel called the name thereof Manna: and it \emph{was} like coriander seed, white; and the taste of it \emph{was} like wafers \emph{made} with honey."],
    ['Exodus','Exo','16','032','32',"And Moses said, This \emph{is} the thing which the LORD commandeth, Fill an omer of it to be kept for your generations; that they may see the bread wherewith I have fed you in the wilderness, when I brought you forth from the land of Egypt."],
    ['Exodus','Exo','16','033','33',"And Moses said unto Aaron, Take a pot, and put an omer full of manna therein, and lay it up before the LORD, to be kept for your generations."],
    ['Exodus','Exo','16','034','34',"As the LORD commanded Moses, so Aaron laid it up before the Testimony, to be kept."],
    ['Exodus','Exo','16','035','35',"And the children of Israel did eat manna forty years, until they came to a land inhabited; they did eat manna, until they came unto the borders of the land of Canaan."],
    ['Exodus','Exo','16','036','36',"Now an omer \emph{is} the tenth \emph{part} of an ephah."]
]    
     
Exodus17 = [
    ['Exodus','Exo','17','001','1',"And all the congregation of the children of Israel journeyed from the wilderness of Sin, after their journeys, according to the commandment of the LORD, and pitched in Rephidim: and \emph{there} \emph{was} no water for the people to drink."],
    ['Exodus','Exo','17','002','2',"Wherefore the people did chide with Moses, and said, Give us water that we may drink. And Moses said unto them, Why chide ye with me? wherefore do ye tempt the LORD?"],
    ['Exodus','Exo','17','003','3',"And the people thirsted there for water; and the people murmured against Moses, and said, Wherefore \emph{is} this \emph{that} thou hast brought us up out of Egypt, to kill us and our children and our cattle with thirst?"],
    ['Exodus','Exo','17','004','4',"And Moses cried unto the LORD, saying, What shall I do unto this people? they be almost ready to stone me."],
    ['Exodus','Exo','17','005','5',"And the LORD said unto Moses, Go on before the people, and take with thee of the elders of Israel; and thy rod, wherewith thou smotest the river, take in thine hand, and go."],
    ['Exodus','Exo','17','006','6',"Behold, I will stand before thee there upon the rock in Horeb; and thou shalt smite the rock, and there shall come water out of it, that the people may drink. And Moses did so in the sight of the elders of Israel."],
['Exodus','Exo','17','007','7',"And he called the name of the place Massah, and Meribah, because of the chiding of the children of Israel, and because they tempted the LORD, saying, Is the LORD among us, or not?"],
['Exodus','Exo','17','008','8',"Then came Amalek, and fought with Israel in Rephidim."],
['Exodus','Exo','17','009','9',"And Moses said unto Joshua, Choose us out men, and go out, fight with Amalek: to morrow I will stand on the top of the hill with the rod of God in mine hand."],
['Exodus','Exo','17','010','10',"So Joshua did as Moses had said to him, and fought with Amalek: and Moses, Aaron, and Hur went up to the top of the hill."],
['Exodus','Exo','17','011','11',"And it came to pass, when Moses held up his hand, that Israel prevailed: and when he let down his hand, Amalek prevailed."],
['Exodus','Exo','17','012','12',"But Moses' hands \emph{were} heavy; and they took a stone, and put \emph{it} under him, and he sat thereon; and Aaron and Hur stayed up his hands, the one on the one side, and the other on the other side; and his hands were steady until the going down of the sun."],
['Exodus','Exo','17','013','13',"And Joshua discomfited Amalek and his people with the edge of the sword."],
['Exodus','Exo','17','014','14',"And the LORD said unto Moses, Write this \emph{for} a memorial in a book, and rehearse \emph{it} in the ears of Joshua: for I will utterly put out the remembrance of Amalek from under heaven."],
['Exodus','Exo','17','015','15',"And Moses built an altar, and called the name of it Jehovah-nissi:"],
    ['Exodus','Exo','17','016','16',"For he said, Because the LORD hath sworn \emph{that} the LORD \emph{will} \emph{have} war with Amalek from generation to generation."]
]
     
Exodus18 = [
    ['Exodus','Exo','18','001','1',"When Jethro, the priest of Midian, Moses' father in law, heard of all that God had done for Moses, and for Israel his people, \emph{and} that the LORD had brought Israel out of Egypt;"],
    ['Exodus','Exo','18','002','2',"Then Jethro, Moses' father in law, took Zipporah, Moses' wife, after he had sent her back,"],
    ['Exodus','Exo','18','003','3',"And her two sons; of which the name of the one \emph{was} Gershom; for he said, I have been an alien in a strange land:"],
    ['Exodus','Exo','18','004','4',"And the name of the other \emph{was} Eliezer; for the God of my father, \emph{said} \emph{he,} \emph{was} mine help, and delivered me from the sword of Pharaoh:"],
    ['Exodus','Exo','18','005','5',"And Jethro, Moses' father in law, came with his sons and his wife unto Moses into the wilderness, where he encamped at the mount of God:"],
    ['Exodus','Exo','18','006','6',"And he said unto Moses, I thy father in law Jethro am come unto thee, and thy wife, and her two sons with her."],
    ['Exodus','Exo','18','007','7',"And Moses went out to meet his father in law, and did obeisance, and kissed him; and they asked each other of \emph{their} welfare; and they came into the tent."],
    ['Exodus','Exo','18','008','8',"And Moses told his father in law all that the LORD had done unto Pharaoh and to the Egyptians for Israel's sake, \emph{and} all the travail that had come upon them by the way, and \emph{how} the LORD delivered them."],
    ['Exodus','Exo','18','009','9',"And Jethro rejoiced for all the goodness which the LORD had done to Israel, whom he had delivered out of the hand of the Egyptians."],
['Exodus','Exo','18','010','10',"And Jethro said, Blessed \emph{be} the LORD, who hath delivered you out of the hand of the Egyptians, and out of the hand of Pharaoh, who hath delivered the people from under the hand of the Egyptians."],
['Exodus','Exo','18','011','11',"Now I know that the LORD \emph{is} greater than all gods: for in the thing wherein they dealt proudly \emph{he} \emph{was} above them."],
['Exodus','Exo','18','012','12',"And Jethro, Moses' father in law, took a burnt offering and sacrifices for God: and Aaron came, and all the elders of Israel, to eat bread with Moses' father in law before God."],
['Exodus','Exo','18','013','13',"And it came to pass on the morrow, that Moses sat to judge the people: and the people stood by Moses from the morning unto the evening."],
['Exodus','Exo','18','014','14',"And when Moses' father in law saw all that he did to the people, he said, What \emph{is} this thing that thou doest to the people? why sittest thou thyself alone, and all the people stand by thee from morning unto even?"],
['Exodus','Exo','18','015','15',"And Moses said unto his father in law, Because the people come unto me to enquire of God:"],
['Exodus','Exo','18','016','16',"When they have a matter, they come unto me; and I judge between one and another, and I do make \emph{them} know the statutes of God, and his laws."],
['Exodus','Exo','18','017','17',"And Moses' father in law said unto him, The thing that thou doest \emph{is} not good."],
['Exodus','Exo','18','018','18',"Thou wilt surely wear away, both thou, and this people that \emph{is} with thee: for this thing \emph{is} too heavy for thee; thou art not able to perform it thyself alone."],
['Exodus','Exo','18','019','19',"Hearken now unto my voice, I will give thee counsel, and God shall be with thee: Be thou for the people to God-ward, that thou mayest bring the causes unto God:"],
['Exodus','Exo','18','020','20',"And thou shalt teach them ordinances and laws, and shalt shew them the way wherein they must walk, and the work that they must do."],
['Exodus','Exo','18','021','21',"Moreover thou shalt provide out of all the people able men, such as fear God, men of truth, hating covetousness; and place \emph{such} over them, \emph{to} \emph{be} rulers of thousands, \emph{and} rulers of hundreds, rulers of fifties, and rulers of tens:"],
    ['Exodus','Exo','18','022','22',"And let them judge the people at all seasons: and it shall be, \emph{that} every great matter they shall bring unto thee, but every small matter they shall judge: so shall it be easier for thyself, and they shall bear \emph{the} \emph{burden} with thee."],
    ['Exodus','Exo','18','023','23',"If thou shalt do this thing, and God command thee \emph{so}, then thou shalt be able to endure, and all this people shall also go to their place in peace."],
    ['Exodus','Exo','18','024','24',"So Moses hearkened to the voice of his father in law, and did all that he had said."],
    ['Exodus','Exo','18','025','25',"And Moses chose able men out of all Israel, and made them heads over the people, rulers of thousands, rulers of hundreds, rulers of fifties, and rulers of tens."],
    ['Exodus','Exo','18','026','26',"And they judged the people at all seasons: the hard causes they brought unto Moses, but every small matter they judged themselves."],
    ['Exodus','Exo','18','027','27',"And Moses let his father in law depart; and he went his way into his own land."]
]     

Exodus19 = [
    ['Exodus','Exo','19','001','1',"In the third month, when the children of Israel were gone forth out of the land of Egypt, the same day came they \emph{into} the wilderness of Sinai."],
    ['Exodus','Exo','19','002','2',"For they were departed from Rephidim, and were come \emph{to} the desert of Sinai, and had pitched in the wilderness; and there Israel camped before the mount."],
    ['Exodus','Exo','19','003','3',"And Moses went up unto God, and the LORD called unto him out of the mountain, saying, Thus shalt thou say to the house of Jacob, and tell the children of Israel;"],
    ['Exodus','Exo','19','004','4',"Ye have seen what I did unto the Egyptians, and \emph{how} I bare you on eagles' wings, and brought you unto myself."],
    ['Exodus','Exo','19','005','5',"Now therefore, if ye will obey my voice indeed, and keep my covenant, then ye shall be a peculiar treasure unto me above all people: for all the earth \emph{is} mine:"],
    ['Exodus','Exo','19','006','6',"And ye shall be unto me a kingdom of priests, and an holy nation. These \emph{are} the words which thou shalt speak unto the children of Israel."],
['Exodus','Exo','19','007','7',"And Moses came and called for the elders of the people, and laid before their faces all these words which the LORD commanded him."],
['Exodus','Exo','19','008','8',"And all the people answered together, and said, All that the LORD hath spoken we will do. And Moses returned the words of the people unto the LORD."],
['Exodus','Exo','19','009','9',"And the LORD said unto Moses, Lo, I come unto thee in a thick cloud, that the people may hear when I speak with thee, and believe thee for ever. And Moses told the words of the people unto the LORD."],
['Exodus','Exo','19','010','10',"And the LORD said unto Moses, Go unto the people, and sanctify them to day and to morrow, and let them wash their clothes,"],
['Exodus','Exo','19','011','11',"And be ready against the third day: for the third day the LORD will come down in the sight of all the people upon mount Sinai."],
['Exodus','Exo','19','012','12',"And thou shalt set bounds unto the people round about, saying, Take heed to yourselves, \emph{that} \emph{ye} go \emph{not} up into the mount, or touch the border of it: whosoever toucheth the mount shall be surely put to death:"],
['Exodus','Exo','19','013','13',"There shall not an hand touch it, but he shall surely be stoned, or shot through; whether \emph{it} \emph{be} beast or man, it shall not live: when the trumpet soundeth long, they shall come up to the mount."],
['Exodus','Exo','19','014','14',"And Moses went down from the mount unto the people, and sanctified the people; and they washed their clothes."],
['Exodus','Exo','19','015','15',"And he said unto the people, Be ready against the third day: come not at \emph{your} wives."],
['Exodus','Exo','19','016','16',"And it came to pass on the third day in the morning, that there were thunders and lightnings, and a thick cloud upon the mount, and the voice of the trumpet exceeding loud; so that all the people that \emph{was} in the camp trembled."],
['Exodus','Exo','19','017','17',"And Moses brought forth the people out of the camp to meet with God; and they stood at the nether part of the mount."],
['Exodus','Exo','19','018','18',"And mount Sinai was altogether on a smoke, because the LORD descended upon it in fire: and the smoke thereof ascended as the smoke of a furnace, and the whole mount quaked greatly."],
['Exodus','Exo','19','019','19',"And when the voice of the trumpet sounded long, and waxed louder and louder, Moses spake, and God answered him by a voice."],
['Exodus','Exo','19','020','20',"And the LORD came down upon mount Sinai, on the top of the mount: and the LORD called Moses \emph{up} to the top of the mount; and Moses went up."],
['Exodus','Exo','19','021','21',"And the LORD said unto Moses, Go down, charge the people, lest they break through unto the LORD to gaze, and many of them perish."],
['Exodus','Exo','19','022','22',"And let the priests also, which come near to the LORD, sanctify themselves, lest the LORD break forth upon them."],
['Exodus','Exo','19','023','23',"And Moses said unto the LORD, The people cannot come up to mount Sinai: for thou chargedst us, saying, Set bounds about the mount, and sanctify it."],
    ['Exodus','Exo','19','024','24',"And the LORD said unto him, Away, get thee down, and thou shalt come up, thou, and Aaron with thee: but let not the priests and the people break through to come up unto the LORD, lest he break forth upon them."],
    ['Exodus','Exo','19','025','25',"So Moses went down unto the people, and spake unto them."]
]  
    
    
Exodus20 = [
    ['Exodus','Exo','20','001','1',"And God spake all these words, saying,"],
['Exodus','Exo','20','002','2',"I \emph{am} the LORD thy God, which have brought thee out of the land of Egypt, out of the house of bondage."],
['Exodus','Exo','20','003','3',"Thou shalt have no other gods before me."],
['Exodus','Exo','20','004','4',"Thou shalt not make unto thee any graven image, or any likeness \emph{of} \emph{any} \emph{thing} that \emph{is} in heaven above, or that \emph{is} in the earth beneath, or that \emph{is} in the water under the earth:"],
['Exodus','Exo','20','005','5',"Thou shalt not bow down thyself to them, nor serve them: for I the LORD thy God \emph{am} a jealous God, visiting the iniquity of the fathers upon the children unto the third and fourth \emph{generation} of them that hate me;"],
['Exodus','Exo','20','006','6',"And shewing mercy unto thousands of them that love me, and keep my commandments."],
['Exodus','Exo','20','007','7',"Thou shalt not take the name of the LORD thy God in vain; for the LORD will not hold him guiltless that taketh his name in vain."],
['Exodus','Exo','20','008','8',"Remember the sabbath day, to keep it holy."],
['Exodus','Exo','20','009','9',"Six days shalt thou labour, and do all thy work:"],
['Exodus','Exo','20','010','10',"But the seventh day \emph{is} the sabbath of the LORD thy God: \emph{in} \emph{it} thou shalt not do any work, thou, nor thy son, nor thy daughter, thy manservant, nor thy maidservant, nor thy cattle, nor thy stranger that \emph{is} within thy gates:"],
['Exodus','Exo','20','011','11',"For \emph{in} six days the LORD made heaven and earth, the sea, and all that in them \emph{is}, and rested the seventh day: wherefore the LORD blessed the sabbath day, and hallowed it."],
['Exodus','Exo','20','012','12',"Honour thy father and thy mother: that thy days may be long upon the land which the LORD thy God giveth thee."],
['Exodus','Exo','20','013','13',"Thou shalt not kill."],
['Exodus','Exo','20','014','14',"Thou shalt not commit adultery."],
['Exodus','Exo','20','015','15',"Thou shalt not steal."],
['Exodus','Exo','20','016','16',"Thou shalt not bear false witness against thy neighbour."],
['Exodus','Exo','20','017','17',"Thou shalt not covet thy neighbour's house, thou shalt not covet thy neighbour's wife, nor his manservant, nor his maidservant, nor his ox, nor his ass, nor any thing that \emph{is} thy neighbour's."],
['Exodus','Exo','20','018','18',"And all the people saw the thunderings, and the lightnings, and the noise of the trumpet, and the mountain smoking: and when the people saw \emph{it}, they removed, and stood afar off."],
['Exodus','Exo','20','019','19',"And they said unto Moses, Speak thou with us, and we will hear: but let not God speak with us, lest we die."],
['Exodus','Exo','20','020','20',"And Moses said unto the people, Fear not: for God is come to prove you, and that his fear may be before your faces, that ye sin not."],
['Exodus','Exo','20','021','21',"And the people stood afar off, and Moses drew near unto the thick darkness where God \emph{was}."],
['Exodus','Exo','20','022','22',"And the LORD said unto Moses, Thus thou shalt say unto the children of Israel, Ye have seen that I have talked with you from heaven."],
['Exodus','Exo','20','023','23',"Ye shall not make with me gods of silver, neither shall ye make unto you gods of gold."],
['Exodus','Exo','20','024','24',"An altar of earth thou shalt make unto me, and shalt sacrifice thereon thy burnt offerings, and thy peace offerings, thy sheep, and thine oxen: in all places where I record my name I will come unto thee, and I will bless thee."],
['Exodus','Exo','20','025','25',"And if thou wilt make me an altar of stone, thou shalt not build it of hewn stone: for if thou lift up thy tool upon it, thou hast polluted it."],
['Exodus','Exo','20','026','26',"Neither shalt thou go up by steps unto mine altar, that thy nakedness be not discovered thereon."]
]

Exodus21 = [
    ['Exodus','Exo','21','001','1',"Now these \emph{are} the judgments which thou shalt set before them."],
['Exodus','Exo','21','002','2',"If thou buy an Hebrew servant, six years he shall serve: and in the seventh he shall go out free for nothing."],
['Exodus','Exo','21','003','3',"If he came in by himself, he shall go out by himself: if he were married, then his wife shall go out with him."],
['Exodus','Exo','21','004','4',"If his master have given him a wife, and she have born him sons or daughters; the wife and her children shall be her master's, and he shall go out by himself."],
['Exodus','Exo','21','005','5',"And if the servant shall plainly say, I love my master, my wife, and my children; I will not go out free:"],
['Exodus','Exo','21','006','6',"Then his master shall bring him unto the judges; he shall also bring him to the door, or unto the door post; and his master shall bore his ear through with an aul; and he shall serve him for ever."],
['Exodus','Exo','21','007','7',"And if a man sell his daughter to be a maidservant, she shall not go out as the menservants do."],
['Exodus','Exo','21','008','8',"If she please not her master, who hath betrothed her to himself, then shall he let her be redeemed: to sell her unto a strange nation he shall have no power, seeing he hath dealt deceitfully with her."],
['Exodus','Exo','21','009','9',"And if he have betrothed her unto his son, he shall deal with her after the manner of daughters."],
['Exodus','Exo','21','010','10',"If he take him another \emph{wife}; her food, her raiment, and her duty of marriage, shall he not diminish."],
['Exodus','Exo','21','011','11',"And if he do not these three unto her, then shall she go out free without money."],
['Exodus','Exo','21','012','12',"He that smiteth a man, so that he die, shall be surely put to death."],
['Exodus','Exo','21','013','13',"And if a man lie not in wait, but God deliver \emph{him} into his hand; then I will appoint thee a place whither he shall flee."],
['Exodus','Exo','21','014','14',"But if a man come presumptuously upon his neighbour, to slay him with guile; thou shalt take him from mine altar, that he may die."],
['Exodus','Exo','21','015','15',"And he that smiteth his father, or his mother, shall be surely put to death."],
['Exodus','Exo','21','016','16',"And he that stealeth a man, and selleth him, or if he be found in his hand, he shall surely be put to death."],
['Exodus','Exo','21','017','17',"And he that curseth his father, or his mother, shall surely be put to death."],
['Exodus','Exo','21','018','18',"And if men strive together, and one smite another with a stone, or with \emph{his} fist, and he die not, but keepeth \emph{his} bed:"],
['Exodus','Exo','21','019','19',"If he rise again, and walk abroad upon his staff, then shall he that smote \emph{him} be quit: only he shall pay \emph{for} the loss of his time, and shall cause \emph{him} to be thoroughly healed."],
['Exodus','Exo','21','020','20',"And if a man smite his servant, or his maid, with a rod, and he die under his hand; he shall be surely punished."],
['Exodus','Exo','21','021','21',"Notwithstanding, if he continue a day or two, he shall not be punished: for he \emph{is} his money."],
['Exodus','Exo','21','022','22',"If men strive, and hurt a woman with child, so that her fruit depart \emph{from} \emph{her}, and yet no mischief follow: he shall be surely punished, according as the woman's husband will lay upon him; and he shall pay as the judges \emph{determine}."],
['Exodus','Exo','21','023','23',"And if \emph{any} mischief follow, then thou shalt give life for life,"],
['Exodus','Exo','21','024','24',"Eye for eye, tooth for tooth, hand for hand, foot for foot,"],
['Exodus','Exo','21','025','25',"Burning for burning, wound for wound, stripe for stripe."],
['Exodus','Exo','21','026','26',"And if a man smite the eye of his servant, or the eye of his maid, that it perish; he shall let him go free for his eye's sake."],
['Exodus','Exo','21','027','27',"And if he smite out his manservant's tooth, or his maidservant's tooth; he shall let him go free for his tooth's sake."],
['Exodus','Exo','21','028','28',"If an ox gore a man or a woman, that they die: then the ox shall be surely stoned, and his flesh shall not be eaten; but the owner of the ox \emph{shall} \emph{be} quit."],
['Exodus','Exo','21','029','29',"But if the ox were wont to push with his horn in time past, and it hath been testified to his owner, and he hath not kept him in, but that he hath killed a man or a woman; the ox shall be stoned, and his owner also shall be put to death."],
['Exodus','Exo','21','030','30',"If there be laid on him a sum of money, then he shall give for the ransom of his life whatsoever is laid upon him."],
['Exodus','Exo','21','031','31',"Whether he have gored a son, or have gored a daughter, according to this judgment shall it be done unto him."],
['Exodus','Exo','21','032','32',"If the ox shall push a manservant or a maidservant; he shall give unto their master thirty shekels of silver, and the ox shall be stoned."],
['Exodus','Exo','21','033','33',"And if a man shall open a pit, or if a man shall dig a pit, and not cover it, and an ox or an ass fall therein;"],
['Exodus','Exo','21','034','34',"The owner of the pit shall make \emph{it} good, \emph{and} give money unto the owner of them; and the dead \emph{beast} shall be his."],
['Exodus','Exo','21','035','35',"And if one man's ox hurt another's, that he die; then they shall sell the live ox, and divide the money of it; and the dead \emph{ox} also they shall divide."],
['Exodus','Exo','21','036','36',"Or if it be known that the ox hath used to push in time past, and his owner hath not kept him in; he shall surely pay ox for ox; and the dead shall be his own."]
]   

Exodus22 = [
    ['Exodus','Exo','22','001','1',"If a man shall steal an ox, or a sheep, and kill it, or sell it; he shall restore five oxen for an ox, and four sheep for a sheep."],
    ['Exodus','Exo','22','002','2',"If a thief be found breaking up, and be smitten that he die, \emph{there} \emph{shall} no blood \emph{be} \emph{shed} for him."],
    ['Exodus','Exo','22','003','3',"If the sun be risen upon him, \emph{there} \emph{shall} \emph{be} blood \emph{shed} for him; \emph{for} he should make full restitution; if he have nothing, then he shall be sold for his theft."],
    ['Exodus','Exo','22','004','4',"If the theft be certainly found in his hand alive, whether it be ox, or ass, or sheep; he shall restore double."],
    ['Exodus','Exo','22','005','5',"If a man shall cause a field or vineyard to be eaten, and shall put in his beast, and shall feed in another man's field; of the best of his own field, and of the best of his own vineyard, shall he make restitution."],
    ['Exodus','Exo','22','006','6',"If fire break out, and catch in thorns, so that the stacks of corn, or the standing corn, or the field, be consumed \emph{therewith}; he that kindled the fire shall surely make restitution."],
    ['Exodus','Exo','22','007','7',"If a man shall deliver unto his neighbour money or stuff to keep, and it be stolen out of the man's house; if the thief be found, let him pay double."],
    ['Exodus','Exo','22','008','8',"If the thief be not found, then the master of the house shall be brought unto the judges, \emph{to} \emph{see} whether he have put his hand unto his neighbour's goods."],
['Exodus','Exo','22','009','9',"For all manner of trespass, \emph{whether} \emph{it} \emph{be} for ox, for ass, for sheep, for raiment, \emph{or} for any manner of lost thing, which \emph{another} challengeth to be his, the cause of both parties shall come before the judges; \emph{and} whom the judges shall condemn, he shall pay double unto his neighbour."],
['Exodus','Exo','22','010','10',"If a man deliver unto his neighbour an ass, or an ox, or a sheep, or any beast, to keep; and it die, or be hurt, or driven away, no man seeing \emph{it}:"],
['Exodus','Exo','22','011','11',"\emph{Then} shall an oath of the LORD be between them both, that he hath not put his hand unto his neighbour's goods; and the owner of it shall accept \emph{thereof}, and he shall not make \emph{it} good."],
['Exodus','Exo','22','012','12',"And if it be stolen from him, he shall make restitution unto the owner thereof."],
['Exodus','Exo','22','013','13',"If it be torn in pieces, \emph{then} let him bring it \emph{for} witness, \emph{and} he shall not make good that which was torn."],
['Exodus','Exo','22','014','14',"And if a man borrow \emph{ought} of his neighbour, and it be hurt, or die, the owner thereof \emph{being} not with it, he shall surely make \emph{it} good."],
['Exodus','Exo','22','015','15',"\emph{But} if the owner thereof \emph{be} with it, he shall not make \emph{it} good: if it \emph{be} an hired \emph{thing}, it came for his hire."],
['Exodus','Exo','22','016','16',"And if a man entice a maid that is not betrothed, and lie with her, he shall surely endow her to be his wife."],
['Exodus','Exo','22','017','17',"If her father utterly refuse to give her unto him, he shall pay money according to the dowry of virgins."],
['Exodus','Exo','22','018','18',"Thou shalt not suffer a witch to live."],
['Exodus','Exo','22','019','19',"Whosoever lieth with a beast shall surely be put to death."],
['Exodus','Exo','22','020','20',"He that sacrificeth unto \emph{any} god, save unto the LORD only, he shall be utterly destroyed."],
['Exodus','Exo','22','021','21',"Thou shalt neither vex a stranger, nor oppress him: for ye were strangers in the land of Egypt."],
['Exodus','Exo','22','022','22',"Ye shall not afflict any widow, or fatherless child."],
['Exodus','Exo','22','023','23',"If thou afflict them in any wise, and they cry at all unto me, I will surely hear their cry;"],
['Exodus','Exo','22','024','24',"And my wrath shall wax hot, and I will kill you with the sword; and your wives shall be widows, and your children fatherless."],
    ['Exodus','Exo','22','025','25',"If thou lend money to \emph{any} \emph{of} my people \emph{that} \emph{is} poor by thee, thou shalt not be to him as an usurer, neither shalt thou lay upon him usury."],
    ['Exodus','Exo','22','026','26',"If thou at all take thy neighbour's raiment to pledge, thou shalt deliver it unto him by that the sun goeth down:"],
    ['Exodus','Exo','22','027','27',"For that \emph{is} his covering only, it \emph{is} his raiment for his skin: wherein shall he sleep? and it shall come to pass, when he crieth unto me, that I will hear; for I \emph{am} gracious."],
    ['Exodus','Exo','22','028','28',"Thou shalt not revile the gods, nor curse the ruler of thy people."],
    ['Exodus','Exo','22','029','29',"Thou shalt not delay \emph{to} \emph{offer} the first of thy ripe fruits, and of thy liquors: the firstborn of thy sons shalt thou give unto me."],
    ['Exodus','Exo','22','030','30',"Likewise shalt thou do with thine oxen, \emph{and} with thy sheep: seven days it shall be with his dam; on the eighth day thou shalt give it me."],
    ['Exodus','Exo','22','031','31',"And ye shall be holy men unto me: neither shall ye eat \emph{any} flesh \emph{that} \emph{is} torn of beasts in the field; ye shall cast it to the dogs."]
]   

Exodus23 = [
    ['Exodus','Exo','23','001','1',"Thou shalt not raise a false report: put not thine hand with the wicked to be an unrighteous witness."],
['Exodus','Exo','23','002','2',"Thou shalt not follow a multitude to \emph{do} evil; neither shalt thou speak in a cause to decline after many to wrest \emph{judgment}:"],
['Exodus','Exo','23','003','3',"Neither shalt thou countenance a poor man in his cause."],
['Exodus','Exo','23','004','4',"If thou meet thine enemy's ox or his ass going astray, thou shalt surely bring it back to him again."],
['Exodus','Exo','23','005','5',"If thou see the ass of him that hateth thee lying under his burden, and wouldest forbear to help him, thou shalt surely help with him."],
['Exodus','Exo','23','006','6',"Thou shalt not wrest the judgment of thy poor in his cause."],
['Exodus','Exo','23','007','7',"Keep thee far from a false matter; and the innocent and righteous slay thou not: for I will not justify the wicked."],
['Exodus','Exo','23','008','8',"And thou shalt take no gift: for the gift blindeth the wise, and perverteth the words of the righteous."],
['Exodus','Exo','23','009','9',"Also thou shalt not oppress a stranger: for ye know the heart of a stranger, seeing ye were strangers in the land of Egypt."],
['Exodus','Exo','23','010','10',"And six years thou shalt sow thy land, and shalt gather in the fruits thereof:"],
['Exodus','Exo','23','011','11',"But the seventh \emph{year} thou shalt let it rest and lie still; that the poor of thy people may eat: and what they leave the beasts of the field shall eat. In like manner thou shalt deal with thy vineyard, \emph{and} with thy oliveyard."],
['Exodus','Exo','23','012','12',"Six days thou shalt do thy work, and on the seventh day thou shalt rest: that thine ox and thine ass may rest, and the son of thy handmaid, and the stranger, may be refreshed."],
['Exodus','Exo','23','013','13',"And in all \emph{things} that I have said unto you be circumspect: and make no mention of the name of other gods, neither let it be heard out of thy mouth."],
['Exodus','Exo','23','014','14',"Three times thou shalt keep a feast unto me in the year."],
['Exodus','Exo','23','015','15',"Thou shalt keep the feast of unleavened bread: (thou shalt eat unleavened bread seven days, as I commanded thee, in the time appointed of the month Abib; for in it thou camest out from Egypt: and none shall appear before me empty:)"],
['Exodus','Exo','23','016','16',"And the feast of harvest, the firstfruits of thy labours, which thou hast sown in the field: and the feast of ingathering, \emph{which} \emph{is} in the end of the year, when thou hast gathered in thy labours out of the field."],
['Exodus','Exo','23','017','17',"Three times in the year all thy males shall appear before the Lord GOD."],
['Exodus','Exo','23','018','18',"Thou shalt not offer the blood of my sacrifice with leavened bread; neither shall the fat of my sacrifice remain until the morning."],
['Exodus','Exo','23','019','19',"The first of the firstfruits of thy land thou shalt bring into the house of the LORD thy God. Thou shalt not seethe a kid in his mother's milk."],
['Exodus','Exo','23','020','20',"Behold, I send an Angel before thee, to keep thee in the way, and to bring thee into the place which I have prepared."],
['Exodus','Exo','23','021','21',"Beware of him, and obey his voice, provoke him not; for he will not pardon your transgressions: for my name \emph{is} in him."],
['Exodus','Exo','23','022','22',"But if thou shalt indeed obey his voice, and do all that I speak; then I will be an enemy unto thine enemies, and an adversary unto thine adversaries."],
['Exodus','Exo','23','023','23',"For mine Angel shall go before thee, and bring thee in unto the Amorites, and the Hittites, and the Perizzites, and the Canaanites, the Hivites, and the Jebusites: and I will cut them off."],
['Exodus','Exo','23','024','24',"Thou shalt not bow down to their gods, nor serve them, nor do after their works: but thou shalt utterly overthrow them, and quite break down their images."],
['Exodus','Exo','23','025','25',"And ye shall serve the LORD your God, and he shall bless thy bread, and thy water; and I will take sickness away from the midst of thee."],
['Exodus','Exo','23','026','26',"There shall nothing cast their young, nor be barren, in thy land: the number of thy days I will fulfil."],
['Exodus','Exo','23','027','27',"I will send my fear before thee, and will destroy all the people to whom thou shalt come, and I will make all thine enemies turn their backs unto thee."],
['Exodus','Exo','23','028','28',"And I will send hornets before thee, which shall drive out the Hivite, the Canaanite, and the Hittite, from before thee."],
['Exodus','Exo','23','029','29',"I will not drive them out from before thee in one year; lest the land become desolate, and the beast of the field multiply against thee."],
['Exodus','Exo','23','030','30',"By little and little I will drive them out from before thee, until thou be increased, and inherit the land."],
['Exodus','Exo','23','031','31',"And I will set thy bounds from the Red sea even unto the sea of the Philistines, and from the desert unto the river: for I will deliver the inhabitants of the land into your hand; and thou shalt drive them out before thee."],
['Exodus','Exo','23','032','32',"Thou shalt make no covenant with them, nor with their gods."],
['Exodus','Exo','23','033','33',"They shall not dwell in thy land, lest they make thee sin against me: for if thou serve their gods, it will surely be a snare unto thee."]
]   
     
Exodus24 = [
    ['Exodus','Exo','24','001','1',"And he said unto Moses, Come up unto the LORD, thou, and Aaron, Nadab, and Abihu, and seventy of the elders of Israel; and worship ye afar off."],
    ['Exodus','Exo','24','002','2',"And Moses alone shall come near the LORD: but they shall not come nigh; neither shall the people go up with him."],
    ['Exodus','Exo','24','003','3',"And Moses came and told the people all the words of the LORD, and all the judgments: and all the people answered with one voice, and said, All the words which the LORD hath said will we do."],
    ['Exodus','Exo','24','004','4',"And Moses wrote all the words of the LORD, and rose up early in the morning, and builded an altar under the hill, and twelve pillars, according to the twelve tribes of Israel."],
    ['Exodus','Exo','24','005','5',"And he sent young men of the children of Israel, which offered burnt offerings, and sacrificed peace offerings of oxen unto the LORD."],
    ['Exodus','Exo','24','006','6',"And Moses took half of the blood, and put \emph{it} in basons; and half of the blood he sprinkled on the altar."],
    ['Exodus','Exo','24','007','7',"And he took the book of the covenant, and read in the audience of the people: and they said, All that the LORD hath said will we do, and be obedient."],
    ['Exodus','Exo','24','008','8',"And Moses took the blood, and sprinkled \emph{it} on the people, and said, Behold the blood of the covenant, which the LORD hath made with you concerning all these words."],
    ['Exodus','Exo','24','009','9',"Then went up Moses, and Aaron, Nadab, and Abihu, and seventy of the elders of Israel:"],
    ['Exodus','Exo','24','010','10',"And they saw the God of Israel: and \emph{there} \emph{was} under his feet as it were a paved work of a sapphire stone, and as it were the body of heaven in \emph{his} clearness."],
    ['Exodus','Exo','24','011','11',"And upon the nobles of the children of Israel he laid not his hand: also they saw God, and did eat and drink."],
    ['Exodus','Exo','24','012','12',"And the LORD said unto Moses, Come up to me into the mount, and be there: and I will give thee tables of stone, and a law, and commandments which I have written; that thou mayest teach them."],
    ['Exodus','Exo','24','013','13',"And Moses rose up, and his minister Joshua: and Moses went up into the mount of God."],
    ['Exodus','Exo','24','014','14',"And he said unto the elders, Tarry ye here for us, until we come again unto you: and, behold, Aaron and Hur \emph{are} with you: if any man have any matters to do, let him come unto them."],
    ['Exodus','Exo','24','015','15',"And Moses went up into the mount, and a cloud covered the mount."],
    ['Exodus','Exo','24','016','16',"And the glory of the LORD abode upon mount Sinai, and the cloud covered it six days: and the seventh day he called unto Moses out of the midst of the cloud."],
    ['Exodus','Exo','24','017','17',"And the sight of the glory of the LORD \emph{was} like devouring fire on the top of the mount in the eyes of the children of Israel."],
    ['Exodus','Exo','24','018','18',"And Moses went into the midst of the cloud, and gat him up into the mount: and Moses was in the mount forty days and forty nights."]
]    

Exodus25 = [
    ['Exodus','Exo','25','001','1',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','25','002','2',"Speak unto the children of Israel, that they bring me an offering: of every man that giveth it willingly with his heart ye shall take my offering."],
['Exodus','Exo','25','003','3',"And this \emph{is} the offering which ye shall take of them; gold, and silver, and brass,"],
['Exodus','Exo','25','004','4',"And blue, and purple, and scarlet, and fine linen, and goats' \emph{hair},"],
['Exodus','Exo','25','005','5',"And rams' skins dyed red, and badgers' skins, and shittim wood,"],
['Exodus','Exo','25','006','6',"Oil for the light, spices for anointing oil, and for sweet incense,"],
['Exodus','Exo','25','007','7',"Onyx stones, and stones to be set in the ephod, and in the breastplate."],
['Exodus','Exo','25','008','8',"And let them make me a sanctuary; that I may dwell among them."],
['Exodus','Exo','25','009','9',"According to all that I shew thee, \emph{after} the pattern of the tabernacle, and the pattern of all the instruments thereof, even so shall ye make \emph{it}."],
['Exodus','Exo','25','010','10',"And they shall make an ark \emph{of} shittim wood: two cubits and a half \emph{shall} \emph{be} the length thereof, and a cubit and a half the breadth thereof, and a cubit and a half the height thereof."],
['Exodus','Exo','25','011','11',"And thou shalt overlay it with pure gold, within and without shalt thou overlay it, and shalt make upon it a crown of gold round about."],
['Exodus','Exo','25','012','12',"And thou shalt cast four rings of gold for it, and put \emph{them} in the four corners thereof; and two rings \emph{shall} \emph{be} in the one side of it, and two rings in the other side of it."],
['Exodus','Exo','25','013','13',"And thou shalt make staves \emph{of} shittim wood, and overlay them with gold."],
['Exodus','Exo','25','014','14',"And thou shalt put the staves into the rings by the sides of the ark, that the ark may be borne with them."],
['Exodus','Exo','25','015','15',"The staves shall be in the rings of the ark: they shall not be taken from it."],
['Exodus','Exo','25','016','16',"And thou shalt put into the ark the testimony which I shall give thee."],
['Exodus','Exo','25','017','17',"And thou shalt make a mercy seat \emph{of} pure gold: two cubits and a half \emph{shall} \emph{be} the length thereof, and a cubit and a half the breadth thereof."],
['Exodus','Exo','25','018','18',"And thou shalt make two cherubims \emph{of} gold, \emph{of} beaten work shalt thou make them, in the two ends of the mercy seat."],
['Exodus','Exo','25','019','19',"And make one cherub on the one end, and the other cherub on the other end: \emph{even} of the mercy seat shall ye make the cherubims on the two ends thereof."],
['Exodus','Exo','25','020','20',"And the cherubims shall stretch forth \emph{their} wings on high, covering the mercy seat with their wings, and their faces \emph{shall} \emph{look} one to another; toward the mercy seat shall the faces of the cherubims be."],
['Exodus','Exo','25','021','21',"And thou shalt put the mercy seat above upon the ark; and in the ark thou shalt put the testimony that I shall give thee."],
['Exodus','Exo','25','022','22',"And there I will meet with thee, and I will commune with thee from above the mercy seat, from between the two cherubims which \emph{are} upon the ark of the testimony, of all \emph{things} which I will give thee in commandment unto the children of Israel."],
['Exodus','Exo','25','023','23',"Thou shalt also make a table \emph{of} shittim wood: two cubits \emph{shall} \emph{be} the length thereof, and a cubit the breadth thereof, and a cubit and a half the height thereof."],
['Exodus','Exo','25','024','24',"And thou shalt overlay it with pure gold, and make thereto a crown of gold round about."],
['Exodus','Exo','25','025','25',"And thou shalt make unto it a border of an hand breadth round about, and thou shalt make a golden crown to the border thereof round about."],
['Exodus','Exo','25','026','26',"And thou shalt make for it four rings of gold, and put the rings in the four corners that \emph{are} on the four feet thereof."],
['Exodus','Exo','25','027','27',"Over against the border shall the rings be for places of the staves to bear the table."],
['Exodus','Exo','25','028','28',"And thou shalt make the staves \emph{of} shittim wood, and overlay them with gold, that the table may be borne with them."],
['Exodus','Exo','25','029','29',"And thou shalt make the dishes thereof, and spoons thereof, and covers thereof, and bowls thereof, to cover withal: \emph{of} pure gold shalt thou make them."],
['Exodus','Exo','25','030','30',"And thou shalt set upon the table shewbread before me alway."],
['Exodus','Exo','25','031','31',"And thou shalt make a candlestick \emph{of} pure gold: \emph{of} beaten work shall the candlestick be made: his shaft, and his branches, his bowls, his knops, and his flowers, shall be of the same."],
['Exodus','Exo','25','032','32',"And six branches shall come out of the sides of it; three branches of the candlestick out of the one side, and three branches of the candlestick out of the other side:"],
['Exodus','Exo','25','033','33',"Three bowls made like unto almonds, \emph{with} a knop and a flower in one branch; and three bowls made like almonds in the other branch, \emph{with} a knop and a flower: so in the six branches that come out of the candlestick."],
['Exodus','Exo','25','034','34',"And in the candlestick \emph{shall} \emph{be} four bowls made like unto almonds, \emph{with} their knops and their flowers."],
['Exodus','Exo','25','035','35',"And \emph{there} \emph{shall} \emph{be} a knop under two branches of the same, and a knop under two branches of the same, and a knop under two branches of the same, according to the six branches that proceed out of the candlestick."],
['Exodus','Exo','25','036','36',"Their knops and their branches shall be of the same: all it \emph{shall} \emph{be} one beaten work \emph{of} pure gold."],
['Exodus','Exo','25','037','37',"And thou shalt make the seven lamps thereof: and they shall light the lamps thereof, that they may give light over against it."],
['Exodus','Exo','25','038','38',"And the tongs thereof, and the snuffdishes thereof, \emph{shall} \emph{be} \emph{of} pure gold."],
['Exodus','Exo','25','039','39',"\emph{Of} a talent of pure gold shall he make it, with all these vessels."],
['Exodus','Exo','25','040','40',"And look that thou make \emph{them} after their pattern, which was shewed thee in the mount."]
]

Exodus26 = [
    ['Exodus','Exo','26','001','1',"Moreover thou shalt make the tabernacle \emph{with} ten curtains \emph{of} fine twined linen, and blue, and purple, and scarlet: \emph{with} cherubims of cunning work shalt thou make them."],
    ['Exodus','Exo','26','002','2',"The length of one curtain \emph{shall} \emph{be} eight and twenty cubits, and the breadth of one curtain four cubits: and every one of the curtains shall have one measure."],
    ['Exodus','Exo','26','003','3',"The five curtains shall be coupled together one to another; and \emph{other} five curtains \emph{shall} \emph{be} coupled one to another."],
    ['Exodus','Exo','26','004','4',"And thou shalt make loops of blue upon the edge of the one curtain from the selvedge in the coupling; and likewise shalt thou make in the uttermost edge of \emph{another} curtain, in the coupling of the second."],
    ['Exodus','Exo','26','005','5',"Fifty loops shalt thou make in the one curtain, and fifty loops shalt thou make in the edge of the curtain that \emph{is} in the coupling of the second; that the loops may take hold one of another."],
    ['Exodus','Exo','26','006','6',"And thou shalt make fifty taches of gold, and couple the curtains together with the taches: and it shall be one tabernacle."],
['Exodus','Exo','26','007','7',"And thou shalt make curtains \emph{of} goats' \emph{hair} to be a covering upon the tabernacle: eleven curtains shalt thou make."],
['Exodus','Exo','26','008','8',"The length of one curtain \emph{shall} \emph{be} thirty cubits, and the breadth of one curtain four cubits: and the eleven curtains \emph{shall} \emph{be} \emph{all} of one measure."],
['Exodus','Exo','26','009','9',"And thou shalt couple five curtains by themselves, and six curtains by themselves, and shalt double the sixth curtain in the forefront of the tabernacle."],
['Exodus','Exo','26','010','10',"And thou shalt make fifty loops on the edge of the one curtain \emph{that} \emph{is} outmost in the coupling, and fifty loops in the edge of the curtain which coupleth the second."],
['Exodus','Exo','26','011','11',"And thou shalt make fifty taches of brass, and put the taches into the loops, and couple the tent together, that it may be one."],
['Exodus','Exo','26','012','12',"And the remnant that remaineth of the curtains of the tent, the half curtain that remaineth, shall hang over the backside of the tabernacle."],
['Exodus','Exo','26','013','13',"And a cubit on the one side, and a cubit on the other side of that which remaineth in the length of the curtains of the tent, it shall hang over the sides of the tabernacle on this side and on that side, to cover it."],
['Exodus','Exo','26','014','14',"And thou shalt make a covering for the tent \emph{of} rams' skins dyed red, and a covering above \emph{of} badgers' skins."],
['Exodus','Exo','26','015','15',"And thou shalt make boards for the tabernacle \emph{of} shittim wood standing up."],
['Exodus','Exo','26','016','16',"Ten cubits \emph{shall} \emph{be} the length of a board, and a cubit and a half \emph{shall} \emph{be} the breadth of one board."],
['Exodus','Exo','26','017','17',"Two tenons \emph{shall} \emph{there} \emph{be} in one board, set in order one against another: thus shalt thou make for all the boards of the tabernacle."],
['Exodus','Exo','26','018','18',"And thou shalt make the boards for the tabernacle, twenty boards on the south side southward."],
['Exodus','Exo','26','019','19',"And thou shalt make forty sockets of silver under the twenty boards; two sockets under one board for his two tenons, and two sockets under another board for his two tenons."],
['Exodus','Exo','26','020','20',"And for the second side of the tabernacle on the north side \emph{there} \emph{shall} \emph{be} twenty boards:"],
['Exodus','Exo','26','021','21',"And their forty sockets \emph{of} silver; two sockets under one board, and two sockets under another board."],
['Exodus','Exo','26','022','22',"And for the sides of the tabernacle westward thou shalt make six boards."],
['Exodus','Exo','26','023','23',"And two boards shalt thou make for the corners of the tabernacle in the two sides."],
['Exodus','Exo','26','024','24',"And they shall be coupled together beneath, and they shall be coupled together above the head of it unto one ring: thus shall it be for them both; they shall be for the two corners."],
['Exodus','Exo','26','025','25',"And they shall be eight boards, and their sockets \emph{of} silver, sixteen sockets; two sockets under one board, and two sockets under another board."],
['Exodus','Exo','26','026','26',"And thou shalt make bars \emph{of} shittim wood; five for the boards of the one side of the tabernacle,"],
['Exodus','Exo','26','027','27',"And five bars for the boards of the other side of the tabernacle, and five bars for the boards of the side of the tabernacle, for the two sides westward."],
['Exodus','Exo','26','028','28',"And the middle bar in the midst of the boards shall reach from end to end."],
['Exodus','Exo','26','029','29',"And thou shalt overlay the boards with gold, and make their rings \emph{of} gold \emph{for} places for the bars: and thou shalt overlay the bars with gold."],
['Exodus','Exo','26','030','30',"And thou shalt rear up the tabernacle according to the fashion thereof which was shewed thee in the mount."],
    ['Exodus','Exo','26','031','31',"And thou shalt make a vail \emph{of} blue, and purple, and scarlet, and fine twined linen of cunning work: with cherubims shall it be made:"],
    ['Exodus','Exo','26','032','32',"And thou shalt hang it upon four pillars of shittim \emph{wood} overlaid with gold: their hooks \emph{shall} \emph{be} \emph{of} gold, upon the four sockets of silver."],
    ['Exodus','Exo','26','033','33',"And thou shalt hang up the vail under the taches, that thou mayest bring in thither within the vail the ark of the testimony: and the vail shall divide unto you between the holy \emph{place} and the most holy."],
    ['Exodus','Exo','26','034','34',"And thou shalt put the mercy seat upon the ark of the testimony in the most holy \emph{place}."],
    ['Exodus','Exo','26','035','35',"And thou shalt set the table without the vail, and the candlestick over against the table on the side of the tabernacle toward the south: and thou shalt put the table on the north side."],
    ['Exodus','Exo','26','036','36',"And thou shalt make an hanging for the door of the tent, \emph{of} blue, and purple, and scarlet, and fine twined linen, wrought with needlework."],
    ['Exodus','Exo','26','037','37',"And thou shalt make for the hanging five pillars \emph{of} shittim \emph{wood}, and overlay them with gold, \emph{and} their hooks \emph{shall} \emph{be} \emph{of} gold: and thou shalt cast five sockets of brass for them."]
]  
     
Exodus27 = [
    ['Exodus','Exo','27','001','1',"And thou shalt make an altar \emph{of} shittim wood, five cubits long, and five cubits broad; the altar shall be foursquare: and the height thereof \emph{shall} \emph{be} three cubits."],
    ['Exodus','Exo','27','002','2',"And thou shalt make the horns of it upon the four corners thereof: his horns shall be of the same: and thou shalt overlay it with brass."],
    ['Exodus','Exo','27','003','3',"And thou shalt make his pans to receive his ashes, and his shovels, and his basons, and his fleshhooks, and his firepans: all the vessels thereof thou shalt make \emph{of} brass."],
    ['Exodus','Exo','27','004','4',"And thou shalt make for it a grate of network \emph{of} brass; and upon the net shalt thou make four brasen rings in the four corners thereof."],
    ['Exodus','Exo','27','005','5',"And thou shalt put it under the compass of the altar beneath, that the net may be even to the midst of the altar."],
    ['Exodus','Exo','27','006','6',"And thou shalt make staves for the altar, staves \emph{of} shittim wood, and overlay them with brass."],
    ['Exodus','Exo','27','007','7',"And the staves shall be put into the rings, and the staves shall be upon the two sides of the altar, to bear it."],
    ['Exodus','Exo','27','008','8',"Hollow with boards shalt thou make it: as it was shewed thee in the mount, so shall they make \emph{it}."],
    ['Exodus','Exo','27','009','9',"And thou shalt make the court of the tabernacle: for the south side southward \emph{there} \emph{shall} \emph{be} hangings for the court \emph{of} fine twined linen of an hundred cubits long for one side:"],
    ['Exodus','Exo','27','010','10',"And the twenty pillars thereof and their twenty sockets \emph{shall} \emph{be} \emph{of} brass; the hooks of the pillars and their fillets \emph{shall} \emph{be} \emph{of} silver."],
    ['Exodus','Exo','27','011','11',"And likewise for the north side in length \emph{there} \emph{shall} \emph{be} hangings of an hundred \emph{cubits} long, and his twenty pillars and their twenty sockets \emph{of} brass; the hooks of the pillars and their fillets \emph{of} silver."],
    ['Exodus','Exo','27','012','12',"And \emph{for} the breadth of the court on the west side \emph{shall} \emph{be} hangings of fifty cubits: their pillars ten, and their sockets ten."],
    ['Exodus','Exo','27','013','13',"And the breadth of the court on the east side eastward \emph{shall} \emph{be} fifty cubits."],
    ['Exodus','Exo','27','014','14',"The hangings of one side \emph{of} \emph{the} \emph{gate} \emph{shall} \emph{be} fifteen cubits: their pillars three, and their sockets three."],
    ['Exodus','Exo','27','015','15',"And on the other side \emph{shall} \emph{be} hangings fifteen \emph{cubits}: their pillars three, and their sockets three."],
    ['Exodus','Exo','27','016','16',"And for the gate of the court \emph{shall} \emph{be} an hanging of twenty cubits, \emph{of} blue, and purple, and scarlet, and fine twined linen, wrought with needlework: \emph{and} their pillars \emph{shall} \emph{be} four, and their sockets four."],
    ['Exodus','Exo','27','017','17',"All the pillars round about the court \emph{shall} \emph{be} filleted with silver; their hooks \emph{shall} \emph{be} \emph{of} silver, and their sockets \emph{of} brass."],
    ['Exodus','Exo','27','018','18',"The length of the court \emph{shall} \emph{be} an hundred cubits, and the breadth fifty every where, and the height five cubits \emph{of} fine twined linen, and their sockets \emph{of} brass."],
    ['Exodus','Exo','27','019','19',"All the vessels of the tabernacle in all the service thereof, and all the pins thereof, and all the pins of the court, \emph{shall} \emph{be} \emph{of} brass."],
    ['Exodus','Exo','27','020','20',"And thou shalt command the children of Israel, that they bring thee pure oil olive beaten for the light, to cause the lamp to burn always."],
    ['Exodus','Exo','27','021','21',"In the tabernacle of the congregation without the vail, which \emph{is} before the testimony, Aaron and his sons shall order it from evening to morning before the LORD: \emph{it} \emph{shall} \emph{be} a statute for ever unto their generations on the behalf of the children of Israel."]
]

Exodus28 = [
    ['Exodus','Exo','28','001','1',"And take thou unto thee Aaron thy brother, and his sons with him, from among the children of Israel, that he may minister unto me in the priest's office, \emph{even} Aaron, Nadab and Abihu, Eleazar and Ithamar, Aaron's sons."],
    ['Exodus','Exo','28','002','2',"And thou shalt make holy garments for Aaron thy brother for glory and for beauty."],
    ['Exodus','Exo','28','003','3',"And thou shalt speak unto all \emph{that} \emph{are} wise hearted, whom I have filled with the spirit of wisdom, that they may make Aaron's garments to consecrate him, that he may minister unto me in the priest's office."],
    ['Exodus','Exo','28','004','4',"And these \emph{are} the garments which they shall make; a breastplate, and an ephod, and a robe, and a broidered coat, a mitre, and a girdle: and they shall make holy garments for Aaron thy brother, and his sons, that he may minister unto me in the priest's office."],
    ['Exodus','Exo','28','005','5',"And they shall take gold, and blue, and purple, and scarlet, and fine linen."],
    ['Exodus','Exo','28','006','6',"And they shall make the ephod \emph{of} gold, \emph{of} blue, and \emph{of} purple, \emph{of} scarlet, and fine twined linen, with cunning work."],
    ['Exodus','Exo','28','007','7',"It shall have the two shoulderpieces thereof joined at the two edges thereof; and \emph{so} it shall be joined together."],
['Exodus','Exo','28','008','8',"And the curious girdle of the ephod, which \emph{is} upon it, shall be of the same, according to the work thereof; \emph{even} \emph{of} gold, \emph{of} blue, and purple, and scarlet, and fine twined linen."],
['Exodus','Exo','28','009','9',"And thou shalt take two onyx stones, and grave on them the names of the children of Israel:"],
['Exodus','Exo','28','010','10',"Six of their names on one stone, and \emph{the} \emph{other} six names of the rest on the other stone, according to their birth."],
['Exodus','Exo','28','011','11',"With the work of an engraver in stone, \emph{like} the engravings of a signet, shalt thou engrave the two stones with the names of the children of Israel: thou shalt make them to be set in ouches of gold."],
['Exodus','Exo','28','012','12',"And thou shalt put the two stones upon the shoulders of the ephod \emph{for} stones of memorial unto the children of Israel: and Aaron shall bear their names before the LORD upon his two shoulders for a memorial."],
['Exodus','Exo','28','013','13',"And thou shalt make ouches \emph{of} gold;"],
['Exodus','Exo','28','014','14',"And two chains \emph{of} pure gold at the ends; \emph{of} wreathen work shalt thou make them, and fasten the wreathen chains to the ouches."],
['Exodus','Exo','28','015','15',"And thou shalt make the breastplate of judgment with cunning work; after the work of the ephod thou shalt make it; \emph{of} gold, \emph{of} blue, and \emph{of} purple, and \emph{of} scarlet, and \emph{of} fine twined linen, shalt thou make it."],
['Exodus','Exo','28','016','16',"Foursquare it shall be \emph{being} doubled; a span \emph{shall} \emph{be} the length thereof, and a span \emph{shall} \emph{be} the breadth thereof."],
['Exodus','Exo','28','017','17',"And thou shalt set in it settings of stones, \emph{even} four rows of stones: \emph{the} \emph{first} row \emph{shall} \emph{be} a sardius, a topaz, and a carbuncle: \emph{this} \emph{shall} \emph{be} the first row."],
['Exodus','Exo','28','018','18',"And the second row \emph{shall} \emph{be} an emerald, a sapphire, and a diamond."],
['Exodus','Exo','28','019','19',"And the third row a ligure, an agate, and an amethyst."],
['Exodus','Exo','28','020','20',"And the fourth row a beryl, and an onyx, and a jasper: they shall be set in gold in their inclosings."],
['Exodus','Exo','28','021','21',"And the stones shall be with the names of the children of Israel, twelve, according to their names, \emph{like} the engravings of a signet; every one with his name shall they be according to the twelve tribes."],
['Exodus','Exo','28','022','22',"And thou shalt make upon the breastplate chains at the ends \emph{of} wreathen work \emph{of} pure gold."],
['Exodus','Exo','28','023','23',"And thou shalt make upon the breastplate two rings of gold, and shalt put the two rings on the two ends of the breastplate."],
['Exodus','Exo','28','024','24',"And thou shalt put the two wreathen \emph{chains} of gold in the two rings \emph{which} \emph{are} on the ends of the breastplate."],
['Exodus','Exo','28','025','25',"And \emph{the} \emph{other} two ends of the two wreathen \emph{chains} thou shalt fasten in the two ouches, and put \emph{them} on the shoulderpieces of the ephod before it."],
['Exodus','Exo','28','026','26',"And thou shalt make two rings of gold, and thou shalt put them upon the two ends of the breastplate in the border thereof, which \emph{is} in the side of the ephod inward."],
['Exodus','Exo','28','027','27',"And two \emph{other} rings of gold thou shalt make, and shalt put them on the two sides of the ephod underneath, toward the forepart thereof, over against the \emph{other} coupling thereof, above the curious girdle of the ephod."],
['Exodus','Exo','28','028','28',"And they shall bind the breastplate by the rings thereof unto the rings of the ephod with a lace of blue, that \emph{it} may be above the curious girdle of the ephod, and that the breastplate be not loosed from the ephod."],
['Exodus','Exo','28','029','29',"And Aaron shall bear the names of the children of Israel in the breastplate of judgment upon his heart, when he goeth in unto the holy \emph{place}, for a memorial before the LORD continually."],
['Exodus','Exo','28','030','30',"And thou shalt put in the breastplate of judgment the Urim and the Thummim; and they shall be upon Aaron's heart, when he goeth in before the LORD: and Aaron shall bear the judgment of the children of Israel upon his heart before the LORD continually."],
['Exodus','Exo','28','031','31',"And thou shalt make the robe of the ephod all \emph{of} blue."],
['Exodus','Exo','28','032','32',"And there shall be an hole in the top of it, in the midst thereof: it shall have a binding of woven work round about the hole of it, as it were the hole of an habergeon, that it be not rent."],
['Exodus','Exo','28','033','33',"And \emph{beneath} upon the hem of it thou shalt make pomegranates \emph{of} blue, and \emph{of} purple, and \emph{of} scarlet, round about the hem thereof; and bells of gold between them round about:"],
['Exodus','Exo','28','034','34',"A golden bell and a pomegranate, a golden bell and a pomegranate, upon the hem of the robe round about."],
['Exodus','Exo','28','035','35',"And it shall be upon Aaron to minister: and his sound shall be heard when he goeth in unto the holy \emph{place} before the LORD, and when he cometh out, that he die not."],
['Exodus','Exo','28','036','36',"And thou shalt make a plate \emph{of} pure gold, and grave upon it, \emph{like} the engravings of a signet, HOLINESS TO THE LORD."],
['Exodus','Exo','28','037','37',"And thou shalt put it on a blue lace, that it may be upon the mitre; upon the forefront of the mitre it shall be."],
    ['Exodus','Exo','28','038','38',"And it shall be upon Aaron's forehead, that Aaron may bear the iniquity of the holy things, which the children of Israel shall hallow in all their holy gifts; and it shall be always upon his forehead, that they may be accepted before the LORD."],
    ['Exodus','Exo','28','039','39',"And thou shalt embroider the coat of fine linen, and thou shalt make the mitre \emph{of} fine linen, and thou shalt make the girdle \emph{of} needlework."],
    ['Exodus','Exo','28','040','40',"And for Aaron's sons thou shalt make coats, and thou shalt make for them girdles, and bonnets shalt thou make for them, for glory and for beauty."],
    ['Exodus','Exo','28','041','41',"And thou shalt put them upon Aaron thy brother, and his sons with him; and shalt anoint them, and consecrate them, and sanctify them, that they may minister unto me in the priest's office."],
    ['Exodus','Exo','28','042','42',"And thou shalt make them linen breeches to cover their nakedness; from the loins even unto the thighs they shall reach:"],
    ['Exodus','Exo','28','043','43',"And they shall be upon Aaron, and upon his sons, when they come in unto the tabernacle of the congregation, or when they come near unto the altar to minister in the holy \emph{place}; that they bear not iniquity, and die: \emph{it} \emph{shall} \emph{be} a statute for ever unto him and his seed after him."]
]    

Exodus29 = [
    ['Exodus','Exo','29','001','1',"And this \emph{is} the thing that thou shalt do unto them to hallow them, to minister unto me in the priest's office: Take one young bullock, and two rams without blemish,"],
    ['Exodus','Exo','29','002','2',"And unleavened bread, and cakes unleavened tempered with oil, and wafers unleavened anointed with oil: \emph{of} wheaten flour shalt thou make them."],
    ['Exodus','Exo','29','003','3',"And thou shalt put them into one basket, and bring them in the basket, with the bullock and the two rams."],
    ['Exodus','Exo','29','004','4',"And Aaron and his sons thou shalt bring unto the door of the tabernacle of the congregation, and shalt wash them with water."],
    ['Exodus','Exo','29','005','5',"And thou shalt take the garments, and put upon Aaron the coat, and the robe of the ephod, and the ephod, and the breastplate, and gird him with the curious girdle of the ephod:"],
['Exodus','Exo','29','006','6',"And thou shalt put the mitre upon his head, and put the holy crown upon the mitre."],
['Exodus','Exo','29','007','7',"Then shalt thou take the anointing oil, and pour \emph{it} upon his head, and anoint him."],
['Exodus','Exo','29','008','8',"And thou shalt bring his sons, and put coats upon them."],
['Exodus','Exo','29','009','9',"And thou shalt gird them with girdles, Aaron and his sons, and put the bonnets on them: and the priest's office shall be their's for a perpetual statute: and thou shalt consecrate Aaron and his sons."],
['Exodus','Exo','29','010','10',"And thou shalt cause a bullock to be brought before the tabernacle of the congregation: and Aaron and his sons shall put their hands upon the head of the bullock."],
['Exodus','Exo','29','011','11',"And thou shalt kill the bullock before the LORD, \emph{by} the door of the tabernacle of the congregation."],
['Exodus','Exo','29','012','12',"And thou shalt take of the blood of the bullock, and put \emph{it} upon the horns of the altar with thy finger, and pour all the blood beside the bottom of the altar."],
['Exodus','Exo','29','013','13',"And thou shalt take all the fat that covereth the inwards, and the caul \emph{that} \emph{is} above the liver, and the two kidneys, and the fat that \emph{is} upon them, and burn \emph{them} upon the altar."],
['Exodus','Exo','29','014','14',"But the flesh of the bullock, and his skin, and his dung, shalt thou burn with fire without the camp: it \emph{is} a sin offering."],
['Exodus','Exo','29','015','15',"Thou shalt also take one ram; and Aaron and his sons shall put their hands upon the head of the ram."],
['Exodus','Exo','29','016','16',"And thou shalt slay the ram, and thou shalt take his blood, and sprinkle \emph{it} round about upon the altar."],
['Exodus','Exo','29','017','17',"And thou shalt cut the ram in pieces, and wash the inwards of him, and his legs, and put \emph{them} unto his pieces, and unto his head."],
['Exodus','Exo','29','018','18',"And thou shalt burn the whole ram upon the altar: it \emph{is} a burnt offering unto the LORD: it \emph{is} a sweet savour, an offering made by fire unto the LORD."],
['Exodus','Exo','29','019','19',"And thou shalt take the other ram; and Aaron and his sons shall put their hands upon the head of the ram."],
['Exodus','Exo','29','020','20',"Then shalt thou kill the ram, and take of his blood, and put \emph{it} upon the tip of the right ear of Aaron, and upon the tip of the right ear of his sons, and upon the thumb of their right hand, and upon the great toe of their right foot, and sprinkle the blood upon the altar round about."],
['Exodus','Exo','29','021','21',"And thou shalt take of the blood that \emph{is} upon the altar, and of the anointing oil, and sprinkle \emph{it} upon Aaron, and upon his garments, and upon his sons, and upon the garments of his sons with him: and he shall be hallowed, and his garments, and his sons, and his sons' garments with him."],
['Exodus','Exo','29','022','22',"Also thou shalt take of the ram the fat and the rump, and the fat that covereth the inwards, and the caul \emph{above} the liver, and the two kidneys, and the fat that \emph{is} upon them, and the right shoulder; for it \emph{is} a ram of consecration:"],
['Exodus','Exo','29','023','23',"And one loaf of bread, and one cake of oiled bread, and one wafer out of the basket of the unleavened bread that \emph{is} before the LORD:"],
['Exodus','Exo','29','024','24',"And thou shalt put all in the hands of Aaron, and in the hands of his sons; and shalt wave them \emph{for} a wave offering before the LORD."],
['Exodus','Exo','29','025','25',"And thou shalt receive them of their hands, and burn \emph{them} upon the altar for a burnt offering, for a sweet savour before the LORD: it \emph{is} an offering made by fire unto the LORD."],
['Exodus','Exo','29','026','26',"And thou shalt take the breast of the ram of Aaron's consecration, and wave it \emph{for} a wave offering before the LORD: and it shall be thy part."],
['Exodus','Exo','29','027','27',"And thou shalt sanctify the breast of the wave offering, and the shoulder of the heave offering, which is waved, and which is heaved up, of the ram of the consecration, \emph{even} of \emph{that} which \emph{is} for Aaron, and of \emph{that} which is for his sons:"],
['Exodus','Exo','29','028','28',"And it shall be Aaron's and his sons' by a statute for ever from the children of Israel: for it \emph{is} an heave offering: and it shall be an heave offering from the children of Israel of the sacrifice of their peace offerings, \emph{even} their heave offering unto the LORD."],
['Exodus','Exo','29','029','29',"And the holy garments of Aaron shall be his sons' after him, to be anointed therein, and to be consecrated in them."],
['Exodus','Exo','29','030','30',"\emph{And} that son that is priest in his stead shall put them on seven days, when he cometh into the tabernacle of the congregation to minister in the holy \emph{place}."],
['Exodus','Exo','29','031','31',"And thou shalt take the ram of the consecration, and seethe his flesh in the holy place."],
['Exodus','Exo','29','032','32',"And Aaron and his sons shall eat the flesh of the ram, and the bread that \emph{is} in the basket, \emph{by} the door of the tabernacle of the congregation."],
['Exodus','Exo','29','033','33',"And they shall eat those things wherewith the atonement was made, to consecrate \emph{and} to sanctify them: but a stranger shall not eat \emph{thereof}, because they \emph{are} holy."],
['Exodus','Exo','29','034','34',"And if ought of the flesh of the consecrations, or of the bread, remain unto the morning, then thou shalt burn the remainder with fire: it shall not be eaten, because it \emph{is} holy."],
['Exodus','Exo','29','035','35',"And thus shalt thou do unto Aaron, and to his sons, according to all \emph{things} which I have commanded thee: seven days shalt thou consecrate them."],
['Exodus','Exo','29','036','36',"And thou shalt offer every day a bullock \emph{for} a sin offering for atonement: and thou shalt cleanse the altar, when thou hast made an atonement for it, and thou shalt anoint it, to sanctify it."],
['Exodus','Exo','29','037','37',"Seven days thou shalt make an atonement for the altar, and sanctify it; and it shall be an altar most holy: whatsoever toucheth the altar shall be holy."],
['Exodus','Exo','29','038','38',"Now this \emph{is} \emph{that} which thou shalt offer upon the altar; two lambs of the first year day by day continually."],
['Exodus','Exo','29','039','39',"The one lamb thou shalt offer in the morning; and the other lamb thou shalt offer at even:"],
['Exodus','Exo','29','040','40',"And with the one lamb a tenth deal of flour mingled with the fourth part of an hin of beaten oil; and the fourth part of an hin of wine \emph{for} a drink offering."],
['Exodus','Exo','29','041','41',"And the other lamb thou shalt offer at even, and shalt do thereto according to the meat offering of the morning, and according to the drink offering thereof, for a sweet savour, an offering made by fire unto the LORD."],
    ['Exodus','Exo','29','042','42',"\emph{This} \emph{shall} \emph{be} a continual burnt offering throughout your generations \emph{at} the door of the tabernacle of the congregation before the LORD: where I will meet you, to speak there unto thee."],
    ['Exodus','Exo','29','043','43',"And there I will meet with the children of Israel, and \emph{the} \emph{tabernacle} shall be sanctified by my glory."],
    ['Exodus','Exo','29','044','44',"And I will sanctify the tabernacle of the congregation, and the altar: I will sanctify also both Aaron and his sons, to minister to me in the priest's office."],
    ['Exodus','Exo','29','045','45',"And I will dwell among the children of Israel, and will be their God."],
    ['Exodus','Exo','29','046','46',"And they shall know that I \emph{am} the LORD their God, that brought them forth out of the land of Egypt, that I may dwell among them: I \emph{am} the LORD their God."]
]     
     
Exodus30 = [
    ['Exodus','Exo','30','001','1',"And thou shalt make an altar to burn incense upon: \emph{of} shittim wood shalt thou make it."],
    ['Exodus','Exo','30','002','2',"A cubit \emph{shall} \emph{be} the length thereof, and a cubit the breadth thereof; foursquare shall it be: and two cubits \emph{shall} \emph{be} the height thereof: the horns thereof \emph{shall} \emph{be} of the same."],
    ['Exodus','Exo','30','003','3',"And thou shalt overlay it with pure gold, the top thereof, and the sides thereof round about, and the horns thereof; and thou shalt make unto it a crown of gold round about."],
    ['Exodus','Exo','30','004','4',"And two golden rings shalt thou make to it under the crown of it, by the two corners thereof, upon the two sides of it shalt thou make \emph{it}; and they shall be for places for the staves to bear it withal."],
    ['Exodus','Exo','30','005','5',"And thou shalt make the staves \emph{of} shittim wood, and overlay them with gold."],
['Exodus','Exo','30','006','6',"And thou shalt put it before the vail that \emph{is} by the ark of the testimony, before the mercy seat that \emph{is} over the testimony, where I will meet with thee."],
['Exodus','Exo','30','007','7',"And Aaron shall burn thereon sweet incense every morning: when he dresseth the lamps, he shall burn incense upon it."],
['Exodus','Exo','30','008','8',"And when Aaron lighteth the lamps at even, he shall burn incense upon it, a perpetual incense before the LORD throughout your generations."],
['Exodus','Exo','30','009','9',"Ye shall offer no strange incense thereon, nor burnt sacrifice, nor meat offering; neither shall ye pour drink offering thereon."],
['Exodus','Exo','30','010','10',"And Aaron shall make an atonement upon the horns of it once in a year with the blood of the sin offering of atonements: once in the year shall he make atonement upon it throughout your generations: it \emph{is} most holy unto the LORD."],
['Exodus','Exo','30','011','11',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','30','012','12',"When thou takest the sum of the children of Israel after their number, then shall they give every man a ransom for his soul unto the LORD, when thou numberest them; that there be no plague among them, when \emph{thou} numberest them."],
['Exodus','Exo','30','013','13',"This they shall give, every one that passeth among them that are numbered, half a shekel after the shekel of the sanctuary: (a shekel \emph{is} twenty gerahs:) an half shekel \emph{shall} \emph{be} the offering of the LORD."],
['Exodus','Exo','30','014','14',"Every one that passeth among them that are numbered, from twenty years old and above, shall give an offering unto the LORD."],
['Exodus','Exo','30','015','15',"The rich shall not give more, and the poor shall not give less than half a shekel, when \emph{they} give an offering unto the LORD, to make an atonement for your souls."],
['Exodus','Exo','30','016','16',"And thou shalt take the atonement money of the children of Israel, and shalt appoint it for the service of the tabernacle of the congregation; that it may be a memorial unto the children of Israel before the LORD, to make an atonement for your souls."],
['Exodus','Exo','30','017','17',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','30','018','18',"Thou shalt also make a laver \emph{of} brass, and his foot \emph{also} \emph{of} brass, to wash \emph{withal}: and thou shalt put it between the tabernacle of the congregation and the altar, and thou shalt put water therein."],
['Exodus','Exo','30','019','19',"For Aaron and his sons shall wash their hands and their feet thereat:"],
['Exodus','Exo','30','020','20',"When they go into the tabernacle of the congregation, they shall wash with water, that they die not; or when they come near to the altar to minister, to burn offering made by fire unto the LORD:"],
['Exodus','Exo','30','021','21',"So they shall wash their hands and their feet, that they die not: and it shall be a statute for ever to them, \emph{even} to him and to his seed throughout their generations."],
['Exodus','Exo','30','022','22',"Moreover the LORD spake unto Moses, saying,"],
['Exodus','Exo','30','023','23',"Take thou also unto thee principal spices, of pure myrrh five hundred \emph{shekels}, and of sweet cinnamon half so much, \emph{even} two hundred and fifty \emph{shekels}, and of sweet calamus two hundred and fifty \emph{shekels},"],
['Exodus','Exo','30','024','24',"And of cassia five hundred \emph{shekels}, after the shekel of the sanctuary, and of oil olive an hin:"],
['Exodus','Exo','30','025','25',"And thou shalt make it an oil of holy ointment, an ointment compound after the art of the apothecary: it shall be an holy anointing oil."],
['Exodus','Exo','30','026','26',"And thou shalt anoint the tabernacle of the congregation therewith, and the ark of the testimony,"],
['Exodus','Exo','30','027','27',"And the table and all his vessels, and the candlestick and his vessels, and the altar of incense,"],
['Exodus','Exo','30','028','28',"And the altar of burnt offering with all his vessels, and the laver and his foot."],
['Exodus','Exo','30','029','29',"And thou shalt sanctify them, that they may be most holy: whatsoever toucheth them shall be holy."],
['Exodus','Exo','30','030','30',"And thou shalt anoint Aaron and his sons, and consecrate them, that \emph{they} may minister unto me in the priest's office."],
['Exodus','Exo','30','031','31',"And thou shalt speak unto the children of Israel, saying, This shall be an holy anointing oil unto me throughout your generations."],
['Exodus','Exo','30','032','32',"Upon man's flesh shall it not be poured, neither shall ye make \emph{any} \emph{other} like it, after the composition of it: it \emph{is} holy, \emph{and} it shall be holy unto you."],
['Exodus','Exo','30','033','33',"Whosoever compoundeth \emph{any} like it, or whosoever putteth \emph{any} of it upon a stranger, shall even be cut off from his people."],
    ['Exodus','Exo','30','034','34',"And the LORD said unto Moses, Take unto thee sweet spices, stacte, and onycha, and galbanum; \emph{these} sweet spices with pure frankincense: of each shall there be a like \emph{weight}:"],
    ['Exodus','Exo','30','035','35',"And thou shalt make it a perfume, a confection after the art of the apothecary, tempered together, pure \emph{and} holy:"],
    ['Exodus','Exo','30','036','36',"And thou shalt beat \emph{some} of it very small, and put of it before the testimony in the tabernacle of the congregation, where I will meet with thee: it shall be unto you most holy."],
    ['Exodus','Exo','30','037','37',"And \emph{as} \emph{for} the perfume which thou shalt make, ye shall not make to yourselves according to the composition thereof: it shall be unto thee holy for the LORD."],
    ['Exodus','Exo','30','038','38',"Whosoever shall make like unto that, to smell thereto, shall even be cut off from his people."]
]

Exodus31 = [
    ['Exodus','Exo','31','001','1',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','31','002','2',"See, I have called by name Bezaleel the son of Uri, the son of Hur, of the tribe of Judah:"],
['Exodus','Exo','31','003','3',"And I have filled him with the spirit of God, in wisdom, and in understanding, and in knowledge, and in all manner of workmanship,"],
['Exodus','Exo','31','004','4',"To devise cunning works, to work in gold, and in silver, and in brass,"],
['Exodus','Exo','31','005','5',"And in cutting of stones, to set \emph{them}, and in carving of timber, to work in all manner of workmanship."],
['Exodus','Exo','31','006','6',"And I, behold, I have given with him Aholiab, the son of Ahisamach, of the tribe of Dan: and in the hearts of all that are wise hearted I have put wisdom, that they may make all that I have commanded thee;"],
['Exodus','Exo','31','007','7',"The tabernacle of the congregation, and the ark of the testimony, and the mercy seat that \emph{is} thereupon, and all the furniture of the tabernacle,"],
['Exodus','Exo','31','008','8',"And the table and his furniture, and the pure candlestick with all his furniture, and the altar of incense,"],
['Exodus','Exo','31','009','9',"And the altar of burnt offering with all his furniture, and the laver and his foot,"],
['Exodus','Exo','31','010','10',"And the cloths of service, and the holy garments for Aaron the priest, and the garments of his sons, to minister in the priest's office,"],
['Exodus','Exo','31','011','11',"And the anointing oil, and sweet incense for the holy \emph{place}: according to all that I have commanded thee shall they do."],
['Exodus','Exo','31','012','12',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','31','013','13',"Speak thou also unto the children of Israel, saying, Verily my sabbaths ye shall keep: for it \emph{is} a sign between me and you throughout your generations; that \emph{ye} may know that I \emph{am} the LORD that doth sanctify you."],
['Exodus','Exo','31','014','14',"Ye shall keep the sabbath therefore; for it \emph{is} holy unto you: every one that defileth it shall surely be put to death: for whosoever doeth \emph{any} work therein, that soul shall be cut off from among his people."],
['Exodus','Exo','31','015','15',"Six days may work be done; but in the seventh \emph{is} the sabbath of rest, holy to the LORD: whosoever doeth \emph{any} work in the sabbath day, he shall surely be put to death."],
['Exodus','Exo','31','016','16',"Wherefore the children of Israel shall keep the sabbath, to observe the sabbath throughout their generations, \emph{for} a perpetual covenant."],
['Exodus','Exo','31','017','17',"It \emph{is} a sign between me and the children of Israel for ever: for \emph{in} six days the LORD made heaven and earth, and on the seventh day he rested, and was refreshed."],
['Exodus','Exo','31','018','18',"And he gave unto Moses, when he had made an end of communing with him upon mount Sinai, two tables of testimony, tables of stone, written with the finger of God."]
]    
    
Exodus32 = [
    ['Exodus','Exo','32','001','1',"And when the people saw that Moses delayed to come down out of the mount, the people gathered themselves together unto Aaron, and said unto him, Up, make us gods, which shall go before us; for \emph{as} \emph{for} this Moses, the man that brought us up out of the land of Egypt, we wot not what is become of him."],
['Exodus','Exo','32','002','2',"And Aaron said unto them, Break off the golden earrings, which \emph{are} in the ears of your wives, of your sons, and of your daughters, and bring \emph{them} unto me."],
['Exodus','Exo','32','003','3',"And all the people brake off the golden earrings which \emph{were} in their ears, and brought \emph{them} unto Aaron."],
['Exodus','Exo','32','004','4',"And he received \emph{them} at their hand, and fashioned it with a graving tool, after he had made it a molten calf: and they said, These \emph{be} thy gods, O Israel, which brought thee up out of the land of Egypt."],
['Exodus','Exo','32','005','5',"And when Aaron saw \emph{it}, he built an altar before it; and Aaron made proclamation, and said, To morrow \emph{is} a feast to the LORD."],
['Exodus','Exo','32','006','6',"And they rose up early on the morrow, and offered burnt offerings, and brought peace offerings; and the people sat down to eat and to drink, and rose up to play."],
['Exodus','Exo','32','007','7',"And the LORD said unto Moses, Go, get thee down; for thy people, which thou broughtest out of the land of Egypt, have corrupted \emph{themselves}:"],
['Exodus','Exo','32','008','8',"They have turned aside quickly out of the way which I commanded them: they have made them a molten calf, and have worshipped it, and have sacrificed thereunto, and said, These \emph{be} thy gods, O Israel, which have brought thee up out of the land of Egypt."],
['Exodus','Exo','32','009','9',"And the LORD said unto Moses, I have seen this people, and, behold, it \emph{is} a stiffnecked people:"],
['Exodus','Exo','32','010','10',"Now therefore let me alone, that my wrath may wax hot against them, and that I may consume them: and I will make of thee a great nation."],
['Exodus','Exo','32','011','11',"And Moses besought the LORD his God, and said, LORD, why doth thy wrath wax hot against thy people, which thou hast brought forth out of the land of Egypt with great power, and with a mighty hand?"],
['Exodus','Exo','32','012','12',"Wherefore should the Egyptians speak, and say, For mischief did he bring them out, to slay them in the mountains, and to consume them from the face of the earth? Turn from thy fierce wrath, and repent of this evil against thy people."],
['Exodus','Exo','32','013','13',"Remember Abraham, Isaac, and Israel, thy servants, to whom thou swarest by thine own self, and saidst unto them, I will multiply your seed as the stars of heaven, and all this land that I have spoken of will I give unto your seed, and they shall inherit \emph{it} for ever."],
['Exodus','Exo','32','014','14',"And the LORD repented of the evil which he thought to do unto his people."],
['Exodus','Exo','32','015','15',"And Moses turned, and went down from the mount, and the two tables of the testimony \emph{were} in his hand: the tables \emph{were} written on both their sides; on the one side and on the other \emph{were} they written."],
['Exodus','Exo','32','016','16',"And the tables \emph{were} the work of God, and the writing \emph{was} the writing of God, graven upon the tables."],
['Exodus','Exo','32','017','17',"And when Joshua heard the noise of the people as they shouted, he said unto Moses, \emph{There} \emph{is} a noise of war in the camp."],
['Exodus','Exo','32','018','18',"And he said, \emph{It} \emph{is} not the voice of \emph{them} \emph{that} shout for mastery, neither \emph{is} \emph{it} the voice of \emph{them} \emph{that} cry for being overcome: \emph{but} the noise of \emph{them} \emph{that} sing do I hear."],
['Exodus','Exo','32','019','19',"And it came to pass, as soon as he came nigh unto the camp, that he saw the calf, and the dancing: and Moses' anger waxed hot, and he cast the tables out of his hands, and brake them beneath the mount."],
['Exodus','Exo','32','020','20',"And he took the calf which they had made, and burnt \emph{it} in the fire, and ground \emph{it} to powder, and strawed \emph{it} upon the water, and made the children of Israel drink \emph{of} \emph{it}."],
['Exodus','Exo','32','021','21',"And Moses said unto Aaron, What did this people unto thee, that thou hast brought so great a sin upon them?"],
['Exodus','Exo','32','022','22',"And Aaron said, Let not the anger of my lord wax hot: thou knowest the people, that they \emph{are} \emph{set} on mischief."],
['Exodus','Exo','32','023','23',"For they said unto me, Make us gods, which shall go before us: for \emph{as} \emph{for} this Moses, the man that brought us up out of the land of Egypt, we wot not what is become of him."],
['Exodus','Exo','32','024','24',"And I said unto them, Whosoever hath any gold, let them break \emph{it} off. So they gave \emph{it} me: then I cast it into the fire, and there came out this calf."],
['Exodus','Exo','32','025','25',"And when Moses saw that the people \emph{were} naked; (for Aaron had made them naked unto \emph{their} shame among their enemies:)"],
['Exodus','Exo','32','026','26',"Then Moses stood in the gate of the camp, and said, Who \emph{is} on the LORD'S side? \emph{let} \emph{him} \emph{come} unto me. And all the sons of Levi gathered themselves together unto him."],
['Exodus','Exo','32','027','27',"And he said unto them, Thus saith the LORD God of Israel, Put every man his sword by his side, \emph{and} go in and out from gate to gate throughout the camp, and slay every man his brother, and every man his companion, and every man his neighbour."],
['Exodus','Exo','32','028','28',"And the children of Levi did according to the word of Moses: and there fell of the people that day about three thousand men."],
['Exodus','Exo','32','029','29',"For Moses had said, Consecrate yourselves to day to the LORD, even every man upon his son, and upon his brother; that he may bestow upon you a blessing this day."],
['Exodus','Exo','32','030','30',"And it came to pass on the morrow, that Moses said unto the people, Ye have sinned a great sin: and now I will go up unto the LORD; peradventure I shall make an atonement for your sin."],
['Exodus','Exo','32','031','31',"And Moses returned unto the LORD, and said, Oh, this people have sinned a great sin, and have made them gods of gold."],
['Exodus','Exo','32','032','32',"Yet now, if thou wilt forgive their sin--; and if not, blot me, I pray thee, out of thy book which thou hast written."],
['Exodus','Exo','32','033','33',"And the LORD said unto Moses, Whosoever hath sinned against me, him will I blot out of my book."],
['Exodus','Exo','32','034','34',"Therefore now go, lead the people unto \emph{the} \emph{place} of which I have spoken unto thee: behold, mine Angel shall go before thee: nevertheless in the day when I visit I will visit their sin upon them."],
['Exodus','Exo','32','035','35',"And the LORD plagued the people, because they made the calf, which Aaron made."]
]

Exodus33 = [
    ['Exodus','Exo','33','001','1',"And the LORD said unto Moses, Depart, \emph{and} go up hence, thou and the people which thou hast brought up out of the land of Egypt, unto the land which I sware unto Abraham, to Isaac, and to Jacob, saying, Unto thy seed will I give it:"],
['Exodus','Exo','33','002','2',"And I will send an angel before thee; and I will drive out the Canaanite, the Amorite, and the Hittite, and the Perizzite, the Hivite, and the Jebusite:"],
['Exodus','Exo','33','003','3',"Unto a land flowing with milk and honey: for I will not go up in the midst of thee; for thou \emph{art} a stiffnecked people: lest I consume thee in the way."],
['Exodus','Exo','33','004','4',"And when the people heard these evil tidings, they mourned: and no man did put on him his ornaments."],
['Exodus','Exo','33','005','5',"For the LORD had said unto Moses, Say unto the children of Israel, Ye \emph{are} a stiffnecked people: I will come up into the midst of thee in a moment, and consume thee: therefore now put off thy ornaments from thee, that I may know what to do unto thee."],
['Exodus','Exo','33','006','6',"And the children of Israel stripped themselves of their ornaments by the mount Horeb."],
['Exodus','Exo','33','007','7',"And Moses took the tabernacle, and pitched it without the camp, afar off from the camp, and called it the Tabernacle of the congregation. And it came to pass, \emph{that} every one which sought the LORD went out unto the tabernacle of the congregation, which \emph{was} without the camp."],
['Exodus','Exo','33','008','8',"And it came to pass, when Moses went out unto the tabernacle, \emph{that} all the people rose up, and stood every man \emph{at} his tent door, and looked after Moses, until he was gone into the tabernacle."],
['Exodus','Exo','33','009','9',"And it came to pass, as Moses entered into the tabernacle, the cloudy pillar descended, and stood \emph{at} the door of the tabernacle, and \emph{the} \emph{LORD} talked with Moses."],
['Exodus','Exo','33','010','10',"And all the people saw the cloudy pillar stand \emph{at} the tabernacle door: and all the people rose up and worshipped, every man \emph{in} his tent door."],
['Exodus','Exo','33','011','11',"And the LORD spake unto Moses face to face, as a man speaketh unto his friend. And he turned again into the camp: but his servant Joshua, the son of Nun, a young man, departed not out of the tabernacle."],
['Exodus','Exo','33','012','12',"And Moses said unto the LORD, See, thou sayest unto me, Bring up this people: and thou hast not let me know whom thou wilt send with me. Yet thou hast said, I know thee by name, and thou hast also found grace in my sight."],
['Exodus','Exo','33','013','13',"Now therefore, I pray thee, if I have found grace in thy sight, shew me now thy way, that I may know thee, that I may find grace in thy sight: and consider that this nation \emph{is} thy people."],
['Exodus','Exo','33','014','14',"And he said, My presence shall go \emph{with} \emph{thee}, and I will give thee rest."],
['Exodus','Exo','33','015','15',"And he said unto him, If thy presence go not \emph{with} \emph{me}, carry us not up hence."],
['Exodus','Exo','33','016','16',"For wherein shall it be known here that I and thy people have found grace in thy sight? \emph{is} \emph{it} not in that thou goest with us? so shall we be separated, I and thy people, from all the people that \emph{are} upon the face of the earth."],
['Exodus','Exo','33','017','17',"And the LORD said unto Moses, I will do this thing also that thou hast spoken: for thou hast found grace in my sight, and I know thee by name."],
['Exodus','Exo','33','018','18',"And he said, I beseech thee, shew me thy glory."],
['Exodus','Exo','33','019','19',"And he said, I will make all my goodness pass before thee, and I will proclaim the name of the LORD before thee; and will be gracious to whom I will be gracious, and will shew mercy on whom I will shew mercy."],
['Exodus','Exo','33','020','20',"And he said, Thou canst not see my face: for there shall no man see me, and live."],
['Exodus','Exo','33','021','21',"And the LORD said, Behold, \emph{there} \emph{is} a place by me, and thou shalt stand upon a rock:"],
['Exodus','Exo','33','022','22',"And it shall come to pass, while my glory passeth by, that I will put thee in a clift of the rock, and will cover thee with my hand while I pass by:"],
['Exodus','Exo','33','023','23',"And I will take away mine hand, and thou shalt see my back parts: but my face shall not be seen."]
]

Exodus34 = [
    ['Exodus','Exo','34','001','1',"And the LORD said unto Moses, Hew thee two tables of stone like unto the first: and I will write upon \emph{these} tables the words that were in the first tables, which thou brakest."],
    ['Exodus','Exo','34','002','2',"And be ready in the morning, and come up in the morning unto mount Sinai, and present thyself there to me in the top of the mount."],
    ['Exodus','Exo','34','003','3',"And no man shall come up with thee, neither let any man be seen throughout all the mount; neither let the flocks nor herds feed before that mount."],
    ['Exodus','Exo','34','004','4',"And he hewed two tables of stone like unto the first; and Moses rose up early in the morning, and went up unto mount Sinai, as the LORD had commanded him, and took in his hand the two tables of stone."],
    ['Exodus','Exo','34','005','5',"And the LORD descended in the cloud, and stood with him there, and proclaimed the name of the LORD."],
    ['Exodus','Exo','34','006','6',"And the LORD passed by before him, and proclaimed, The LORD, The LORD God, merciful and gracious, longsuffering, and abundant in goodness and truth,"],
    ['Exodus','Exo','34','007','7',"Keeping mercy for thousands, forgiving iniquity and transgression and sin, and that will by no means clear \emph{the} \emph{guilty}; visiting the iniquity of the fathers upon the children, and upon the children's children, unto the third and to the fourth \emph{generation}."],
    ['Exodus','Exo','34','008','8',"And Moses made haste, and bowed his head toward the earth, and worshipped."],
    ['Exodus','Exo','34','009','9',"And he said, If now I have found grace in thy sight, O Lord, let my Lord, I pray thee, go among us; for it \emph{is} a stiffnecked people; and pardon our iniquity and our sin, and take us for thine inheritance."],
    ['Exodus','Exo','34','010','10',"And he said, Behold, I make a covenant: before all thy people I will do marvels, such as have not been done in all the earth, nor in any nation: and all the people among which thou \emph{art} shall see the work of the LORD: for it \emph{is} a terrible thing that I will do with thee."],
['Exodus','Exo','34','011','11',"Observe thou that which I command thee this day: behold, I drive out before thee the Amorite, and the Canaanite, and the Hittite, and the Perizzite, and the Hivite, and the Jebusite."],
['Exodus','Exo','34','012','12',"Take heed to thyself, lest thou make a covenant with the inhabitants of the land whither thou goest, lest it be for a snare in the midst of thee:"],
['Exodus','Exo','34','013','13',"But ye shall destroy their altars, break their images, and cut down their groves:"],
['Exodus','Exo','34','014','14',"For thou shalt worship no other god: for the LORD, whose name \emph{is} Jealous, \emph{is} a jealous God:"],
['Exodus','Exo','34','015','15',"Lest thou make a covenant with the inhabitants of the land, and they go a whoring after their gods, and do sacrifice unto their gods, and \emph{one} call thee, and thou eat of his sacrifice;"],
['Exodus','Exo','34','016','16',"And thou take of their daughters unto thy sons, and their daughters go a whoring after their gods, and make thy sons go a whoring after their gods."],
['Exodus','Exo','34','017','17',"Thou shalt make thee no molten gods."],
['Exodus','Exo','34','018','18',"The feast of unleavened bread shalt thou keep. Seven days thou shalt eat unleavened bread, as I commanded thee, in the time of the month Abib: for in the month Abib thou camest out from Egypt."],
['Exodus','Exo','34','019','19',"All that openeth the matrix \emph{is} mine; and every firstling among thy cattle, \emph{whether} ox or sheep, \emph{that} \emph{is} \emph{male}."],
['Exodus','Exo','34','020','20',"But the firstling of an ass thou shalt redeem with a lamb: and if thou redeem \emph{him} not, then shalt thou break his neck. All the firstborn of thy sons thou shalt redeem. And none shall appear before me empty."],
['Exodus','Exo','34','021','21',"Six days thou shalt work, but on the seventh day thou shalt rest: in earing time and in harvest thou shalt rest."],
['Exodus','Exo','34','022','22',"And thou shalt observe the feast of weeks, of the firstfruits of wheat harvest, and the feast of ingathering at the year's end."],
['Exodus','Exo','34','023','23',"Thrice in the year shall all your men children appear before the Lord GOD, the God of Israel."],
['Exodus','Exo','34','024','24',"For I will cast out the nations before thee, and enlarge thy borders: neither shall any man desire thy land, when thou shalt go up to appear before the LORD thy God thrice in the year."],
['Exodus','Exo','34','025','25',"Thou shalt not offer the blood of my sacrifice with leaven; neither shall the sacrifice of the feast of the passover be left unto the morning."],
    ['Exodus','Exo','34','026','26',"The first of the firstfruits of thy land thou shalt bring unto the house of the LORD thy God. Thou shalt not seethe a kid in his mother's milk."],
    ['Exodus','Exo','34','027','27',"And the LORD said unto Moses, Write thou these words: for after the tenor of these words I have made a covenant with thee and with Israel."],
    ['Exodus','Exo','34','028','28',"And he was there with the LORD forty days and forty nights; he did neither eat bread, nor drink water. And he wrote upon the tables the words of the covenant, the ten commandments."],
    ['Exodus','Exo','34','029','29',"And it came to pass, when Moses came down from mount Sinai with the two tables of testimony in Moses' hand, when he came down from the mount, that Moses wist not that the skin of his face shone while he talked with him."],
    ['Exodus','Exo','34','030','30',"And when Aaron and all the children of Israel saw Moses, behold, the skin of his face shone; and they were afraid to come nigh him."],
    ['Exodus','Exo','34','031','31',"And Moses called unto them; and Aaron and all the rulers of the congregation returned unto him: and Moses talked with them."],
    ['Exodus','Exo','34','032','32',"And afterward all the children of Israel came nigh: and he gave them in commandment all that the LORD had spoken with him in mount Sinai."],
    ['Exodus','Exo','34','033','33',"And \emph{till} Moses had done speaking with them, he put a vail on his face."],
    ['Exodus','Exo','34','034','34',"But when Moses went in before the LORD to speak with him, he took the vail off, until he came out. And he came out, and spake unto the children of Israel \emph{that} which he was commanded."],
    ['Exodus','Exo','34','035','35',"And the children of Israel saw the face of Moses, that the skin of Moses' face shone: and Moses put the vail upon his face again, until he went in to speak with him."]
]   

Exodus35 = [
    ['Exodus','Exo','35','001','1',"And Moses gathered all the congregation of the children of Israel together, and said unto them, These \emph{are} the words which the LORD hath commanded, that \emph{ye} should do them."],
    ['Exodus','Exo','35','002','2',"Six days shall work be done, but on the seventh day there shall be to you an holy day, a sabbath of rest to the LORD: whosoever doeth work therein shall be put to death."],
    ['Exodus','Exo','35','003','3',"Ye shall kindle no fire throughout your habitations upon the sabbath day."],
    ['Exodus','Exo','35','004','4',"And Moses spake unto all the congregation of the children of Israel, saying, This \emph{is} the thing which the LORD commanded, saying,"],
    ['Exodus','Exo','35','005','5',"Take ye from among you an offering unto the LORD: whosoever \emph{is} of a willing heart, let him bring it, an offering of the LORD; gold, and silver, and brass,"],
['Exodus','Exo','35','006','6',"And blue, and purple, and scarlet, and fine linen, and goats' \emph{hair},"],
['Exodus','Exo','35','007','7',"And rams' skins dyed red, and badgers' skins, and shittim wood,"],
['Exodus','Exo','35','008','8',"And oil for the light, and spices for anointing oil, and for the sweet incense,"],
['Exodus','Exo','35','009','9',"And onyx stones, and stones to be set for the ephod, and for the breastplate."],
['Exodus','Exo','35','010','10',"And every wise hearted among you shall come, and make all that the LORD hath commanded;"],
['Exodus','Exo','35','011','11',"The tabernacle, his tent, and his covering, his taches, and his boards, his bars, his pillars, and his sockets,"],
['Exodus','Exo','35','012','12',"The ark, and the staves thereof, \emph{with} the mercy seat, and the vail of the covering,"],
['Exodus','Exo','35','013','13',"The table, and his staves, and all his vessels, and the shewbread,"],
['Exodus','Exo','35','014','14',"The candlestick also for the light, and his furniture, and his lamps, with the oil for the light,"],
['Exodus','Exo','35','015','15',"And the incense altar, and his staves, and the anointing oil, and the sweet incense, and the hanging for the door at the entering in of the tabernacle,"],
['Exodus','Exo','35','016','16',"The altar of burnt offering, with his brasen grate, his staves, and all his vessels, the laver and his foot,"],
['Exodus','Exo','35','017','17',"The hangings of the court, his pillars, and their sockets, and the hanging for the door of the court,"],
['Exodus','Exo','35','018','18',"The pins of the tabernacle, and the pins of the court, and their cords,"],
['Exodus','Exo','35','019','19',"The cloths of service, to do service in the holy \emph{place}, the holy garments for Aaron the priest, and the garments of his sons, to minister in the priest's office."],
['Exodus','Exo','35','020','20',"And all the congregation of the children of Israel departed from the presence of Moses."],
['Exodus','Exo','35','021','21',"And they came, every one whose heart stirred him up, and every one whom his spirit made willing, \emph{and} they brought the LORD'S offering to the work of the tabernacle of the congregation, and for all his service, and for the holy garments."],
['Exodus','Exo','35','022','22',"And they came, both men and women, as many as were willing hearted, \emph{and} brought bracelets, and earrings, and rings, and tablets, all jewels of gold: and every man that offered \emph{offered} an offering of gold unto the LORD."],
['Exodus','Exo','35','023','23',"And every man, with whom was found blue, and purple, and scarlet, and fine linen, and goats' \emph{hair}, and red skins of rams, and badgers' skins, brought \emph{them}."],
['Exodus','Exo','35','024','24',"Every one that did offer an offering of silver and brass brought the LORD'S offering: and every man, with whom was found shittim wood for any work of the service, brought \emph{it}."],
['Exodus','Exo','35','025','25',"And all the women that were wise hearted did spin with their hands, and brought that which they had spun, \emph{both} of blue, and of purple, \emph{and} of scarlet, and of fine linen."],
['Exodus','Exo','35','026','26',"And all the women whose heart stirred them up in wisdom spun goats' \emph{hair}."],
['Exodus','Exo','35','027','27',"And the rulers brought onyx stones, and stones to be set, for the ephod, and for the breastplate;"],
['Exodus','Exo','35','028','28',"And spice, and oil for the light, and for the anointing oil, and for the sweet incense."],
['Exodus','Exo','35','029','29',"The children of Israel brought a willing offering unto the LORD, every man and woman, whose heart made them willing to bring for all manner of work, which the LORD had commanded to be made by the hand of Moses."],
['Exodus','Exo','35','030','30',"And Moses said unto the children of Israel, See, the LORD hath called by name Bezaleel the son of Uri, the son of Hur, of the tribe of Judah;"],
    ['Exodus','Exo','35','031','31',"And he hath filled him with the spirit of God, in wisdom, in understanding, and in knowledge, and in all manner of workmanship;"],
    ['Exodus','Exo','35','032','32',"And to devise curious works, to work in gold, and in silver, and in brass,"],
    ['Exodus','Exo','35','033','33',"And in the cutting of stones, to set \emph{them}, and in carving of wood, to make any manner of cunning work."],
    ['Exodus','Exo','35','034','34',"And he hath put in his heart that he may teach, \emph{both} he, and Aholiab, the son of Ahisamach, of the tribe of Dan."],
    ['Exodus','Exo','35','035','35',"Them hath he filled with wisdom of heart, to work all manner of work, of the engraver, and of the cunning workman, and of the embroiderer, in blue, and in purple, in scarlet, and in fine linen, and of the weaver, \emph{even} of them that do any work, and of those that devise cunning work."]
]    

Exodus36 = [
    ['Exodus','Exodus','36','01','1',"Then wrought Bezaleel and Aholiab, and every wise hearted man, in whom the LORD put wisdom and understanding to know how to work all manner of work for the service of the sanctuary, according to all that the LORD had commanded."],
    ['Exodus','Exodus','36','02','2',"And Moses called Bezaleel and Aholiab, and every wise hearted man, in whose heart the LORD had put wisdom, \emph{even} every one whose heart stirred him up to come unto the work to do it:"],
    ['Exodus','Exodus','36','03','3',"And they received of Moses all the offering, which the children of Israel had brought for the work of the service of the sanctuary, to make it \emph{withal}. And they brought yet unto him free offerings every morning."],
    ['Exodus','Exodus','36','04','4',"And all the wise men, that wrought all the work of the sanctuary, came every man from his work which they made;"],
    ['Exodus','Exodus','36','05','5',"And they spake unto Moses, saying, The people bring much more than enough for the service of the work, which the LORD commanded to make."],
    ['Exodus','Exodus','36','06','6',"And Moses gave commandment, and they caused it to be proclaimed throughout the camp, saying, Let neither man nor woman make any more work for the offering of the sanctuary. So the people were restrained from bringing."],
    ['Exodus','Exodus','36','07','7',"For the stuff they had was sufficient for all the work to make it, and too much."],
    ['Exodus','Exodus','36','08','8',"And every wise hearted man among them that wrought the work of the tabernacle made ten curtains \emph{of} fine twined linen, and blue, and purple, and scarlet: \emph{with} cherubims of cunning work made he them."],
    ['Exodus','Exodus','36','09','9',"The length of one curtain \emph{was} twenty and eight cubits, and the breadth of one curtain four cubits: the curtains \emph{were} all of one size."],
    ['Exodus','Exodus','36','10','10',"And he coupled the five curtains one unto another: and \emph{the} \emph{other} five curtains he coupled one unto another."],
    ['Exodus','Exodus','36','11','11',"And he made loops of blue on the edge of one curtain from the selvedge in the coupling: likewise he made in the uttermost side of \emph{another} curtain, in the coupling of the second."],
    ['Exodus','Exodus','36','12','12',"Fifty loops made he in one curtain, and fifty loops made he in the edge of the curtain which \emph{was} in the coupling of the second: the loops held one \emph{curtain} to another."],
    ['Exodus','Exodus','36','13','13',"And he made fifty taches of gold, and coupled the curtains one unto another with the taches: so it became one tabernacle."],
    ['Exodus','Exodus','36','14','14',"And he made curtains \emph{of} goats' \emph{hair} for the tent over the tabernacle: eleven curtains he made them."],
    ['Exodus','Exodus','36','15','15',"The length of one curtain \emph{was} thirty cubits, and four cubits \emph{was} the breadth of one curtain: the eleven curtains \emph{were} of one size."],
    ['Exodus','Exodus','36','16','16',"And he coupled five curtains by themselves, and six curtains by themselves."],
    ['Exodus','Exodus','36','17','17',"And he made fifty loops upon the uttermost edge of the curtain in the coupling, and fifty loops made he upon the edge of the curtain which coupleth the second.v"],
    ['Exodus','Exodus','36','18','18',"And he made fifty taches \emph{of} brass to couple the tent together, that it might be one."],
    ['Exodus','Exodus','36','19','19',"And he made a covering for the tent \emph{of} rams' skins dyed red, and a covering \emph{of} badgers' skins above \emph{that}."],
    ['Exodus','Exodus','36','20','20',"And he made boards for the tabernacle \emph{of} shittim wood, standing up."],
    ['Exodus','Exodus','36','21','21',"The length of a board \emph{was} ten cubits, and the breadth of a board one cubit and a half."],
    ['Exodus','Exodus','36','22','22',"One board had two tenons, equally distant one from another: thus did he make for all the boards of the tabernacle."],
    ['Exodus','Exodus','36','23','23',"And he made boards for the tabernacle; twenty boards for the south side southward:"],
    ['Exodus','Exodus','36','24','24',"And forty sockets of silver he made under the twenty boards; two sockets under one board for his two tenons, and two sockets under another board for his two tenons."],
    ['Exodus','Exodus','36','25','25',"And for the other side of the tabernacle, \emph{which} \emph{is} toward the north corner, he made twenty boards,"],
    ['Exodus','Exodus','36','26','26',"And their forty sockets of silver; two sockets under one board, and two sockets under another board."],
    ['Exodus','Exodus','36','27','27',"And for the sides of the tabernacle westward he made six boards."],
    ['Exodus','Exodus','36','28','28',"And two boards made he for the corners of the tabernacle in the two sides."],
    ['Exodus','Exodus','36','29','29',"And they were coupled beneath, and coupled together at the head thereof, to one ring: thus he did to both of them in both the corners."],
    ['Exodus','Exodus','36','30','30',"And there were eight boards; and their sockets \emph{were} sixteen sockets of silver, under every board two sockets."],
    ['Exodus','Exodus','36','31','31',"And he made bars of shittim wood; five for the boards of the one side of the tabernacle,"],
    ['Exodus','Exodus','36','32','32',"And five bars for the boards of the other side of the tabernacle, and five bars for the boards of the tabernacle for the sides westward."],
    ['Exodus','Exodus','36','33','33',"And he made the middle bar to shoot through the boards from the one end to the other."],
    ['Exodus','Exodus','36','34','34',"And he overlaid the boards with gold, and made their rings \emph{of} gold \emph{to} \emph{be} places for the bars, and overlaid the bars with gold."],
    ['Exodus','Exodus','36','35','35',"And he made a vail \emph{of} blue, and purple, and scarlet, and fine twined linen: \emph{with} cherubims made he it of cunning work."],
    ['Exodus','Exodus','36','36','36',"And he made thereunto four pillars \emph{of} shittim \emph{wood}, and overlaid them with gold: their hooks \emph{were} \emph{of} gold; and he cast for them four sockets of silver."],
    ['Exodus','Exodus','36','37','37',"And he made an hanging for the tabernacle door \emph{of} blue, and purple, and scarlet, and fine twined linen, of needlework;"],
    ['Exodus','Exodus','36','38','38',"And the five pillars of it with their hooks: and he overlaid their chapiters and their fillets with gold: but their five sockets \emph{were} \emph{of} brass."]]    

Exodus37 = [
    ['Exodus','Exo','37','001','1',"And Bezaleel made the ark \emph{of} shittim wood: two cubits and a half \emph{was} the length of it, and a cubit and a half the breadth of it, and a cubit and a half the height of it:"],
['Exodus','Exo','37','002','2',"And he overlaid it with pure gold within and without, and made a crown of gold to it round about."],
['Exodus','Exo','37','003','3',"And he cast for it four rings of gold, \emph{to} \emph{be} \emph{set} by the four corners of it; even two rings upon the one side of it, and two rings upon the other side of it."],
['Exodus','Exo','37','004','4',"And he made staves \emph{of} shittim wood, and overlaid them with gold."],
['Exodus','Exo','37','005','5',"And he put the staves into the rings by the sides of the ark, to bear the ark."],
['Exodus','Exo','37','006','6',"And he made the mercy seat \emph{of} pure gold: two cubits and a half \emph{was} the length thereof, and one cubit and a half the breadth thereof."],
['Exodus','Exo','37','007','7',"And he made two cherubims \emph{of} gold, beaten out of one piece made he them, on the two ends of the mercy seat;"],
['Exodus','Exo','37','008','8',"One cherub on the end on this side, and another cherub on the \emph{other} end on that side: out of the mercy seat made he the cherubims on the two ends thereof."],
['Exodus','Exo','37','009','9',"And the cherubims spread out \emph{their} wings on high, \emph{and} covered with their wings over the mercy seat, with their faces one to another; \emph{even} to the mercy seatward were the faces of the cherubims."],
['Exodus','Exo','37','010','10',"And he made the table \emph{of} shittim wood: two cubits \emph{was} the length thereof, and a cubit the breadth thereof, and a cubit and a half the height thereof:"],
['Exodus','Exo','37','011','11',"And he overlaid it with pure gold, and made thereunto a crown of gold round about."],
['Exodus','Exo','37','012','12',"Also he made thereunto a border of an handbreadth round about; and made a crown of gold for the border thereof round about."],
['Exodus','Exo','37','013','13',"And he cast for it four rings of gold, and put the rings upon the four corners that \emph{were} in the four feet thereof."],
['Exodus','Exo','37','014','14',"Over against the border were the rings, the places for the staves to bear the table."],
['Exodus','Exo','37','015','15',"And he made the staves \emph{of} shittim wood, and overlaid them with gold, to bear the table."],
['Exodus','Exo','37','016','16',"And he made the vessels which \emph{were} upon the table, his dishes, and his spoons, and his bowls, and his covers to cover withal, \emph{of} pure gold."],
['Exodus','Exo','37','017','17',"And he made the candlestick \emph{of} pure gold: \emph{of} beaten work made he the candlestick; his shaft, and his branch, his bowls, his knops, and his flowers, were of the same:"],
['Exodus','Exo','37','018','18',"And six branches going out of the sides thereof; three branches of the candlestick out of the one side thereof, and three branches of the candlestick out of the other side thereof:"],
['Exodus','Exo','37','019','19',"Three bowls made after the fashion of almonds in one branch, a knop and a flower; and three bowls made like almonds in another branch, a knop and a flower: so throughout the six branches going out of the candlestick."],
['Exodus','Exo','37','020','20',"And in the candlestick \emph{were} four bowls made like almonds, his knops, and his flowers:"],
['Exodus','Exo','37','021','21',"And a knop under two branches of the same, and a knop under two branches of the same, and a knop under two branches of the same, according to the six branches going out of it."],
['Exodus','Exo','37','022','22',"Their knops and their branches were of the same: all of it \emph{was} one beaten work \emph{of} pure gold."],
['Exodus','Exo','37','023','23',"And he made his seven lamps, and his snuffers, and his snuffdishes, \emph{of} pure gold."],
['Exodus','Exo','37','024','24',"\emph{Of} a talent of pure gold made he it, and all the vessels thereof."],
['Exodus','Exo','37','025','25',"And he made the incense altar \emph{of} shittim wood: the length of it \emph{was} a cubit, and the breadth of it a cubit; \emph{it} \emph{was} foursquare; and two cubits \emph{was} the height of it; the horns thereof were of the same."],
['Exodus','Exo','37','026','26',"And he overlaid it with pure gold, \emph{both} the top of it, and the sides thereof round about, and the horns of it: also he made unto it a crown of gold round about."],
['Exodus','Exo','37','027','27',"And he made two rings of gold for it under the crown thereof, by the two corners of it, upon the two sides thereof, to be places for the staves to bear it withal."],
['Exodus','Exo','37','028','28',"And he made the staves \emph{of} shittim wood, and overlaid them with gold."],
['Exodus','Exo','37','029','29',"And he made the holy anointing oil, and the pure incense of sweet spices, according to the work of the apothecary."]
]

Exodus38 = [
    ['Exodus','Exo','38','001','1',"And he made the altar of burnt offering \emph{of} shittim wood: five cubits \emph{was} the length thereof, and five cubits the breadth thereof; \emph{it} \emph{was} foursquare; and three cubits the height thereof."],
['Exodus','Exo','38','002','2',"And he made the horns thereof on the four corners of it; the horns thereof were of the same: and he overlaid it with brass."],
['Exodus','Exo','38','003','3',"And he made all the vessels of the altar, the pots, and the shovels, and the basons, \emph{and} the fleshhooks, and the firepans: all the vessels thereof made he \emph{of} brass."],
['Exodus','Exo','38','004','4',"And he made for the altar a brasen grate of network under the compass thereof beneath unto the midst of it."],
['Exodus','Exo','38','005','5',"And he cast four rings for the four ends of the grate of brass, \emph{to} \emph{be} places for the staves."],
['Exodus','Exo','38','006','6',"And he made the staves \emph{of} shittim wood, and overlaid them with brass."],
['Exodus','Exo','38','007','7',"And he put the staves into the rings on the sides of the altar, to bear it withal; he made the altar hollow with boards."],
['Exodus','Exo','38','008','8',"And he made the laver \emph{of} brass, and the foot of it \emph{of} brass, of the lookingglasses of \emph{the} \emph{women} assembling, which assembled \emph{at} the door of the tabernacle of the congregation."],
['Exodus','Exo','38','009','9',"And he made the court: on the south side southward the hangings of the court \emph{were} \emph{of} fine twined linen, an hundred cubits:"],
['Exodus','Exo','38','010','10',"Their pillars \emph{were} twenty, and their brasen sockets twenty; the hooks of the pillars and their fillets \emph{were} \emph{of} silver."],
['Exodus','Exo','38','011','11',"And for the north side \emph{the} \emph{hangings} \emph{were} an hundred cubits, their pillars \emph{were} twenty, and their sockets of brass twenty; the hooks of the pillars and their fillets \emph{of} silver."],
['Exodus','Exo','38','012','12',"And for the west side \emph{were} hangings of fifty cubits, their pillars ten, and their sockets ten; the hooks of the pillars and their fillets \emph{of} silver."],
['Exodus','Exo','38','013','13',"And for the east side eastward fifty cubits."],
['Exodus','Exo','38','014','14',"The hangings of the one side \emph{of} \emph{the} \emph{gate} \emph{were} fifteen cubits; their pillars three, and their sockets three."],
['Exodus','Exo','38','015','15',"And for the other side of the court gate, on this hand and that hand, \emph{were} hangings of fifteen cubits; their pillars three, and their sockets three."],
['Exodus','Exo','38','016','16',"All the hangings of the court round about \emph{were} of fine twined linen."],
['Exodus','Exo','38','017','17',"And the sockets for the pillars \emph{were} \emph{of} brass; the hooks of the pillars and their fillets \emph{of} silver; and the overlaying of their chapiters \emph{of} silver; and all the pillars of the court \emph{were} filleted with silver."],
['Exodus','Exo','38','018','18',"And the hanging for the gate of the court \emph{was} needlework, \emph{of} blue, and purple, and scarlet, and fine twined linen: and twenty cubits \emph{was} the length, and the height in the breadth \emph{was} five cubits, answerable to the hangings of the court."],
['Exodus','Exo','38','019','19',"And their pillars \emph{were} four, and their sockets \emph{of} brass four; their hooks \emph{of} silver, and the overlaying of their chapiters and their fillets \emph{of} silver."],
['Exodus','Exo','38','020','20',"And all the pins of the tabernacle, and of the court round about, \emph{were} \emph{of} brass."],
['Exodus','Exo','38','021','21',"This is the sum of the tabernacle, \emph{even} of the tabernacle of testimony, as it was counted, according to the commandment of Moses, \emph{for} the service of the Levites, by the hand of Ithamar, son to Aaron the priest."],
['Exodus','Exo','38','022','22',"And Bezaleel the son of Uri, the son of Hur, of the tribe of Judah, made all that the LORD commanded Moses."],
['Exodus','Exo','38','023','23',"And with him \emph{was} Aholiab, son of Ahisamach, of the tribe of Dan, an engraver, and a cunning workman, and an embroiderer in blue, and in purple, and in scarlet, and fine linen."],
['Exodus','Exo','38','024','24',"All the gold that was occupied for the work in all the work of the holy \emph{place}, even the gold of the offering, was twenty and nine talents, and seven hundred and thirty shekels, after the shekel of the sanctuary."],
['Exodus','Exo','38','025','25',"And the silver of them that were numbered of the congregation \emph{was} an hundred talents, and a thousand seven hundred and threescore and fifteen shekels, after the shekel of the sanctuary:"],
['Exodus','Exo','38','026','26',"A bekah for every man, \emph{that} \emph{is}, half a shekel, after the shekel of the sanctuary, for every one that went to be numbered, from twenty years old and upward, for six hundred thousand and three thousand and five hundred and fifty \emph{men}."],
['Exodus','Exo','38','027','27',"And of the hundred talents of silver were cast the sockets of the sanctuary, and the sockets of the vail; an hundred sockets of the hundred talents, a talent for a socket."],
['Exodus','Exo','38','028','28',"And of the thousand seven hundred seventy and five shekels he made hooks for the pillars, and overlaid their chapiters, and filleted them."],
['Exodus','Exo','38','029','29',"And the brass of the offering \emph{was} seventy talents, and two thousand and four hundred shekels."],
['Exodus','Exo','38','030','30',"And therewith he made the sockets to the door of the tabernacle of the congregation, and the brasen altar, and the brasen grate for it, and all the vessels of the altar,"],
['Exodus','Exo','38','031','31',"And the sockets of the court round about, and the sockets of the court gate, and all the pins of the tabernacle, and all the pins of the court round about."]
]

Exodus39 = [
    ['Exodus','Exo','39','001','1',"And of the blue, and purple, and scarlet, they made cloths of service, to do service in the holy \emph{place}, and made the holy garments for Aaron; as the LORD commanded Moses."],
['Exodus','Exo','39','002','2',"And he made the ephod \emph{of} gold, blue, and purple, and scarlet, and fine twined linen."],
['Exodus','Exo','39','003','3',"And they did beat the gold into thin plates, and cut \emph{it} \emph{into} wires, to work \emph{it} in the blue, and in the purple, and in the scarlet, and in the fine linen, \emph{with} cunning work."],
['Exodus','Exo','39','004','4',"They made shoulderpieces for it, to couple \emph{it} together: by the two edges was it coupled together."],
['Exodus','Exo','39','005','5',"And the curious girdle of his ephod, that \emph{was} upon it, \emph{was} of the same, according to the work thereof; \emph{of} gold, blue, and purple, and scarlet, and fine twined linen; as the LORD commanded Moses."],
['Exodus','Exo','39','006','6',"And they wrought onyx stones inclosed in ouches of gold, graven, as signets are graven, with the names of the children of Israel."],
['Exodus','Exo','39','007','7',"And he put them on the shoulders of the ephod, \emph{that} \emph{they} \emph{should} \emph{be} stones for a memorial to the children of Israel; as the LORD commanded Moses."],
['Exodus','Exo','39','008','8',"And he made the breastplate \emph{of} cunning work, like the work of the ephod; \emph{of} gold, blue, and purple, and scarlet, and fine twined linen."],
['Exodus','Exo','39','009','9',"It was foursquare; they made the breastplate double: a span \emph{was} the length thereof, and a span the breadth thereof, \emph{being} doubled."],
['Exodus','Exo','39','010','10',"And they set in it four rows of stones: \emph{the} \emph{first} row \emph{was} a sardius, a topaz, and a carbuncle: this \emph{was} the first row."],
['Exodus','Exo','39','011','11',"And the second row, an emerald, a sapphire, and a diamond."],
['Exodus','Exo','39','012','12',"And the third row, a ligure, an agate, and an amethyst."],
['Exodus','Exo','39','013','13',"And the fourth row, a beryl, an onyx, and a jasper: \emph{they} \emph{were} inclosed in ouches of gold in their inclosings."],
['Exodus','Exo','39','014','14',"And the stones \emph{were} according to the names of the children of Israel, twelve, according to their names, \emph{like} the engravings of a signet, every one with his name, according to the twelve tribes."],
['Exodus','Exo','39','015','15',"And they made upon the breastplate chains at the ends, \emph{of} wreathen work \emph{of} pure gold."],
['Exodus','Exo','39','016','16',"And they made two ouches \emph{of} gold, and two gold rings; and put the two rings in the two ends of the breastplate."],
['Exodus','Exo','39','017','17',"And they put the two wreathen chains of gold in the two rings on the ends of the breastplate."],
['Exodus','Exo','39','018','18',"And the two ends of the two wreathen chains they fastened in the two ouches, and put them on the shoulderpieces of the ephod, before it."],
['Exodus','Exo','39','019','19',"And they made two rings of gold, and put \emph{them} on the two ends of the breastplate, upon the border of it, which \emph{was} on the side of the ephod inward."],
['Exodus','Exo','39','020','20',"And they made two \emph{other} golden rings, and put them on the two sides of the ephod underneath, toward the forepart of it, over against the \emph{other} coupling thereof, above the curious girdle of the ephod."],
['Exodus','Exo','39','021','21',"And they did bind the breastplate by his rings unto the rings of the ephod with a lace of blue, that it might be above the curious girdle of the ephod, and that the breastplate might not be loosed from the ephod; as the LORD commanded Moses."],
['Exodus','Exo','39','022','22',"And he made the robe of the ephod \emph{of} woven work, all \emph{of} blue."],
['Exodus','Exo','39','023','23',"And \emph{there} \emph{was} an hole in the midst of the robe, as the hole of an habergeon, \emph{with} a band round about the hole, that it should not rend."],
['Exodus','Exo','39','024','24',"And they made upon the hems of the robe pomegranates \emph{of} blue, and purple, and scarlet, \emph{and} twined \emph{linen}."],
['Exodus','Exo','39','025','25',"And they made bells \emph{of} pure gold, and put the bells between the pomegranates upon the hem of the robe, round about between the pomegranates;"],
['Exodus','Exo','39','026','26',"A bell and a pomegranate, a bell and a pomegranate, round about the hem of the robe to minister \emph{in}; as the LORD commanded Moses."],
['Exodus','Exo','39','027','27',"And they made coats \emph{of} fine linen \emph{of} woven work for Aaron, and for his sons,"],
['Exodus','Exo','39','028','28',"And a mitre \emph{of} fine linen, and goodly bonnets \emph{of} fine linen, and linen breeches \emph{of} fine twined linen,"],
['Exodus','Exo','39','029','29',"And a girdle \emph{of} fine twined linen, and blue, and purple, and scarlet, \emph{of} needlework; as the LORD commanded Moses."],
['Exodus','Exo','39','030','30',"And they made the plate of the holy crown \emph{of} pure gold, and wrote upon it a writing, \emph{like} \emph{to} the engravings of a signet, HOLINESS TO THE LORD."],
['Exodus','Exo','39','031','31',"And they tied unto it a lace of blue, to fasten \emph{it} on high upon the mitre; as the LORD commanded Moses."],
['Exodus','Exo','39','032','32',"Thus was all the work of the tabernacle of the tent of the congregation finished: and the children of Israel did according to all that the LORD commanded Moses, so did they."],
['Exodus','Exo','39','033','33',"And they brought the tabernacle unto Moses, the tent, and all his furniture, his taches, his boards, his bars, and his pillars, and his sockets,"],
['Exodus','Exo','39','034','34',"And the covering of rams' skins dyed red, and the covering of badgers' skins, and the vail of the covering,"],
['Exodus','Exo','39','035','35',"The ark of the testimony, and the staves thereof, and the mercy seat,"],
['Exodus','Exo','39','036','36',"The table, \emph{and} all the vessels thereof, and the shewbread,"],
['Exodus','Exo','39','037','37',"The pure candlestick, \emph{with} the lamps thereof, \emph{even} \emph{with} the lamps to be set in order, and all the vessels thereof, and the oil for light,"],
['Exodus','Exo','39','038','38',"And the golden altar, and the anointing oil, and the sweet incense, and the hanging for the tabernacle door,"],
['Exodus','Exo','39','039','39',"The brasen altar, and his grate of brass, his staves, and all his vessels, the laver and his foot,"],
['Exodus','Exo','39','040','40',"The hangings of the court, his pillars, and his sockets, and the hanging for the court gate, his cords, and his pins, and all the vessels of the service of the tabernacle, for the tent of the congregation,"],
['Exodus','Exo','39','041','41',"The cloths of service to do service in the holy \emph{place}, and the holy garments for Aaron the priest, and his sons' garments, to minister in the priest's office."],
['Exodus','Exo','39','042','42',"According to all that the LORD commanded Moses, so the children of Israel made all the work."],
['Exodus','Exo','39','043','43',"And Moses did look upon all the work, and, behold, they had done it as the LORD had commanded, even so had they done it: and Moses blessed them."]]    
Exodus40 = [
    ['Exodus','Exo','40','001','1',"And the LORD spake unto Moses, saying,"],
['Exodus','Exo','40','002','2',"On the first day of the first month shalt thou set up the tabernacle of the tent of the congregation."],
['Exodus','Exo','40','003','3',"And thou shalt put therein the ark of the testimony, and cover the ark with the vail."],
['Exodus','Exo','40','004','4',"And thou shalt bring in the table, and set in order the things that are to be set in order upon it; and thou shalt bring in the candlestick, and light the lamps thereof."],
['Exodus','Exo','40','005','5',"And thou shalt set the altar of gold for the incense before the ark of the testimony, and put the hanging of the door to the tabernacle."],
['Exodus','Exo','40','006','6',"And thou shalt set the altar of the burnt offering before the door of the tabernacle of the tent of the congregation."],
['Exodus','Exo','40','007','7',"And thou shalt set the laver between the tent of the congregation and the altar, and shalt put water therein."],
['Exodus','Exo','40','008','8',"And thou shalt set up the court round about, and hang up the hanging at the court gate."],
['Exodus','Exo','40','009','9',"And thou shalt take the anointing oil, and anoint the tabernacle, and all that \emph{is} therein, and shalt hallow it, and all the vessels thereof: and it shall be holy."],
['Exodus','Exo','40','010','10',"And thou shalt anoint the altar of the burnt offering, and all his vessels, and sanctify the altar: and it shall be an altar most holy."],
['Exodus','Exo','40','011','11',"And thou shalt anoint the laver and his foot, and sanctify it."],
['Exodus','Exo','40','012','12',"And thou shalt bring Aaron and his sons unto the door of the tabernacle of the congregation, and wash them with water."],
['Exodus','Exo','40','013','13',"And thou shalt put upon Aaron the holy garments, and anoint him, and sanctify him; that he may minister unto me in the priest's office."],
['Exodus','Exo','40','014','14',"And thou shalt bring his sons, and clothe them with coats:"],
['Exodus','Exo','40','015','15',"And thou shalt anoint them, as thou didst anoint their father, that they may minister unto me in the priest's office: for their anointing shall surely be an everlasting priesthood throughout their generations."],
['Exodus','Exo','40','016','16',"Thus did Moses: according to all that the LORD commanded him, so did he."],
['Exodus','Exo','40','017','17',"And it came to pass in the first month in the second year, on the first \emph{day} of the month, \emph{that} the tabernacle was reared up."],
['Exodus','Exo','40','018','18',"And Moses reared up the tabernacle, and fastened his sockets, and set up the boards thereof, and put in the bars thereof, and reared up his pillars."],
['Exodus','Exo','40','019','19',"And he spread abroad the tent over the tabernacle, and put the covering of the tent above upon it; as the LORD commanded Moses."],
['Exodus','Exo','40','020','20',"And he took and put the testimony into the ark, and set the staves on the ark, and put the mercy seat above upon the ark:"],
['Exodus','Exo','40','021','21',"And he brought the ark into the tabernacle, and set up the vail of the covering, and covered the ark of the testimony; as the LORD commanded Moses."],
['Exodus','Exo','40','022','22',"And he put the table in the tent of the congregation, upon the side of the tabernacle northward, without the vail."],
['Exodus','Exo','40','023','23',"And he set the bread in order upon it before the LORD; as the LORD had commanded Moses."],
['Exodus','Exo','40','024','24',"And he put the candlestick in the tent of the congregation, over against the table, on the side of the tabernacle southward."],
['Exodus','Exo','40','025','25',"And he lighted the lamps before the LORD; as the LORD commanded Moses."],
['Exodus','Exo','40','026','26',"And he put the golden altar in the tent of the congregation before the vail:"],
['Exodus','Exo','40','027','27',"And he burnt sweet incense thereon; as the LORD commanded Moses."],
['Exodus','Exo','40','028','28',"And he set up the hanging \emph{at} the door of the tabernacle."],
['Exodus','Exo','40','029','29',"And he put the altar of burnt offering \emph{by} the door of the tabernacle of the tent of the congregation, and offered upon it the burnt offering and the meat offering; as the LORD commanded Moses."],
['Exodus','Exo','40','030','30',"And he set the laver between the tent of the congregation and the altar, and put water there, to wash \emph{withal}."],
['Exodus','Exo','40','031','31',"And Moses and Aaron and his sons washed their hands and their feet thereat:"],
['Exodus','Exo','40','032','32',"When they went into the tent of the congregation, and when they came near unto the altar, they washed; as the LORD commanded Moses."],
['Exodus','Exo','40','033','33',"And he reared up the court round about the tabernacle and the altar, and set up the hanging of the court gate. So Moses finished the work."],
['Exodus','Exo','40','034','34',"Then a cloud covered the tent of the congregation, and the glory of the LORD filled the tabernacle."],
['Exodus','Exo','40','035','35',"And Moses was not able to enter into the tent of the congregation, because the cloud abode thereon, and the glory of the LORD filled the tabernacle."],
['Exodus','Exo','40','036','36',"And when the cloud was taken up from over the tabernacle, the children of Israel went onward in all their journeys:"],
['Exodus','Exo','40','037','37',"But if the cloud were not taken up, then they journeyed not till the day that it was taken up."],
['Exodus','Exo','40','038','38',"For the cloud of the LORD \emph{was} upon the tabernacle by day, and fire was on it by night, in the sight of all the house of Israel, throughout all their journeys."]
]    
    
    
     
def main():
    ProcessVerses('Exodus38',Exodus38)
      
if __name__ == "__main__":
    main()